In [1]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import os

In [3]:
# 配置API密钥（需替换为你的实际密钥）
os.environ["OPENAI_API_KEY"] = "sk-Gzo6arNf2T7NmrUYZUcxcQgRnIYclT5tqmgu9NfB34HfBkRd"
os.environ["OPENAI_API_BASE"] = "https://api2.aigcbest.top/v1"  # 必填代理地址
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
import os
# claude-opus-4-1-20250805
# claude-opus-4-1-thinking-all
# 初始化 GPT-4o 模型
model = ChatOpenAI(
    model="gemini-3-pro-preview-thinking",
    temperature=0.7,
)

# 创建对话历史记录
chat_history = []

def chat(user_input):
    # 添加用户消息到历史
    chat_history.append(HumanMessage(content=user_input))
    # 生成回复
    response = model.invoke(chat_history)
    # 提取并显示 AI 回复
    ai_reply = response.content
    # 保存 AI 消息到历史
    chat_history.append(AIMessage(content=ai_reply))
    print(ai_reply)
    return ai_reply

In [4]:
response = chat('''
我在用verl框架实现on policy distillation，目标如下
"""
1.student先根据query生成response，这里面肯定没有hack的问题，因为query里没有ground truth
2.把query + student response + ground truth做个摘要
3.用teacher mode重新算一遍student的response的概率，输入是 query + 上面的摘要 + student的response，这里不做生成，只前向传播计算概率
4.最小化student的response和第3步中的概率的reverse KL loss，注意这里不是用第3步中的所有概率，而是只取第3步中response部分的前向传播概率，所以要做截断。
"""
我写的teacher_student_reflective_trainer.py代码如下
"""
import torch
import numpy as np
from typing import List, Dict, Optional
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
import re
from verl import DataProto
from verl.trainer.ppo.ray_trainer import RayPPOTrainer, compute_response_mask, compute_advantage, Role
from verl.utils.metric import (
    reduce_metrics,
)
import uuid
from tqdm import tqdm
from verl.trainer.ppo.core_algos import AdvantageEstimator
from verl.utils.debug import marked_timer
from verl.utils.torch_functional import masked_mean
from verl.utils.tracking import Tracking
from omegaconf import OmegaConf, open_dict
from verl.single_controller.ray import RayClassWithInitArgs, RayResourcePool, RayWorkerGroup
from verl.single_controller.ray.base import create_colocated_worker_cls

class TeacherStudentReflectiveTrainer(RayPPOTrainer):
    """
    Teacher-Student Reflective Trainer.
    
    Architecture:
    - Student (Actor): Trainable. Generates Response.
    - Teacher (RefPolicy): Frozen. Generates Summary AND Computes LogProb.
    
    Resource Strategy:
    - Splits available GPUs into two pools (Student Pool & Teacher Pool) to allow 
      two simultaneous vLLM instances without conflict.
    """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        assert self.use_reference_policy, "TeacherStudentReflectiveTrainer requires a Reference Policy (Teacher)!"
        self.use_critic = False
        self.use_rm = False
        
        print(">>> TeacherStudentReflectiveTrainer Initialized.")
        print(">>> Strategy: Frozen Teacher generates Summary & Scores.")

    def init_workers(self):
        """
        Manually split resources and initialize two independent vLLM-enabled workers.
        """
        # 1. Calculate Resource Split
        # We assume homogeneous nodes. We split GPUs evenly between Student and Teacher.
        n_gpus_per_node = self.config.trainer.n_gpus_per_node
        n_nodes = self.config.trainer.nnodes
        
        if n_gpus_per_node < 2:
            raise ValueError(f"Need at least 2 GPUs per node to split Student/Teacher. Got {n_gpus_per_node}.")
            
        student_gpus = n_gpus_per_node // 2
        teacher_gpus = n_gpus_per_node - student_gpus
        
        print(f">>> Resource Splitting: Student={student_gpus} GPUs, Teacher={teacher_gpus} GPUs (per node)")

        # 2. Create Independent Resource Pools
        # Note: We create new RayResourcePools, ignoring the one passed from main.
        # This ensures Ray allocates non-overlapping GPUs to each pool.
        student_pool = RayResourcePool(
            process_on_nodes=[student_gpus] * n_nodes,
            use_gpu=True,
            max_colocate_count=1,
            name_prefix="student_pool"
        )
        
        teacher_pool = RayResourcePool(
            process_on_nodes=[teacher_gpus] * n_nodes,
            use_gpu=True,
            max_colocate_count=1,
            name_prefix="teacher_pool"
        )
        
        self.resource_pool_to_cls = {student_pool: {}, teacher_pool: {}}

        # 3. Initialize Student (Actor) Config
        # Deepcopy to prevent config pollution
        student_config = deepcopy(self.config.actor_rollout_ref)
        # Adjust micro_batch_size if necessary (optional, but good practice since world_size changed)
        # student_config.actor.ppo_mini_batch_size //= 2 # Logic handled by worker usually
        
        student_cls = RayClassWithInitArgs(
            cls=self.role_worker_mapping[Role.ActorRollout],
            config=student_config,
            role="actor_rollout",
            profile_option=self.config.trainer.npu_profile.options,
        )
        self.resource_pool_to_cls[student_pool]["actor_rollout"] = student_cls

        # 4. Initialize Teacher (Ref) Config
        # We define it as 'actor_rollout' role to FORCE vLLM initialization.
        teacher_config = deepcopy(self.config.actor_rollout_ref)
        
        teacher_cls = RayClassWithInitArgs(
            cls=self.role_worker_mapping[Role.RefPolicy],
            config=teacher_config,
            role="actor_rollout", # <--- Hack: Pretend to be Actor to enable vLLM
            profile_option=self.config.trainer.npu_profile.options,
        )
        self.resource_pool_to_cls[teacher_pool]["ref"] = teacher_cls

        # 5. Spawn Workers
        all_wg = {}
        wg_kwargs = {
            "ray_wait_register_center_timeout": self.config.trainer.ray_wait_register_center_timeout,
            "device_name": self.device_name
        }
        
        for resource_pool, class_dict in self.resource_pool_to_cls.items():
            worker_dict_cls = create_colocated_worker_cls(class_dict=class_dict)
            wg_dict = self.ray_worker_group_cls(
                resource_pool=resource_pool,
                ray_cls_with_init=worker_dict_cls,
                **wg_kwargs,
            )
            spawn_wg = wg_dict.spawn(prefix_set=class_dict.keys())
            all_wg.update(spawn_wg)

        # 6. Bind to Trainer
        self.actor_rollout_wg = all_wg["actor_rollout"]
        self.actor_rollout_wg.init_model()
        
        self.ref_policy_wg = all_wg["ref"]
        self.ref_policy_wg.init_model()
            
        # Async Rollout Manager (Only for Student)
        self.async_rollout_mode = False
        if self.config.actor_rollout_ref.rollout.mode == 'async':
            from verl.experimental.agent_loop import AgentLoopManager
            self.async_rollout_mode = True
            self.async_rollout_manager = AgentLoopManager(
                config=self.config,
                worker_group=self.actor_rollout_wg
            )

    def _prepare_summary_generation_batch(self, batch: DataProto, raw_prompts: List[str], ground_truths: List[str]) -> DataProto:
        """
        构造用于生成总结 (Reflection) 的 Batch。
        包含: 强力正则清洗，去除 System Prompt 和嵌套标记。
        """
        responses = batch.batch['responses']
        
        assert len(raw_prompts) == len(responses)
        assert len(ground_truths) == len(responses)

        input_ids_list = []
        attention_mask_list = []

        for i, (r_ids, gt_text) in enumerate(zip(responses, ground_truths)):
            p_text_dirty = raw_prompts[i]
            
            # --- 正则清洗逻辑 (提取纯 User Query) ---
            p_text_clean = p_text_dirty
            
            # 1. 尝试截取最后一个 User 标记之后的内容
            split_patterns = [
                r"<\|im_start\|>user\s*",  # Qwen/ChatML
                r"user\s*\n",              # Qwen decode
                r"User:\s*",               # Common
                r"\[INST\]\s*",            # Llama
                r"Human:\s*"               # Anthropic
            ]
            
            found_user = False
            for pattern in split_patterns:
                matches = list(re.finditer(pattern, p_text_dirty, re.IGNORECASE))
                if matches:
                    last_match = matches[-1]
                    p_text_clean = p_text_dirty[last_match.end():]
                    found_user = True
                    break
            
            # 2. 移除可能残留的 Assistant 标记 (Prompt 结尾)
            stop_patterns = [
                r"<\|im_start\|>assistant",
                r"assistant\s*\n",
                r"Assistant:",
                r"\[/INST\]"
            ]
            for pattern in stop_patterns:
                match = re.search(pattern, p_text_clean, re.IGNORECASE) # 使用 escape 防止正则错误
                if match:
                    p_text_clean = p_text_clean[:match.start()]
            
            # 3. 移除可能残留的 User 结束标记 (如 <|im_end|>)
            if "<|im_end|>" in p_text_clean:
                p_text_clean = p_text_clean.replace("<|im_end|>", "")
            
            p_text_clean = p_text_clean.strip()
            # ----------------------------------------

            r_text = self.tokenizer.decode(r_ids, skip_special_tokens=True)
            
            # Prompt for the Teacher (Summary Generation)
            content = (
                f"Question: {p_text_clean}\n\n"
                f"Standard Answer: {gt_text}\n\n"
                f"Student Answer: {r_text}\n\n"
                f"Task: Verify the Student Answer step-by-step. Is it correct?\n"
                f"Answer concisely."
            )

            messages = [
                {"role": "system", "content": "You are a helpful math assistant."},
                {"role": "user", "content": content}
            ]

            enc_ids = self.tokenizer.apply_chat_template(
                messages, 
                tokenize=True, 
                add_generation_prompt=True,
                return_tensors='pt'
            )
            
            if enc_ids.dim() == 2:
                enc_ids = enc_ids[0]

            input_ids_list.append(enc_ids)
            attention_mask_list.append(torch.ones_like(enc_ids))

        # --- Manual Left Padding & Position IDs ---
        max_len = max([len(t) for t in input_ids_list])
        pad_token_id = self.tokenizer.pad_token_id
        
        padded_input_ids = []
        padded_attention_mask = []
        padded_position_ids = []
        
        for ids, mask in zip(input_ids_list, attention_mask_list):
            pad_len = max_len - len(ids)
            
            pad_ids = torch.full((pad_len,), pad_token_id, dtype=ids.dtype, device=ids.device)
            pad_mask = torch.full((pad_len,), 0, dtype=mask.dtype, device=mask.device)
            
            final_ids = torch.cat([pad_ids, ids])
            final_mask = torch.cat([pad_mask, mask])
            
            padded_input_ids.append(final_ids)
            padded_attention_mask.append(final_mask)
            
            seq_len = len(ids)
            pos_content = torch.arange(seq_len, dtype=torch.long, device=ids.device)
            pos_pad = torch.zeros(pad_len, dtype=torch.long, device=ids.device)
            final_pos = torch.cat([pos_pad, pos_content])
            
            padded_position_ids.append(final_pos)
            
        input_ids = torch.stack(padded_input_ids)
        attention_mask = torch.stack(padded_attention_mask)
        position_ids = torch.stack(padded_position_ids)

        summary_batch = DataProto.from_dict({
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "position_ids": position_ids
        })
        
        summary_batch.meta_info = {
            "do_sample": True,
            "temperature": 0.1, 
            "max_new_tokens": 512,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id
        }
        
        return summary_batch

    def _prepare_teacher_forward_batch(self, batch: DataProto, summaries: torch.Tensor) -> DataProto:
        """
        构造 Teacher Forward 的 Batch。
        逻辑：[Question] + [Summary (Hint)] -> Teacher 预测 [Correct Response]
        实际输入：[Question] + [Summary] + [Student Response]
        
        如果 Student Response 是错的，Teacher (试图生成正确答案) 会给出低 LogProb。
        如果 Student Response 是对的，Teacher (试图生成正确答案) 会给出高 LogProb。
        """
        prompts = batch.batch['prompts']
        responses = batch.batch['responses']
        
        new_input_ids = []
        new_attention_mask = []
        new_response_masks = []
        new_prompts_list = []
        new_responses_list = []

        # 分隔符
        sep_ids = self.tokenizer.encode("\n\nSolution:\n", add_special_tokens=False)
        separator = torch.tensor(sep_ids, device=prompts.device)

        for i in range(len(prompts)):
            # 1. 解码 Prompt (包含 System + User Query)
            p_ids = prompts[i]
            p_ids = p_ids[p_ids != self.tokenizer.pad_token_id]
            p_text_dirty = self.tokenizer.decode(p_ids, skip_special_tokens=False)
            
            # --- 强力清洗逻辑 (只提取 User Query) ---
            p_text_clean = p_text_dirty
            # 尝试截取最后一个 User 标记之后的内容
            split_patterns = [r"<\|im_start\|>user\s*", r"user\s*\n", r"User:\s*", r"\[INST\]\s*", r"Human:\s*"]
            for pattern in split_patterns:
                matches = list(re.finditer(pattern, p_text_dirty, re.IGNORECASE))
                if matches:
                    p_text_clean = p_text_dirty[matches[-1].end():]
                    break
            
            stop_patterns = [r"<\|im_start\|>assistant", r"assistant\s*\n", r"Assistant:", r"\[/INST\]"]
            for pattern in stop_patterns:
                match = re.search(pattern, p_text_clean, re.IGNORECASE)
                if match:
                    p_text_clean = p_text_clean[:match.start()]
            
            if "<|im_end|>" in p_text_clean:
                p_text_clean = p_text_clean.replace("<|im_end|>", "")
            
            p_text_clean = p_text_clean.strip()
            # ----------------------------------------
            
            # 2. 获取 Summary (作为 Hint/Analysis)
            s_ids = summaries[i]
            s_ids = s_ids[s_ids != self.tokenizer.pad_token_id]
            s_text = self.tokenizer.decode(s_ids, skip_special_tokens=True)
            
            # 3. 获取 Student Response (Target)
            r_ids = responses[i]
            r_ids = r_ids[r_ids != self.tokenizer.pad_token_id]
            
            # 4. 构造 Teacher 的 Context (Corrective Guidance)
            # 这里的 Prompt 设计至关重要
            teacher_content = (
                f"Problem: {p_text_clean}\n\n"
                f"Analysis/Hint: {s_text}\n\n"
                f"Based on the analysis above, please provide the correct, step-by-step solution."
            )
            
            messages = [
                {"role": "system", "content": "You are a helpful math assistant."},
                {"role": "user", "content": teacher_content}
            ]
            
            # 5. Apply Chat Template
            prefix_ids = self.tokenizer.apply_chat_template(
                messages, 
                tokenize=True, 
                add_generation_prompt=True,
                return_tensors='pt'
            )[0].to(r_ids.device)
            
            # 6. 拼接: [Prefix] + [Separator] + [Student Response]
            # Teacher 看到的是：
            # User: Problem + Hint + "Please provide correct solution"
            # Assistant: [Student Response]
            
            # 如果 Student Response 是错的，Teacher 本来想生成对的，结果看到错的 -> LogProb 低
            
            # 注意：separator 可以直接拼在 prefix_ids 后面，或者作为 response 的开头
            # 这里我们不加额外的 separator，因为 apply_chat_template 已经加了 assistant 标签
            # 直接拼接 r_ids 即可
            
            teacher_full = torch.cat([prefix_ids, r_ids])
            
            # 7. Masks
            att_mask = torch.ones_like(teacher_full)
            resp_mask = torch.zeros_like(teacher_full)
            
            # 只计算 Response 部分的 LogProb
            start_idx = len(prefix_ids)
            resp_mask[start_idx:] = 1 
            
            new_input_ids.append(teacher_full)
            new_attention_mask.append(att_mask)
            new_response_masks.append(resp_mask)
            
            new_prompts_list.append(prefix_ids)
            new_responses_list.append(r_ids)

        # Padding
        input_ids = pad_sequence(new_input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attention_mask = pad_sequence(new_attention_mask, batch_first=True, padding_value=0)
        response_mask = pad_sequence(new_response_masks, batch_first=True, padding_value=0)
        
        prompts_padded = pad_sequence(new_prompts_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        responses_padded = pad_sequence(new_responses_list, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        
        position_ids = torch.cumsum(attention_mask, dim=-1) - 1
        position_ids.masked_fill_(attention_mask == 0, 0)

        teacher_batch = DataProto.from_dict({
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "position_ids": position_ids,
            "response_mask": response_mask,
            "prompts": prompts_padded,   
            "responses": responses_padded 
        })
        
        return teacher_batch

    def fit(self):
        logger = Tracking(
            project_name=self.config.trainer.project_name,
            experiment_name=self.config.trainer.experiment_name,
            default_backend=self.config.trainer.logger,
            config=OmegaConf.to_container(self.config, resolve=True),
        )

        self.global_steps = 0
        self._load_checkpoint()

        progress_bar = tqdm(total=self.total_training_steps, initial=self.global_steps, desc="Training Progress")
        self.global_steps += 1

        # === Sanity Check: Is Teacher Brain-Dead? ===
        print("Running Teacher Sanity Check...")
        test_prompts = ["1+1="]
        test_responses = ["2"]
        
        # 1. 构造单个样本
        p_ids = self.tokenizer.encode(test_prompts[0], return_tensors='pt')[0]
        r_ids = self.tokenizer.encode(test_responses[0], return_tensors='pt')[0]
        
        input_ids = torch.cat([p_ids, r_ids])
        attention_mask = torch.ones_like(input_ids)
        position_ids = torch.arange(len(input_ids))
        response_mask = torch.zeros_like(input_ids)
        response_mask[len(p_ids):] = 1
        
        # 2. 获取 Worker 数量 (World Size)
        # 我们需要让 batch size 能够被 world size 整除
        world_size = self.ref_policy_wg.world_size
        
        # 3. 复制数据以匹配 World Size
        input_ids = input_ids.unsqueeze(0).repeat(world_size, 1)
        attention_mask = attention_mask.unsqueeze(0).repeat(world_size, 1)
        position_ids = position_ids.unsqueeze(0).repeat(world_size, 1)
        response_mask = response_mask.unsqueeze(0).repeat(world_size, 1)
        
        # Prompts 和 Responses 在 DataProto 中主要是占位，但也需要对齐
        # 注意：DataProto.from_dict 对 tensor 的处理比较严格，这里我们只放必要的 keys
        # 为了避免复杂，我们只放 input_ids 等核心 tensor，prompts/responses 可以留空或者也 repeat
        # compute_log_prob 主要依赖 input_ids, attention_mask, position_ids, responses(用于mask)
        
        # 重新构造 responses (tensor)
        responses_tensor = r_ids.unsqueeze(0).repeat(world_size, 1)

        sanity_batch = DataProto.from_dict({
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "position_ids": position_ids,
            "response_mask": response_mask,
            "responses": responses_tensor # 必须有，用于 compute_log_prob 内部切片
        })
        
        # 补充 meta_info
        sanity_batch.meta_info = {
            "micro_batch_size": world_size, # 一次性跑完
            "temperature": 1.0,
            "use_dynamic_bsz": False
        }
        
        # Teacher Forward
        out = self.ref_policy_wg.compute_log_prob(sanity_batch)
        
        # 取第一个样本的结果
        log_prob = out.batch['old_log_probs'][0, 0].item() 
        
        print(f"Teacher Sanity Check: LogProb('2' | '1+1=') = {log_prob:.4f}")
        
        if log_prob < -5.0:
            print("!!! CRITICAL WARNING: Teacher LogProb is extremely low. Model might be randomly initialized!")
            # raise RuntimeError("Teacher Model is broken!")
        else:
            print("Teacher seems healthy.")
        # ============================================

        for epoch in range(self.config.trainer.total_epochs):
            for batch_dict in self.train_dataloader:
                metrics = {}
                timing_raw = {}

                batch: DataProto = DataProto.from_single_dict(batch_dict)
                
                # --- Step 1: Data Alignment ---
                B = batch.batch.batch_size[0]
                N = self.config.actor_rollout_ref.rollout.n
                
                raw_prompts_list = []
                for p_ids in batch.batch['input_ids']: 
                    raw_prompts_list.append(self.tokenizer.decode(p_ids, skip_special_tokens=True))
                
                ground_truths_list = []
                possible_gt_keys = ["ground_truth", "solution", "reward_model"]
                found_gt = False
                for key in possible_gt_keys:
                    if key in batch.non_tensor_batch:
                        data = batch.non_tensor_batch[key]
                        if key == "reward_model":
                            ground_truths_list = [item.get('ground_truth', '') if isinstance(item, dict) else '' for item in data]
                        else:
                            ground_truths_list = data.tolist() if isinstance(data, np.ndarray) else data
                        found_gt = True
                        break
                if not found_gt:
                    ground_truths_list = [""] * B

                expanded_raw_prompts = [p for p in raw_prompts_list for _ in range(N)]
                expanded_ground_truths = [g for g in ground_truths_list for _ in range(N)]
                
                batch_keys_to_pop = ['input_ids', 'attention_mask', 'position_ids']
                possible_non_tensor_keys = ['raw_prompt_ids', 'multi_modal_data', 'raw_prompt', 'tools_kwargs', 'interaction_kwargs', 'index', 'agent_name']
                non_tensor_batch_keys_to_pop = [k for k in possible_non_tensor_keys if k in batch.non_tensor_batch]
                
                gen_batch = batch.pop(batch_keys=batch_keys_to_pop, non_tensor_batch_keys=non_tensor_batch_keys_to_pop)
                gen_batch.meta_info['global_steps'] = self.global_steps
                gen_batch = gen_batch.repeat(repeat_times=N, interleave=True)

                # --- Step 2: Student Generation ---
                with marked_timer("gen_student", timing_raw):
                    if not self.async_rollout_mode:
                        gen_batch_output = self.actor_rollout_wg.generate_sequences(gen_batch)
                    else:
                        gen_batch_output = self.async_rollout_manager.generate_sequences(gen_batch)
                    
                    batch = batch.repeat(repeat_times=N, interleave=True)
                    batch = batch.union(gen_batch_output)

                batch.non_tensor_batch["uid"] = np.array(
                    [str(uuid.uuid4()) for _ in range(len(batch.batch))], dtype=object
                )
                if "response_mask" not in batch.batch.keys():
                    batch.batch["response_mask"] = compute_response_mask(batch)
                batch.meta_info['global_token_num'] = torch.sum(batch.batch['attention_mask'], dim=-1).tolist()

                # --- Step 3: Reflection & Reward ---
                with marked_timer("reward_reflection", timing_raw, color="yellow"):
                    # 3.1 Extract Prompts
                    tensor_prompts = batch.batch['prompts']
                    decoded_prompts = []
                    for p_ids in tensor_prompts:
                        decoded_prompts.append(self.tokenizer.decode(p_ids, skip_special_tokens=False))
                    
                    # 3.2 Extract GT
                    current_ground_truths = []
                    if "reward_model" in batch.non_tensor_batch:
                        rm_data = batch.non_tensor_batch["reward_model"]
                        current_ground_truths = [item.get('ground_truth', '') if isinstance(item, dict) else '' for item in rm_data]
                    elif "solution" in batch.non_tensor_batch:
                        current_ground_truths = batch.non_tensor_batch["solution"].tolist()
                    elif "ground_truth" in batch.non_tensor_batch:
                        current_ground_truths = batch.non_tensor_batch["ground_truth"].tolist()
                    else:
                        current_ground_truths = [""] * len(tensor_prompts)

                    # 3.3 Teacher Generates Summary
                    summary_input_batch = self._prepare_summary_generation_batch(
                        batch, 
                        decoded_prompts, 
                        current_ground_truths
                    )
                    
                    summary_output = self.actor_rollout_wg.generate_sequences(summary_input_batch)
                    summaries = summary_output.batch['responses']
                    
                    # 3.4 Teacher Computes LogProb
                    teacher_batch = self._prepare_teacher_forward_batch(batch, summaries)
                    teacher_log_prob_output = self.ref_policy_wg.compute_log_prob(teacher_batch)
                    teacher_full_log_probs = teacher_log_prob_output.batch['old_log_probs']

                    # 3.5 Student Computes LogProb
                    student_log_prob_output = self.actor_rollout_wg.compute_log_prob(batch)
                    student_full_log_probs = student_log_prob_output.batch['old_log_probs']
                    
                    # === Log Part 1: Generation Content ===
                    if True:
                        print(f"\n{'='*20} Teacher-Student Reflection Debug (Step {self.global_steps}) {'='*20}")
                        try:
                            idx = 0 
                            # Summary Input
                            sum_in_ids = summary_input_batch.batch['input_ids'][idx]
                            sum_in_ids = sum_in_ids[sum_in_ids != self.tokenizer.pad_token_id]
                            sum_in_text = self.tokenizer.decode(sum_in_ids, skip_special_tokens=False)
                            
                            # Summary Output
                            s_ids = summaries[idx]
                            s_ids = s_ids[s_ids != self.tokenizer.pad_token_id]
                            s_text = self.tokenizer.decode(s_ids, skip_special_tokens=True)
                            
                            # Teacher Input (Prompt + Summary + Response)
                            # 这里展示 Teacher LogProb 看到的完整输入
                            t_full_ids = teacher_batch.batch['input_ids'][idx]
                            t_full_ids = t_full_ids[t_full_ids != self.tokenizer.pad_token_id]
                            t_full_text = self.tokenizer.decode(t_full_ids, skip_special_tokens=False)

                            print(f"--- [0] Summary Generation Input ---\n{sum_in_text.strip()}\n")
                            print(f"--- [1] Summary Output ---\n{s_text.strip()}\n")
                            print(f"--- [2] Teacher LogProb Input (Full Context) ---\n{t_full_text.strip()}\n") # 只看最后500字符
                        except Exception as e:
                            print(f"Log Error 1: {e}")
                        print(f"{'='*60}\n")
                    # ======================================

                    # 3.6 Reward Calculation
                    token_level_rewards = torch.zeros_like(student_full_log_probs)
                    s_probs = student_full_log_probs
                    t_probs = teacher_full_log_probs
                    
                    min_len = min(s_probs.shape[1], t_probs.shape[1])
                    s_part = s_probs[:, :min_len]
                    t_part = t_probs[:, :min_len]
                    
                    kl_diff = t_part - s_part
                    
                    # === Log Part 2: Token-Level KL Analysis ===
                    if True:
                        print(f"\n{'='*20} Token-Level KL Analysis (Step {self.global_steps}) {'='*20}")
                        try:
                            idx = 0 
                            # 1. Student Response (Remove Pad)
                            s_ids = batch.batch['responses'][idx]
                            s_ids = s_ids[s_ids != self.tokenizer.pad_token_id]
                            
                            # 2. Teacher Response (Extract using Mask)
                            t_full_ids = teacher_batch.batch['input_ids'][idx]
                            t_mask = teacher_batch.batch['response_mask'][idx]
                            t_ids = t_full_ids[t_mask.bool()]
                            t_ids = t_ids[t_ids != self.tokenizer.pad_token_id]
                            
                            # 3. Decode
                            s_text_check = self.tokenizer.decode(s_ids, skip_special_tokens=False)
                            t_text_check = self.tokenizer.decode(t_ids, skip_special_tokens=False)
                            
                            print(f"--- Sequence Alignment Check ---")

                            
                            if len(s_ids) != len(t_ids) or not torch.equal(s_ids, t_ids):
                                print("[WARNING] ID Mismatch! Teacher sees different tokens than Student generated!")
                                print(f"S IDs: {s_ids.tolist()}")
                                print(f"T IDs: {t_ids.tolist()}")
                                print(f"Student Seq (Len={len(s_ids)}): {s_text_check}...")
                                print(f"Teacher Seq (Len={len(t_ids)}): {t_text_check}...")
                            else:
                                print("[OK] Token IDs match perfectly.")

                            print(f"\n--- Token-wise KL Breakdown (First 50 tokens) ---")
                            print(f"{'Token':<15} | {'ID':<6} | {'S_LogP':<8} | {'T_LogP':<8} | {'Diff':<8}")
                            print("-" * 75)
                            
                            s_vals = s_part[idx].tolist()
                            t_vals = t_part[idx].tolist()
                            diff_vals = kl_diff[idx].tolist()
                            
                            valid_count = 0
                            for i in range(len(s_ids)):
                                if valid_count >= 50: break
                                if i >= len(s_vals): break
                                
                                tid = s_ids[i].item()
                                token_str = self.tokenizer.decode([tid]).replace('\n', '\\n')
                                print(f"{token_str:<15} | {tid:<6} | {s_vals[i]:.2f}   | {t_vals[i]:.2f}   | {diff_vals[i]:.2f}")
                                valid_count += 1
                                
                        except Exception as e:
                            print(f"Log Error 2: {e}")
                        print(f"{'='*60}\n")
                    # ==========================================

                    kl_diff = torch.clamp(kl_diff, min=-1.0, max=1.0)
                    token_level_rewards[:, :min_len] = kl_diff
                    
                    batch.batch['token_level_rewards'] = token_level_rewards
                    
                    valid_mask = batch.batch['response_mask']
                    with torch.no_grad():
                        mean_kl = (token_level_rewards * valid_mask).sum() / (valid_mask.sum() + 1e-6)
                        metrics['reward/reflection_kl'] = mean_kl.item()

                # --- Step 4: PPO Flow ---
                entropys = student_log_prob_output.batch['entropys']
                entropy_agg = torch.mean(entropys)
                metrics['actor/entropy'] = entropy_agg.item()
                
                student_log_prob_output.batch.pop('entropys', None)
                batch = batch.union(student_log_prob_output)

                if "values" not in batch.batch.keys():
                    batch.batch["values"] = torch.zeros_like(batch.batch["token_level_rewards"])

                with marked_timer("adv", timing_raw):
                    batch = compute_advantage(
                        batch,
                        adv_estimator=self.config.algorithm.adv_estimator,
                        gamma=self.config.algorithm.gamma,
                        lam=self.config.algorithm.lam,
                        num_repeat=self.config.actor_rollout_ref.rollout.n,
                        norm_adv_by_std_in_grpo=self.config.algorithm.get('norm_adv_by_std_in_grpo', True),
                        config=self.config.algorithm
                    )
                
                batch.batch['advantages'] = batch.batch['token_level_rewards']

                with marked_timer("update_actor", timing_raw):
                    actor_output = self.actor_rollout_wg.update_actor(batch)
                    actor_output_metrics = reduce_metrics(actor_output.meta_info["metrics"])
                    metrics.update(actor_output_metrics)

                metrics.update({
                    "training/global_step": self.global_steps,
                    "training/epoch": epoch
                })
                
                logger.log(data=metrics, step=self.global_steps)
                progress_bar.update(1)
                self.global_steps += 1
                
                if self.config.trainer.save_freq > 0 and self.global_steps % self.config.trainer.save_freq == 0:
                    self._save_checkpoint()

                if self.global_steps >= self.total_training_steps:
                    progress_bar.close()
                    return
"""
ray_trainer.py的代码如下
"""
# Copyright 2024 Bytedance Ltd. and/or its affiliates
# Copyright 2023-2024 SGLang Team
# Copyright 2025 ModelBest Inc. and/or its affiliates
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
PPO Trainer with Ray-based single controller.
This trainer supports model-agonistic model initialization with huggingface
"""

import json
import os
import uuid
from collections import defaultdict
from copy import deepcopy
from dataclasses import dataclass, field
from enum import Enum
from pprint import pprint
from typing import Optional

import numpy as np
import ray
import torch
from omegaconf import OmegaConf, open_dict
from torch.utils.data import Dataset, Sampler
from torchdata.stateful_dataloader import StatefulDataLoader
from tqdm import tqdm

from verl import DataProto
from verl.experimental.dataset.sampler import AbstractCurriculumSampler
from verl.protocol import pad_dataproto_to_divisor, unpad_dataproto
from verl.single_controller.base import Worker
from verl.single_controller.ray import RayClassWithInitArgs, RayResourcePool, RayWorkerGroup
from verl.single_controller.ray.base import create_colocated_worker_cls
from verl.trainer.config import AlgoConfig
from verl.trainer.ppo import core_algos
from verl.trainer.ppo.core_algos import AdvantageEstimator, agg_loss
from verl.trainer.ppo.metric_utils import (
    compute_data_metrics,
    compute_throughout_metrics,
    compute_timing_metrics,
    process_validation_metrics,
)
from verl.trainer.ppo.reward import compute_reward, compute_reward_async
from verl.utils.checkpoint.checkpoint_manager import find_latest_ckpt_path, should_save_ckpt_esi
from verl.utils.debug import marked_timer
from verl.utils.metric import (
    reduce_metrics,
)
from verl.utils.seqlen_balancing import get_seqlen_balanced_partitions, log_seqlen_unbalance
from verl.utils.torch_functional import masked_mean
from verl.utils.tracking import ValidationGenerationsLogger

WorkerType = type[Worker]


class Role(Enum):
    """
    To create more roles dynamically, you can subclass Role and add new members
    """

    Actor = 0
    Rollout = 1
    ActorRollout = 2
    Critic = 3
    RefPolicy = 4
    RewardModel = 5
    ActorRolloutRef = 6


@dataclass
class ResourcePoolManager:
    """
    Define a resource pool specification. Resource pool will be initialized first.
    """

    resource_pool_spec: dict[str, list[int]]
    mapping: dict[Role, str]
    resource_pool_dict: dict[str, RayResourcePool] = field(default_factory=dict)

    def create_resource_pool(self):
        """Create Ray resource pools for distributed training.

        Initializes resource pools based on the resource pool specification,
        with each pool managing GPU resources across multiple nodes.
        For FSDP backend, uses max_colocate_count=1 to merge WorkerGroups.
        For Megatron backend, uses max_colocate_count>1 for different models.
        """
        for resource_pool_name, process_on_nodes in self.resource_pool_spec.items():
            # max_colocate_count means the number of WorkerGroups (i.e. processes) in each RayResourcePool
            # For FSDP backend, we recommend using max_colocate_count=1 that merge all WorkerGroups into one.
            # For Megatron backend, we recommend using max_colocate_count>1
            # that can utilize different WorkerGroup for differnt models
            resource_pool = RayResourcePool(
                process_on_nodes=process_on_nodes, use_gpu=True, max_colocate_count=1, name_prefix=resource_pool_name
            )
            self.resource_pool_dict[resource_pool_name] = resource_pool

        self._check_resource_available()

    def get_resource_pool(self, role: Role) -> RayResourcePool:
        """Get the resource pool of the worker_cls"""
        return self.resource_pool_dict[self.mapping[role]]

    def get_n_gpus(self) -> int:
        """Get the number of gpus in this cluster."""
        return sum([n_gpus for process_on_nodes in self.resource_pool_spec.values() for n_gpus in process_on_nodes])

    def _check_resource_available(self):
        """Check if the resource pool can be satisfied in this ray cluster."""
        node_available_resources = ray.state.available_resources_per_node()
        node_available_gpus = {
            node: node_info.get("GPU", 0) if "GPU" in node_info else node_info.get("NPU", 0)
            for node, node_info in node_available_resources.items()
        }

        # check total required gpus can be satisfied
        total_available_gpus = sum(node_available_gpus.values())
        total_required_gpus = sum(
            [n_gpus for process_on_nodes in self.resource_pool_spec.values() for n_gpus in process_on_nodes]
        )
        if total_available_gpus < total_required_gpus:
            raise ValueError(
                f"Total available GPUs {total_available_gpus} is less than total desired GPUs {total_required_gpus}"
            )

        # check each resource pool can be satisfied, O(#resource_pools * #nodes)
        for resource_pool_name, process_on_nodes in self.resource_pool_spec.items():
            num_gpus, num_nodes = process_on_nodes[0], len(process_on_nodes)
            for node, available_gpus in node_available_gpus.items():
                if available_gpus >= num_gpus:
                    node_available_gpus[node] -= num_gpus
                    num_nodes -= 1
                    if num_nodes == 0:
                        break
            if num_nodes > 0:
                raise ValueError(
                    f"Resource pool {resource_pool_name}: {num_gpus}*{num_nodes}"
                    + "cannot be satisfied in this ray cluster"
                )


def apply_kl_penalty(data: DataProto, kl_ctrl: core_algos.AdaptiveKLController, kl_penalty="kl"):
    """Apply KL penalty to the token-level rewards.

    This function computes the KL divergence between the reference policy and current policy,
    then applies a penalty to the token-level rewards based on this divergence.

    Args:
        data (DataProto): The data containing batched model outputs and inputs.
        kl_ctrl (core_algos.AdaptiveKLController): Controller for adaptive KL penalty.
        kl_penalty (str, optional): Type of KL penalty to apply. Defaults to "kl".
        multi_turn (bool, optional): Whether the data is from a multi-turn conversation. Defaults to False.

    Returns:
        tuple: A tuple containing:
            - The updated data with token-level rewards adjusted by KL penalty
            - A dictionary of metrics related to the KL penalty
    """
    response_mask = data.batch["response_mask"]
    token_level_scores = data.batch["token_level_scores"]
    batch_size = data.batch.batch_size[0]

    # compute kl between ref_policy and current policy
    # When apply_kl_penalty, algorithm.use_kl_in_reward=True, so the reference model has been enabled.
    kld = core_algos.kl_penalty(
        data.batch["old_log_probs"], data.batch["ref_log_prob"], kl_penalty=kl_penalty
    )  # (batch_size, response_length)
    kld = kld * response_mask
    beta = kl_ctrl.value

    token_level_rewards = token_level_scores - beta * kld

    current_kl = masked_mean(kld, mask=response_mask, axis=-1)  # average over sequence
    current_kl = torch.mean(current_kl, dim=0).item()

    # according to https://github.com/huggingface/trl/blob/951ca1841f29114b969b57b26c7d3e80a39f75a0/trl/trainer/ppo_trainer.py#L837
    kl_ctrl.update(current_kl=current_kl, n_steps=batch_size)
    data.batch["token_level_rewards"] = token_level_rewards

    metrics = {"actor/reward_kl_penalty": current_kl, "actor/reward_kl_penalty_coeff": beta}

    return data, metrics


def compute_response_mask(data: DataProto):
    """Compute the attention mask for the response part of the sequence.

    This function extracts the portion of the attention mask that corresponds to the model's response,
    which is used for masking computations that should only apply to response tokens.

    Args:
        data (DataProto): The data containing batched model outputs and inputs.

    Returns:
        torch.Tensor: The attention mask for the response tokens.
    """
    responses = data.batch["responses"]
    response_length = responses.size(1)
    attention_mask = data.batch["attention_mask"]
    return attention_mask[:, -response_length:]


def compute_advantage(
    data: DataProto,
    adv_estimator: AdvantageEstimator,
    gamma: float = 1.0,
    lam: float = 1.0,
    num_repeat: int = 1,
    norm_adv_by_std_in_grpo: bool = True,
    config: Optional[AlgoConfig] = None,
) -> DataProto:
    """Compute advantage estimates for policy optimization.

    This function computes advantage estimates using various estimators like GAE, GRPO, REINFORCE++, etc.
    The advantage estimates are used to guide policy optimization in RL algorithms.

    Args:
        data (DataProto): The data containing batched model outputs and inputs.
        adv_estimator (AdvantageEstimator): The advantage estimator to use (e.g., GAE, GRPO, REINFORCE++).
        gamma (float, optional): Discount factor for future rewards. Defaults to 1.0.
        lam (float, optional): Lambda parameter for GAE. Defaults to 1.0.
        num_repeat (int, optional): Number of times to repeat the computation. Defaults to 1.
        norm_adv_by_std_in_grpo (bool, optional): Whether to normalize advantages by standard deviation in
            GRPO. Defaults to True.
        config (dict, optional): Configuration dictionary for algorithm settings. Defaults to None.

    Returns:
        DataProto: The updated data with computed advantages and returns.
    """
    # Back-compatible with trainers that do not compute response mask in fit
    if "response_mask" not in data.batch.keys():
        data.batch["response_mask"] = compute_response_mask(data)
    # prepare response group
    if adv_estimator == AdvantageEstimator.GAE:
        # Compute advantages and returns using Generalized Advantage Estimation (GAE)
        advantages, returns = core_algos.compute_gae_advantage_return(
            token_level_rewards=data.batch["token_level_rewards"],
            values=data.batch["values"],
            response_mask=data.batch["response_mask"],
            gamma=gamma,
            lam=lam,
        )
        data.batch["advantages"] = advantages
        data.batch["returns"] = returns
        if config.get("use_pf_ppo", False):
            data = core_algos.compute_pf_ppo_reweight_data(
                data,
                config.pf_ppo.reweight_method,
                config.pf_ppo.weight_pow,
            )
    elif adv_estimator == AdvantageEstimator.GRPO:
        # Initialize the mask for GRPO calculation
        grpo_calculation_mask = data.batch["response_mask"]
        # Call compute_grpo_outcome_advantage with parameters matching its definition
        advantages, returns = core_algos.compute_grpo_outcome_advantage(
            token_level_rewards=data.batch["token_level_rewards"],
            response_mask=grpo_calculation_mask,
            index=data.non_tensor_batch["uid"],
            norm_adv_by_std_in_grpo=norm_adv_by_std_in_grpo,
        )
        data.batch["advantages"] = advantages
        data.batch["returns"] = returns
    else:
        # handle all other adv estimator type other than GAE and GRPO
        adv_estimator_fn = core_algos.get_adv_estimator_fn(adv_estimator)
        adv_kwargs = {
            "token_level_rewards": data.batch["token_level_rewards"],
            "response_mask": data.batch["response_mask"],
            "config": config,
        }
        if "uid" in data.non_tensor_batch:  # optional
            adv_kwargs["index"] = data.non_tensor_batch["uid"]
        if "reward_baselines" in data.batch:  # optional
            adv_kwargs["reward_baselines"] = data.batch["reward_baselines"]

        # calculate advantage estimator
        advantages, returns = adv_estimator_fn(**adv_kwargs)
        data.batch["advantages"] = advantages
        data.batch["returns"] = returns
    return data


class RayPPOTrainer:
    """Distributed PPO trainer using Ray for scalable reinforcement learning.

    This trainer orchestrates distributed PPO training across multiple nodes and GPUs,
    managing actor rollouts, critic training, and reward computation with Ray backend.
    Supports various model architectures including FSDP, Megatron, and vLLM integration.
    """

    # TODO: support each role have individual ray_worker_group_cls,
    # i.e., support different backend of different role
    def __init__(
        self,
        config,
        tokenizer,
        role_worker_mapping: dict[Role, WorkerType],
        resource_pool_manager: ResourcePoolManager,
        ray_worker_group_cls: RayWorkerGroup = RayWorkerGroup,
        processor=None,
        reward_fn=None,
        val_reward_fn=None,
        train_dataset: Optional[Dataset] = None,
        val_dataset: Optional[Dataset] = None,
        collate_fn=None,
        train_sampler: Optional[Sampler] = None,
        device_name=None,
    ):
        """
        Initialize distributed PPO trainer with Ray backend.
        Note that this trainer runs on the driver process on a single CPU/GPU node.

        Args:
            config: Configuration object containing training parameters.
            tokenizer: Tokenizer used for encoding and decoding text.
            role_worker_mapping (dict[Role, WorkerType]): Mapping from roles to worker classes.
            resource_pool_manager (ResourcePoolManager): Manager for Ray resource pools.
            ray_worker_group_cls (RayWorkerGroup, optional): Class for Ray worker groups. Defaults to RayWorkerGroup.
            processor: Optional data processor, used for multimodal data
            reward_fn: Function for computing rewards during training.
            val_reward_fn: Function for computing rewards during validation.
            train_dataset (Optional[Dataset], optional): Training dataset. Defaults to None.
            val_dataset (Optional[Dataset], optional): Validation dataset. Defaults to None.
            collate_fn: Function to collate data samples into batches.
            train_sampler (Optional[Sampler], optional): Sampler for the training dataset. Defaults to None.
            device_name (str, optional): Device name for training (e.g., "cuda", "cpu"). Defaults to None.
        """

        # Store the tokenizer for text processing
        self.tokenizer = tokenizer
        self.processor = processor
        self.config = config
        self.reward_fn = reward_fn
        self.val_reward_fn = val_reward_fn

        self.hybrid_engine = config.actor_rollout_ref.hybrid_engine
        assert self.hybrid_engine, "Currently, only support hybrid engine"

        if self.hybrid_engine:
            assert Role.ActorRollout in role_worker_mapping, f"{role_worker_mapping.keys()=}"

        self.role_worker_mapping = role_worker_mapping
        self.resource_pool_manager = resource_pool_manager
        self.use_reference_policy = Role.RefPolicy in role_worker_mapping
        self.use_rm = Role.RewardModel in role_worker_mapping
        self.ray_worker_group_cls = ray_worker_group_cls
        self.device_name = device_name if device_name else self.config.trainer.device
        self.validation_generations_logger = ValidationGenerationsLogger(
            project_name=self.config.trainer.project_name,
            experiment_name=self.config.trainer.experiment_name,
        )

        # if ref_in_actor is True, the reference policy will be actor without lora applied
        self.ref_in_actor = config.actor_rollout_ref.model.get("lora_rank", 0) > 0

        # define in-reward KL control
        # kl loss control currently not suppoorted
        if self.config.algorithm.use_kl_in_reward:
            self.kl_ctrl_in_reward = core_algos.get_kl_controller(self.config.algorithm.kl_ctrl)

        if self.config.algorithm.adv_estimator == AdvantageEstimator.GAE:
            self.use_critic = True
        elif self.config.algorithm.adv_estimator in [
            AdvantageEstimator.GRPO,
            AdvantageEstimator.GRPO_PASSK,
            AdvantageEstimator.REINFORCE_PLUS_PLUS,
            AdvantageEstimator.REMAX,
            AdvantageEstimator.RLOO,
            AdvantageEstimator.OPO,
            AdvantageEstimator.REINFORCE_PLUS_PLUS_BASELINE,
            AdvantageEstimator.GPG,
        ]:
            self.use_critic = False
        else:
            raise NotImplementedError

        self._validate_config()
        self._create_dataloader(train_dataset, val_dataset, collate_fn, train_sampler)

    def _validate_config(self):
        config = self.config
        # number of GPUs total
        n_gpus = config.trainer.n_gpus_per_node * config.trainer.nnodes
        if config.actor_rollout_ref.actor.strategy == "megatron":
            model_parallel_size = (
                config.actor_rollout_ref.actor.megatron.tensor_model_parallel_size
                * config.actor_rollout_ref.actor.megatron.pipeline_model_parallel_size
            )
            assert (
                n_gpus % (model_parallel_size * config.actor_rollout_ref.actor.megatron.context_parallel_size) == 0
            ), (
                f"n_gpus ({n_gpus}) must be divisible by model_parallel_size ({model_parallel_size}) times "
                f"context_parallel_size ({config.actor_rollout_ref.actor.megatron.context_parallel_size})"
            )
            megatron_dp = n_gpus // (
                model_parallel_size * config.actor_rollout_ref.actor.megatron.context_parallel_size
            )
            minimal_bsz = megatron_dp * config.actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu
        else:
            minimal_bsz = n_gpus

        # 1. Check total batch size for data correctness
        real_train_batch_size = config.data.train_batch_size * config.actor_rollout_ref.rollout.n
        assert real_train_batch_size % minimal_bsz == 0, (
            f"real_train_batch_size ({real_train_batch_size}) must be divisible by minimal possible batch size "
            f"({minimal_bsz})"
        )

        # A helper function to check "micro_batch_size" vs "micro_batch_size_per_gpu"
        # We throw an error if the user sets both. The new convention is "..._micro_batch_size_per_gpu".
        def check_mutually_exclusive(mbs, mbs_per_gpu, name: str):
            """Validate mutually exclusive micro batch size configuration options.

            Ensures that users don't set both deprecated micro_batch_size and
            the new micro_batch_size_per_gpu parameters simultaneously.

            Args:
                mbs: Deprecated micro batch size parameter value.
                mbs_per_gpu: New micro batch size per GPU parameter value.
                name (str): Configuration section name for error messages.

            Raises:
                ValueError: If both parameters are set or neither is set.
            """
            settings = {
                "actor_rollout_ref.actor": "micro_batch_size",
                "critic": "micro_batch_size",
                "reward_model": "micro_batch_size",
                "actor_rollout_ref.ref": "log_prob_micro_batch_size",
                "actor_rollout_ref.rollout": "log_prob_micro_batch_size",
            }

            if name in settings:
                param = settings[name]
                param_per_gpu = f"{param}_per_gpu"

                if mbs is None and mbs_per_gpu is None:
                    raise ValueError(
                        f"[{name}] Please set at least one of '{name}.{param}' or '{name}.{param_per_gpu}'."
                    )

                if mbs is not None and mbs_per_gpu is not None:
                    raise ValueError(
                        f"[{name}] You have set both '{name}.{param}' AND '{name}.{param_per_gpu}'. Please remove "
                        f"'{name}.{param}' because only '*_{param_per_gpu}' is supported (the former is deprecated)."
                    )

        if not config.actor_rollout_ref.actor.use_dynamic_bsz:
            # actor: ppo_micro_batch_size vs. ppo_micro_batch_size_per_gpu
            check_mutually_exclusive(
                config.actor_rollout_ref.actor.ppo_micro_batch_size,
                config.actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu,
                "actor_rollout_ref.actor",
            )

            if self.use_reference_policy:
                # reference: log_prob_micro_batch_size vs. log_prob_micro_batch_size_per_gpu
                check_mutually_exclusive(
                    config.actor_rollout_ref.ref.log_prob_micro_batch_size,
                    config.actor_rollout_ref.ref.log_prob_micro_batch_size_per_gpu,
                    "actor_rollout_ref.ref",
                )

            #  The rollout section also has log_prob_micro_batch_size vs. log_prob_micro_batch_size_per_gpu
            check_mutually_exclusive(
                config.actor_rollout_ref.rollout.log_prob_micro_batch_size,
                config.actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu,
                "actor_rollout_ref.rollout",
            )

        if self.use_critic and not config.critic.use_dynamic_bsz:
            # Check for critic micro-batch size conflicts
            check_mutually_exclusive(
                config.critic.ppo_micro_batch_size, config.critic.ppo_micro_batch_size_per_gpu, "critic"
            )

        # Check for reward model micro-batch size conflicts
        if config.reward_model.enable and not config.reward_model.use_dynamic_bsz:
            check_mutually_exclusive(
                config.reward_model.micro_batch_size, config.reward_model.micro_batch_size_per_gpu, "reward_model"
            )

        # Actor
        # check if train_batch_size is larger than ppo_mini_batch_size
        # if NOT dynamic_bsz, we must ensure:
        #    ppo_mini_batch_size is divisible by ppo_micro_batch_size
        #    ppo_micro_batch_size * sequence_parallel_size >= n_gpus
        if not config.actor_rollout_ref.actor.use_dynamic_bsz:
            assert config.data.train_batch_size >= config.actor_rollout_ref.actor.ppo_mini_batch_size
            sp_size = config.actor_rollout_ref.actor.get("ulysses_sequence_parallel_size", 1)
            if config.actor_rollout_ref.actor.ppo_micro_batch_size is not None:
                assert (
                    config.actor_rollout_ref.actor.ppo_mini_batch_size
                    % config.actor_rollout_ref.actor.ppo_micro_batch_size
                    == 0
                )
                assert config.actor_rollout_ref.actor.ppo_micro_batch_size * sp_size >= n_gpus

        assert config.actor_rollout_ref.actor.loss_agg_mode in [
            "token-mean",
            "seq-mean-token-sum",
            "seq-mean-token-mean",
            "seq-mean-token-sum-norm",
        ], f"Invalid loss_agg_mode: {config.actor_rollout_ref.actor.loss_agg_mode}"

        if self.config.algorithm.use_kl_in_reward and config.actor_rollout_ref.actor.use_kl_loss:
            print("NOTICE: You have both enabled in-reward kl and kl loss.")

        # critic
        if self.use_critic and not config.critic.use_dynamic_bsz:
            assert config.data.train_batch_size >= config.critic.ppo_mini_batch_size
            sp_size = config.critic.get("ulysses_sequence_parallel_size", 1)
            if config.critic.ppo_micro_batch_size is not None:
                assert config.critic.ppo_mini_batch_size % config.critic.ppo_micro_batch_size == 0
                assert config.critic.ppo_micro_batch_size * sp_size >= n_gpus

        # Check if use_remove_padding is enabled when using sequence parallelism for fsdp
        if config.actor_rollout_ref.actor.strategy in {"fsdp", "fsdp2"} and (
            config.actor_rollout_ref.actor.get("ulysses_sequence_parallel_size", 1) > 1
            or config.actor_rollout_ref.ref.get("ulysses_sequence_parallel_size", 1) > 1
        ):
            assert config.actor_rollout_ref.model.use_remove_padding, (
                "When using sequence parallelism for actor/ref policy, you must enable `use_remove_padding`."
            )

        if self.use_critic and config.critic.strategy in {"fsdp", "fsdp2"}:
            if config.critic.get("ulysses_sequence_parallel_size", 1) > 1:
                assert config.critic.model.use_remove_padding, (
                    "When using sequence parallelism for critic, you must enable `use_remove_padding`."
                )

        if config.data.get("val_batch_size", None) is not None:
            print(
                "WARNING: val_batch_size is deprecated."
                + " Validation datasets are sent to inference engines as a whole batch,"
                + " which will schedule the memory themselves."
            )

        # check eval config
        if config.actor_rollout_ref.rollout.val_kwargs.do_sample:
            assert config.actor_rollout_ref.rollout.temperature > 0, (
                "validation gen temperature should be greater than 0 when enabling do_sample"
            )

        print("[validate_config] All configuration checks passed successfully!")

    def _create_dataloader(self, train_dataset, val_dataset, collate_fn, train_sampler: Optional[Sampler]):
        """
        Creates the train and validation dataloaders.
        """
        # TODO: we have to make sure the batch size is divisible by the dp size
        from verl.trainer.main_ppo import create_rl_dataset, create_rl_sampler

        if train_dataset is None:
            train_dataset = create_rl_dataset(
                self.config.data.train_files, self.config.data, self.tokenizer, self.processor
            )
        if val_dataset is None:
            val_dataset = create_rl_dataset(
                self.config.data.val_files, self.config.data, self.tokenizer, self.processor
            )
        self.train_dataset, self.val_dataset = train_dataset, val_dataset

        if train_sampler is None:
            train_sampler = create_rl_sampler(self.config.data, self.train_dataset)
        if collate_fn is None:
            from verl.utils.dataset.rl_dataset import collate_fn as default_collate_fn

            collate_fn = default_collate_fn

        num_workers = self.config.data["dataloader_num_workers"]

        self.train_dataloader = StatefulDataLoader(
            dataset=self.train_dataset,
            batch_size=self.config.data.get("gen_batch_size", self.config.data.train_batch_size),
            num_workers=num_workers,
            drop_last=True,
            collate_fn=collate_fn,
            sampler=train_sampler,
        )

        val_batch_size = self.config.data.val_batch_size  # Prefer config value if set
        if val_batch_size is None:
            val_batch_size = len(self.val_dataset)

        self.val_dataloader = StatefulDataLoader(
            dataset=self.val_dataset,
            batch_size=val_batch_size,
            num_workers=num_workers,
            shuffle=self.config.data.get("validation_shuffle", True),
            drop_last=False,
            collate_fn=collate_fn,
        )

        assert len(self.train_dataloader) >= 1, "Train dataloader is empty!"
        assert len(self.val_dataloader) >= 1, "Validation dataloader is empty!"

        print(
            f"Size of train dataloader: {len(self.train_dataloader)}, Size of val dataloader: "
            f"{len(self.val_dataloader)}"
        )

        total_training_steps = len(self.train_dataloader) * self.config.trainer.total_epochs

        if self.config.trainer.total_training_steps is not None:
            total_training_steps = self.config.trainer.total_training_steps

        self.total_training_steps = total_training_steps
        print(f"Total training steps: {self.total_training_steps}")

        try:
            OmegaConf.set_struct(self.config, True)
            with open_dict(self.config):
                if OmegaConf.select(self.config, "actor_rollout_ref.actor.optim"):
                    self.config.actor_rollout_ref.actor.optim.total_training_steps = total_training_steps
                if OmegaConf.select(self.config, "critic.optim"):
                    self.config.critic.optim.total_training_steps = total_training_steps
        except Exception as e:
            print(f"Warning: Could not set total_training_steps in config. Structure missing? Error: {e}")

    def _dump_generations(self, inputs, outputs, scores, reward_extra_infos_dict, dump_path):
        """Dump rollout/validation samples as JSONL."""
        os.makedirs(dump_path, exist_ok=True)
        filename = os.path.join(dump_path, f"{self.global_steps}.jsonl")

        n = len(inputs)
        base_data = {
            "input": inputs,
            "output": outputs,
            "score": scores,
            "step": [self.global_steps] * n,
        }

        for k, v in reward_extra_infos_dict.items():
            if len(v) == n:
                base_data[k] = v

        lines = []
        for i in range(n):
            entry = {k: v[i] for k, v in base_data.items()}
            lines.append(json.dumps(entry, ensure_ascii=False))

        with open(filename, "w") as f:
            f.write("\n".join(lines) + "\n")

        print(f"Dumped generations to {filename}")

    def _maybe_log_val_generations(self, inputs, outputs, scores):
        """Log a table of validation samples to the configured logger (wandb or swanlab)"""

        generations_to_log = self.config.trainer.log_val_generations

        if generations_to_log == 0:
            return

        import numpy as np

        # Create tuples of (input, output, score) and sort by input text
        samples = list(zip(inputs, outputs, scores, strict=True))
        samples.sort(key=lambda x: x[0])  # Sort by input text

        # Use fixed random seed for deterministic shuffling
        rng = np.random.RandomState(42)
        rng.shuffle(samples)

        # Take first N samples after shuffling
        samples = samples[:generations_to_log]

        # Log to each configured logger
        self.validation_generations_logger.log(self.config.trainer.logger, samples, self.global_steps)

    def _validate(self):
        data_source_lst = []
        reward_extra_infos_dict: dict[str, list] = defaultdict(list)

        # Lists to collect samples for the table
        sample_inputs = []
        sample_outputs = []
        sample_scores = []
        sample_turns = []

        for test_data in self.val_dataloader:
            test_batch = DataProto.from_single_dict(test_data)

            # repeat test batch
            test_batch = test_batch.repeat(
                repeat_times=self.config.actor_rollout_ref.rollout.val_kwargs.n, interleave=True
            )

            # we only do validation on rule-based rm
            if self.config.reward_model.enable and test_batch[0].non_tensor_batch["reward_model"]["style"] == "model":
                return {}

            # Store original inputs
            input_ids = test_batch.batch["input_ids"]
            # TODO: Can we keep special tokens except for padding tokens?
            input_texts = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]
            sample_inputs.extend(input_texts)

            batch_keys_to_pop = ["input_ids", "attention_mask", "position_ids"]
            non_tensor_batch_keys_to_pop = ["raw_prompt_ids"]
            if "multi_modal_data" in test_batch.non_tensor_batch:
                non_tensor_batch_keys_to_pop.append("multi_modal_data")
            if "raw_prompt" in test_batch.non_tensor_batch:
                non_tensor_batch_keys_to_pop.append("raw_prompt")
            if "tools_kwargs" in test_batch.non_tensor_batch:
                non_tensor_batch_keys_to_pop.append("tools_kwargs")
            if "interaction_kwargs" in test_batch.non_tensor_batch:
                non_tensor_batch_keys_to_pop.append("interaction_kwargs")
            if "agent_name" in test_batch.non_tensor_batch:
                non_tensor_batch_keys_to_pop.append("agent_name")
            test_gen_batch = test_batch.pop(
                batch_keys=batch_keys_to_pop,
                non_tensor_batch_keys=non_tensor_batch_keys_to_pop,
            )

            test_gen_batch.meta_info = {
                "eos_token_id": self.tokenizer.eos_token_id,
                "pad_token_id": self.tokenizer.pad_token_id,
                "recompute_log_prob": False,
                "do_sample": self.config.actor_rollout_ref.rollout.val_kwargs.do_sample,
                "validate": True,
                "global_steps": self.global_steps,
            }
            print(f"test_gen_batch meta info: {test_gen_batch.meta_info}")

            # pad to be divisible by dp_size
            size_divisor = (
                self.actor_rollout_wg.world_size
                if not self.async_rollout_mode
                else self.config.actor_rollout_ref.rollout.agent.num_workers
            )
            test_gen_batch_padded, pad_size = pad_dataproto_to_divisor(test_gen_batch, size_divisor)
            if not self.async_rollout_mode:
                test_output_gen_batch_padded = self.actor_rollout_wg.generate_sequences(test_gen_batch_padded)
            else:
                test_output_gen_batch_padded = self.async_rollout_manager.generate_sequences(test_gen_batch_padded)

            # unpad
            test_output_gen_batch = unpad_dataproto(test_output_gen_batch_padded, pad_size=pad_size)

            print("validation generation end")

            # Store generated outputs
            output_ids = test_output_gen_batch.batch["responses"]
            output_texts = [self.tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]
            sample_outputs.extend(output_texts)

            test_batch = test_batch.union(test_output_gen_batch)
            test_batch.meta_info["validate"] = True

            # evaluate using reward_function
            result = self.val_reward_fn(test_batch, return_dict=True)
            reward_tensor = result["reward_tensor"]
            scores = reward_tensor.sum(-1).cpu().tolist()
            sample_scores.extend(scores)

            reward_extra_infos_dict["reward"].extend(scores)
            print(f"len reward_extra_infos_dict['reward']: {len(reward_extra_infos_dict['reward'])}")
            if "reward_extra_info" in result:
                for key, lst in result["reward_extra_info"].items():
                    reward_extra_infos_dict[key].extend(lst)
                    print(f"len reward_extra_infos_dict['{key}']: {len(reward_extra_infos_dict[key])}")

            # collect num_turns of each prompt
            if "__num_turns__" in test_batch.non_tensor_batch:
                sample_turns.append(test_batch.non_tensor_batch["__num_turns__"])

            data_source_lst.append(test_batch.non_tensor_batch.get("data_source", ["unknown"] * reward_tensor.shape[0]))

        self._maybe_log_val_generations(inputs=sample_inputs, outputs=sample_outputs, scores=sample_scores)

        # dump generations
        val_data_dir = self.config.trainer.get("validation_data_dir", None)
        if val_data_dir:
            self._dump_generations(
                inputs=sample_inputs,
                outputs=sample_outputs,
                scores=sample_scores,
                reward_extra_infos_dict=reward_extra_infos_dict,
                dump_path=val_data_dir,
            )

        for key_info, lst in reward_extra_infos_dict.items():
            assert len(lst) == 0 or len(lst) == len(sample_scores), f"{key_info}: {len(lst)=}, {len(sample_scores)=}"

        data_sources = np.concatenate(data_source_lst, axis=0)

        data_src2var2metric2val = process_validation_metrics(data_sources, sample_inputs, reward_extra_infos_dict)
        metric_dict = {}
        for data_source, var2metric2val in data_src2var2metric2val.items():
            core_var = "acc" if "acc" in var2metric2val else "reward"
            for var_name, metric2val in var2metric2val.items():
                n_max = max([int(name.split("@")[-1].split("/")[0]) for name in metric2val.keys()])
                for metric_name, metric_val in metric2val.items():
                    if (
                        (var_name == core_var)
                        and any(metric_name.startswith(pfx) for pfx in ["mean", "maj", "best"])
                        and (f"@{n_max}" in metric_name)
                    ):
                        metric_sec = "val-core"
                    else:
                        metric_sec = "val-aux"
                    pfx = f"{metric_sec}/{data_source}/{var_name}/{metric_name}"
                    metric_dict[pfx] = metric_val

        if len(sample_turns) > 0:
            sample_turns = np.concatenate(sample_turns)
            metric_dict["val-aux/num_turns/min"] = sample_turns.min()
            metric_dict["val-aux/num_turns/max"] = sample_turns.max()
            metric_dict["val-aux/num_turns/mean"] = sample_turns.mean()

        return metric_dict

    def init_workers(self):
        """Initialize distributed training workers using Ray backend.

        Creates:
        1. Ray resource pools from configuration
        2. Worker groups for each role (actor, critic, etc.)
        """
        self.resource_pool_manager.create_resource_pool()

        self.resource_pool_to_cls = {pool: {} for pool in self.resource_pool_manager.resource_pool_dict.values()}

        # create actor and rollout
        if self.hybrid_engine:
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.ActorRollout)
            actor_rollout_cls = RayClassWithInitArgs(
                cls=self.role_worker_mapping[Role.ActorRollout],
                config=self.config.actor_rollout_ref,
                role="actor_rollout",
                profile_option=self.config.trainer.npu_profile.options,
            )
            self.resource_pool_to_cls[resource_pool]["actor_rollout"] = actor_rollout_cls
        else:
            raise NotImplementedError

        # create critic
        if self.use_critic:
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.Critic)
            critic_cls = RayClassWithInitArgs(cls=self.role_worker_mapping[Role.Critic], config=self.config.critic)
            self.resource_pool_to_cls[resource_pool]["critic"] = critic_cls

        # create reference policy if needed
        if self.use_reference_policy:
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.RefPolicy)
            ref_policy_cls = RayClassWithInitArgs(
                self.role_worker_mapping[Role.RefPolicy],
                config=self.config.actor_rollout_ref,
                role="ref",
                profile_option=self.config.trainer.npu_profile.options,
            )
            self.resource_pool_to_cls[resource_pool]["ref"] = ref_policy_cls

        # create a reward model if reward_fn is None
        if self.use_rm:
            # we create a RM here
            resource_pool = self.resource_pool_manager.get_resource_pool(Role.RewardModel)
            rm_cls = RayClassWithInitArgs(self.role_worker_mapping[Role.RewardModel], config=self.config.reward_model)
            self.resource_pool_to_cls[resource_pool]["rm"] = rm_cls

        # initialize WorkerGroup
        # NOTE: if you want to use a different resource pool for each role, which can support different parallel size,
        # you should not use `create_colocated_worker_cls`.
        # Instead, directly pass different resource pool to different worker groups.
        # See https://github.com/volcengine/verl/blob/master/examples/ray/tutorial.ipynb for more information.
        all_wg = {}
        wg_kwargs = {}  # Setting up kwargs for RayWorkerGroup
        if OmegaConf.select(self.config.trainer, "ray_wait_register_center_timeout") is not None:
            wg_kwargs["ray_wait_register_center_timeout"] = self.config.trainer.ray_wait_register_center_timeout
        if OmegaConf.select(self.config.trainer, "profile_steps") is not None:
            wg_kwargs["profile_steps"] = OmegaConf.select(self.config.trainer, "profile_steps")
            assert OmegaConf.select(self.config.trainer, "worker_nsight_options") is not None, (
                "worker_nsight_options must be set when profile_steps is set"
            )
            wg_kwargs["worker_nsight_options"] = OmegaConf.to_container(
                OmegaConf.select(self.config.trainer, "worker_nsight_options")
            )
        wg_kwargs["device_name"] = self.device_name

        for resource_pool, class_dict in self.resource_pool_to_cls.items():
            worker_dict_cls = create_colocated_worker_cls(class_dict=class_dict)
            wg_dict = self.ray_worker_group_cls(
                resource_pool=resource_pool,
                ray_cls_with_init=worker_dict_cls,
                **wg_kwargs,
            )
            spawn_wg = wg_dict.spawn(prefix_set=class_dict.keys())
            all_wg.update(spawn_wg)

        if self.use_critic:
            self.critic_wg = all_wg["critic"]
            self.critic_wg.init_model()

        if self.use_reference_policy and not self.ref_in_actor:
            self.ref_policy_wg = all_wg["ref"]
            self.ref_policy_wg.init_model()

        if self.use_rm:
            self.rm_wg = all_wg["rm"]
            self.rm_wg.init_model()

        # we should create rollout at the end so that vllm can have a better estimation of kv cache memory
        self.actor_rollout_wg = all_wg["actor_rollout"]
        self.actor_rollout_wg.init_model()

        # create async rollout manager and request scheduler
        self.async_rollout_mode = False
        if self.config.actor_rollout_ref.rollout.mode == "async":
            from verl.experimental.agent_loop import AgentLoopManager

            self.async_rollout_mode = True
            self.async_rollout_manager = AgentLoopManager(
                config=self.config,
                worker_group=self.actor_rollout_wg,
            )

    def _save_checkpoint(self):
        from verl.utils.fs import local_mkdir_safe

        # path: given_path + `/global_step_{global_steps}` + `/actor`
        local_global_step_folder = os.path.join(
            self.config.trainer.default_local_dir, f"global_step_{self.global_steps}"
        )

        print(f"local_global_step_folder: {local_global_step_folder}")
        actor_local_path = os.path.join(local_global_step_folder, "actor")

        actor_remote_path = (
            None
            if self.config.trainer.default_hdfs_dir is None
            else os.path.join(self.config.trainer.default_hdfs_dir, f"global_step_{self.global_steps}", "actor")
        )

        remove_previous_ckpt_in_save = self.config.trainer.get("remove_previous_ckpt_in_save", False)
        if remove_previous_ckpt_in_save:
            print(
                "Warning: remove_previous_ckpt_in_save is deprecated,"
                + " set max_actor_ckpt_to_keep=1 and max_critic_ckpt_to_keep=1 instead"
            )
        max_actor_ckpt_to_keep = (
            self.config.trainer.get("max_actor_ckpt_to_keep", None) if not remove_previous_ckpt_in_save else 1
        )
        max_critic_ckpt_to_keep = (
            self.config.trainer.get("max_critic_ckpt_to_keep", None) if not remove_previous_ckpt_in_save else 1
        )

        self.actor_rollout_wg.save_checkpoint(
            actor_local_path, actor_remote_path, self.global_steps, max_ckpt_to_keep=max_actor_ckpt_to_keep
        )

        if self.use_critic:
            critic_local_path = os.path.join(local_global_step_folder, "critic")
            critic_remote_path = (
                None
                if self.config.trainer.default_hdfs_dir is None
                else os.path.join(self.config.trainer.default_hdfs_dir, f"global_step_{self.global_steps}", "critic")
            )
            self.critic_wg.save_checkpoint(
                critic_local_path, critic_remote_path, self.global_steps, max_ckpt_to_keep=max_critic_ckpt_to_keep
            )

        # save dataloader
        local_mkdir_safe(local_global_step_folder)
        dataloader_local_path = os.path.join(local_global_step_folder, "data.pt")
        dataloader_state_dict = self.train_dataloader.state_dict()
        torch.save(dataloader_state_dict, dataloader_local_path)

        # latest checkpointed iteration tracker (for atomic usage)
        local_latest_checkpointed_iteration = os.path.join(
            self.config.trainer.default_local_dir, "latest_checkpointed_iteration.txt"
        )
        with open(local_latest_checkpointed_iteration, "w") as f:
            f.write(str(self.global_steps))

    def _load_checkpoint(self):
        if self.config.trainer.resume_mode == "disable":
            return 0

        # load from hdfs
        if self.config.trainer.default_hdfs_dir is not None:
            raise NotImplementedError("load from hdfs is not implemented yet")
        else:
            checkpoint_folder = self.config.trainer.default_local_dir  # TODO: check path
            if not os.path.isabs(checkpoint_folder):
                working_dir = os.getcwd()
                checkpoint_folder = os.path.join(working_dir, checkpoint_folder)
            global_step_folder = find_latest_ckpt_path(checkpoint_folder)  # None if no latest

        # find global_step_folder
        if self.config.trainer.resume_mode == "auto":
            if global_step_folder is None:
                print("Training from scratch")
                return 0
        else:
            if self.config.trainer.resume_mode == "resume_path":
                assert isinstance(self.config.trainer.resume_from_path, str), "resume ckpt must be str type"
                assert "global_step_" in self.config.trainer.resume_from_path, (
                    "resume ckpt must specify the global_steps"
                )
                global_step_folder = self.config.trainer.resume_from_path
                if not os.path.isabs(global_step_folder):
                    working_dir = os.getcwd()
                    global_step_folder = os.path.join(working_dir, global_step_folder)
        print(f"Load from checkpoint folder: {global_step_folder}")
        # set global step
        self.global_steps = int(global_step_folder.split("global_step_")[-1])

        print(f"Setting global step to {self.global_steps}")
        print(f"Resuming from {global_step_folder}")

        actor_path = os.path.join(global_step_folder, "actor")
        critic_path = os.path.join(global_step_folder, "critic")
        # load actor
        self.actor_rollout_wg.load_checkpoint(
            actor_path, del_local_after_load=self.config.trainer.del_local_ckpt_after_load
        )
        # load critic
        if self.use_critic:
            self.critic_wg.load_checkpoint(
                critic_path, del_local_after_load=self.config.trainer.del_local_ckpt_after_load
            )

        # load dataloader,
        # TODO: from remote not implemented yet
        dataloader_local_path = os.path.join(global_step_folder, "data.pt")
        if os.path.exists(dataloader_local_path):
            dataloader_state_dict = torch.load(dataloader_local_path, weights_only=False)
            self.train_dataloader.load_state_dict(dataloader_state_dict)
        else:
            print(f"Warning: No dataloader state found at {dataloader_local_path}, will start from scratch")

    def _start_profiling(self, do_profile: bool) -> None:
        """Start profiling for all worker groups if profiling is enabled."""
        if do_profile:
            self.actor_rollout_wg.start_profile(role="e2e", profile_step=self.global_steps)
            if self.use_reference_policy:
                self.ref_policy_wg.start_profile()
            if self.use_critic:
                self.critic_wg.start_profile()
            if self.use_rm:
                self.rm_wg.start_profile()

    def _stop_profiling(self, do_profile: bool) -> None:
        """Stop profiling for all worker groups if profiling is enabled."""
        if do_profile:
            self.actor_rollout_wg.stop_profile()
            if self.use_reference_policy:
                self.ref_policy_wg.stop_profile()
            if self.use_critic:
                self.critic_wg.stop_profile()
            if self.use_rm:
                self.rm_wg.stop_profile()

    def _balance_batch(self, batch: DataProto, metrics, logging_prefix="global_seqlen"):
        """Reorder the data on single controller such that each dp rank gets similar total tokens"""
        attention_mask = batch.batch["attention_mask"]
        batch_size = attention_mask.shape[0]
        global_seqlen_lst = batch.batch["attention_mask"].view(batch_size, -1).sum(-1).tolist()  # (train_batch_size,)
        world_size = self.actor_rollout_wg.world_size
        global_partition_lst = get_seqlen_balanced_partitions(
            global_seqlen_lst, k_partitions=world_size, equal_size=True
        )
        # reorder based on index. The data will be automatically equally partitioned by dispatch function
        global_idx = torch.tensor([j for partition in global_partition_lst for j in partition])
        batch.reorder(global_idx)
        global_balance_stats = log_seqlen_unbalance(
            seqlen_list=global_seqlen_lst, partitions=global_partition_lst, prefix=logging_prefix
        )
        metrics.update(global_balance_stats)

    def fit(self):
        """
        The training loop of PPO.
        The driver process only need to call the compute functions of the worker group through RPC
        to construct the PPO dataflow.
        The light-weight advantage computation is done on the driver process.
        """
        from omegaconf import OmegaConf

        from verl.utils.tracking import Tracking

        logger = Tracking(
            project_name=self.config.trainer.project_name,
            experiment_name=self.config.trainer.experiment_name,
            default_backend=self.config.trainer.logger,
            config=OmegaConf.to_container(self.config, resolve=True),
        )

        self.global_steps = 0

        # load checkpoint before doing anything
        self._load_checkpoint()

        # perform validation before training
        # currently, we only support validation using the reward_function.
        if self.val_reward_fn is not None and self.config.trainer.get("val_before_train", True):
            val_metrics = self._validate()
            assert val_metrics, f"{val_metrics=}"
            pprint(f"Initial validation metrics: {val_metrics}")
            logger.log(data=val_metrics, step=self.global_steps)
            if self.config.trainer.get("val_only", False):
                return

        # add tqdm
        progress_bar = tqdm(total=self.total_training_steps, initial=self.global_steps, desc="Training Progress")

        # we start from step 1
        self.global_steps += 1
        last_val_metrics = None
        self.max_steps_duration = 0

        for epoch in range(self.config.trainer.total_epochs):
            for batch_dict in self.train_dataloader:
                metrics = {}
                timing_raw = {}

                do_profile = (
                    self.global_steps in self.config.trainer.profile_steps
                    if self.config.trainer.profile_steps is not None
                    else False
                )
                with marked_timer("start_profile", timing_raw):
                    self._start_profiling(do_profile)

                batch: DataProto = DataProto.from_single_dict(batch_dict)

                # pop those keys for generation
                batch_keys_to_pop = ["input_ids", "attention_mask", "position_ids"]
                non_tensor_batch_keys_to_pop = ["raw_prompt_ids"]
                if "multi_modal_data" in batch.non_tensor_batch:
                    non_tensor_batch_keys_to_pop.append("multi_modal_data")
                if "raw_prompt" in batch.non_tensor_batch:
                    non_tensor_batch_keys_to_pop.append("raw_prompt")
                if "tools_kwargs" in batch.non_tensor_batch:
                    non_tensor_batch_keys_to_pop.append("tools_kwargs")
                if "interaction_kwargs" in batch.non_tensor_batch:
                    non_tensor_batch_keys_to_pop.append("interaction_kwargs")
                if "index" in batch.non_tensor_batch:
                    non_tensor_batch_keys_to_pop.append("index")
                if "agent_name" in batch.non_tensor_batch:
                    non_tensor_batch_keys_to_pop.append("agent_name")

                gen_batch = batch.pop(
                    batch_keys=batch_keys_to_pop,
                    non_tensor_batch_keys=non_tensor_batch_keys_to_pop,
                )

                # pass global_steps to trace
                gen_batch.meta_info["global_steps"] = self.global_steps
                gen_batch = gen_batch.repeat(repeat_times=self.config.actor_rollout_ref.rollout.n, interleave=True)

                is_last_step = self.global_steps >= self.total_training_steps

                with marked_timer("step", timing_raw):
                    # generate a batch
                    with marked_timer("gen", timing_raw, color="red"):
                        if not self.async_rollout_mode:
                            gen_batch_output = self.actor_rollout_wg.generate_sequences(gen_batch)
                        else:
                            gen_batch_output = self.async_rollout_manager.generate_sequences(gen_batch)
                        timing_raw.update(gen_batch_output.meta_info["timing"])
                        gen_batch_output.meta_info.pop("timing", None)

                    if self.config.algorithm.adv_estimator == AdvantageEstimator.REMAX:
                        with marked_timer("gen_max", timing_raw, color="purple"):
                            gen_baseline_batch = deepcopy(gen_batch)
                            gen_baseline_batch.meta_info["do_sample"] = False
                            if not self.async_rollout_mode:
                                gen_baseline_output = self.actor_rollout_wg.generate_sequences(gen_baseline_batch)
                            else:
                                gen_baseline_output = self.async_rollout_manager.generate_sequences(gen_baseline_batch)
                            batch = batch.union(gen_baseline_output)
                            reward_baseline_tensor = self.reward_fn(batch)
                            reward_baseline_tensor = reward_baseline_tensor.sum(dim=-1)

                            batch.pop(batch_keys=list(gen_baseline_output.batch.keys()))

                            batch.batch["reward_baselines"] = reward_baseline_tensor

                            del gen_baseline_batch, gen_baseline_output

                    batch.non_tensor_batch["uid"] = np.array(
                        [str(uuid.uuid4()) for _ in range(len(batch.batch))], dtype=object
                    )
                    # repeat to align with repeated responses in rollout
                    batch = batch.repeat(repeat_times=self.config.actor_rollout_ref.rollout.n, interleave=True)
                    batch = batch.union(gen_batch_output)

                    if "response_mask" not in batch.batch.keys():
                        batch.batch["response_mask"] = compute_response_mask(batch)
                    # Balance the number of valid tokens across DP ranks.
                    # NOTE: This usually changes the order of data in the `batch`,
                    # which won't affect the advantage calculation (since it's based on uid),
                    # but might affect the loss calculation (due to the change of mini-batching).
                    # TODO: Decouple the DP balancing and mini-batching.
                    if self.config.trainer.balance_batch:
                        self._balance_batch(batch, metrics=metrics)

                    # compute global_valid tokens
                    batch.meta_info["global_token_num"] = torch.sum(batch.batch["attention_mask"], dim=-1).tolist()

                    with marked_timer("reward", timing_raw, color="yellow"):
                        # compute reward model score
                        if self.use_rm:
                            reward_tensor = self.rm_wg.compute_rm_score(batch)
                            batch = batch.union(reward_tensor)

                        if self.config.reward_model.launch_reward_fn_async:
                            future_reward = compute_reward_async.remote(data=batch, reward_fn=self.reward_fn)
                        else:
                            reward_tensor, reward_extra_infos_dict = compute_reward(batch, self.reward_fn)

                    # recompute old_log_probs
                    with marked_timer("old_log_prob", timing_raw, color="blue"):
                        old_log_prob = self.actor_rollout_wg.compute_log_prob(batch)
                        entropys = old_log_prob.batch["entropys"]
                        response_masks = batch.batch["response_mask"]
                        loss_agg_mode = self.config.actor_rollout_ref.actor.loss_agg_mode
                        entropy_agg = agg_loss(loss_mat=entropys, loss_mask=response_masks, loss_agg_mode=loss_agg_mode)
                        old_log_prob_metrics = {"actor/entropy": entropy_agg.detach().item()}
                        metrics.update(old_log_prob_metrics)
                        old_log_prob.batch.pop("entropys")
                        batch = batch.union(old_log_prob)

                        if "rollout_log_probs" in batch.batch.keys():
                            # TODO: we may want to add diff of probs too.
                            rollout_old_log_probs = batch.batch["rollout_log_probs"]
                            actor_old_log_probs = batch.batch["old_log_probs"]
                            attention_mask = batch.batch["attention_mask"]
                            responses = batch.batch["responses"]
                            response_length = responses.size(1)
                            response_mask = attention_mask[:, -response_length:]

                            rollout_probs = torch.exp(rollout_old_log_probs)
                            actor_probs = torch.exp(actor_old_log_probs)
                            rollout_probs_diff = torch.abs(rollout_probs - actor_probs)
                            rollout_probs_diff = torch.masked_select(rollout_probs_diff, response_mask.bool())
                            rollout_probs_diff_max = torch.max(rollout_probs_diff)
                            rollout_probs_diff_mean = torch.mean(rollout_probs_diff)
                            rollout_probs_diff_std = torch.std(rollout_probs_diff)
                            metrics.update(
                                {
                                    "training/rollout_probs_diff_max": rollout_probs_diff_max.detach().item(),
                                    "training/rollout_probs_diff_mean": rollout_probs_diff_mean.detach().item(),
                                    "training/rollout_probs_diff_std": rollout_probs_diff_std.detach().item(),
                                }
                            )

                    if self.use_reference_policy:
                        # compute reference log_prob
                        with marked_timer("ref", timing_raw, color="olive"):
                            if not self.ref_in_actor:
                                ref_log_prob = self.ref_policy_wg.compute_ref_log_prob(batch)
                            else:
                                ref_log_prob = self.actor_rollout_wg.compute_ref_log_prob(batch)
                            batch = batch.union(ref_log_prob)

                    # compute values
                    if self.use_critic:
                        with marked_timer("values", timing_raw, color="cyan"):
                            values = self.critic_wg.compute_values(batch)
                            batch = batch.union(values)

                    with marked_timer("adv", timing_raw, color="brown"):
                        # we combine with rule-based rm
                        reward_extra_infos_dict: dict[str, list]
                        if self.config.reward_model.launch_reward_fn_async:
                            reward_tensor, reward_extra_infos_dict = ray.get(future_reward)
                        batch.batch["token_level_scores"] = reward_tensor

                        if reward_extra_infos_dict:
                            batch.non_tensor_batch.update({k: np.array(v) for k, v in reward_extra_infos_dict.items()})

                        # compute rewards. apply_kl_penalty if available
                        if self.config.algorithm.use_kl_in_reward:
                            batch, kl_metrics = apply_kl_penalty(
                                batch, kl_ctrl=self.kl_ctrl_in_reward, kl_penalty=self.config.algorithm.kl_penalty
                            )
                            metrics.update(kl_metrics)
                        else:
                            batch.batch["token_level_rewards"] = batch.batch["token_level_scores"]

                        # compute advantages, executed on the driver process

                        norm_adv_by_std_in_grpo = self.config.algorithm.get(
                            "norm_adv_by_std_in_grpo", True
                        )  # GRPO adv normalization factor

                        batch = compute_advantage(
                            batch,
                            adv_estimator=self.config.algorithm.adv_estimator,
                            gamma=self.config.algorithm.gamma,
                            lam=self.config.algorithm.lam,
                            num_repeat=self.config.actor_rollout_ref.rollout.n,
                            norm_adv_by_std_in_grpo=norm_adv_by_std_in_grpo,
                            config=self.config.algorithm,
                        )

                    # update critic
                    if self.use_critic:
                        with marked_timer("update_critic", timing_raw, color="pink"):
                            critic_output = self.critic_wg.update_critic(batch)
                        critic_output_metrics = reduce_metrics(critic_output.meta_info["metrics"])
                        metrics.update(critic_output_metrics)

                    # implement critic warmup
                    if self.config.trainer.critic_warmup <= self.global_steps:
                        # update actor
                        with marked_timer("update_actor", timing_raw, color="red"):
                            batch.meta_info["multi_turn"] = self.config.actor_rollout_ref.rollout.multi_turn.enable
                            actor_output = self.actor_rollout_wg.update_actor(batch)
                        actor_output_metrics = reduce_metrics(actor_output.meta_info["metrics"])
                        metrics.update(actor_output_metrics)

                    # Log rollout generations if enabled
                    rollout_data_dir = self.config.trainer.get("rollout_data_dir", None)
                    if rollout_data_dir:
                        with marked_timer("dump_rollout_generations", timing_raw, color="green"):
                            inputs = self.tokenizer.batch_decode(batch.batch["prompts"], skip_special_tokens=True)
                            outputs = self.tokenizer.batch_decode(batch.batch["responses"], skip_special_tokens=True)
                            scores = batch.batch["token_level_scores"].sum(-1).cpu().tolist()
                            if "request_id" in batch.non_tensor_batch:
                                reward_extra_infos_dict.setdefault(
                                    "request_id",
                                    batch.non_tensor_batch["request_id"].tolist(),
                                )
                            self._dump_generations(
                                inputs=inputs,
                                outputs=outputs,
                                scores=scores,
                                reward_extra_infos_dict=reward_extra_infos_dict,
                                dump_path=rollout_data_dir,
                            )

                    # validate
                    if (
                        self.val_reward_fn is not None
                        and self.config.trainer.test_freq > 0
                        and (is_last_step or self.global_steps % self.config.trainer.test_freq == 0)
                    ):
                        with marked_timer("testing", timing_raw, color="green"):
                            val_metrics: dict = self._validate()
                            if is_last_step:
                                last_val_metrics = val_metrics
                        metrics.update(val_metrics)

                    # Check if the ESI (Elastic Server Instance)/training plan is close to expiration.
                    esi_close_to_expiration = should_save_ckpt_esi(
                        max_steps_duration=self.max_steps_duration,
                        redundant_time=self.config.trainer.esi_redundant_time,
                    )
                    # Check if the conditions for saving a checkpoint are met.
                    # The conditions include a mandatory condition (1) and
                    # one of the following optional conditions (2/3/4):
                    # 1. The save frequency is set to a positive value.
                    # 2. It's the last training step.
                    # 3. The current step number is a multiple of the save frequency.
                    # 4. The ESI(Elastic Server Instance)/training plan is close to expiration.
                    if self.config.trainer.save_freq > 0 and (
                        is_last_step
                        or self.global_steps % self.config.trainer.save_freq == 0
                        or esi_close_to_expiration
                    ):
                        if esi_close_to_expiration:
                            print("Force saving checkpoint: ESI instance expiration approaching.")
                        with marked_timer("save_checkpoint", timing_raw, color="green"):
                            self._save_checkpoint()

                with marked_timer("stop_profile", timing_raw):
                    self._stop_profiling(do_profile)

                steps_duration = timing_raw["step"]
                self.max_steps_duration = max(self.max_steps_duration, steps_duration)

                # training metrics
                metrics.update(
                    {
                        "training/global_step": self.global_steps,
                        "training/epoch": epoch,
                    }
                )
                # collect metrics
                metrics.update(compute_data_metrics(batch=batch, use_critic=self.use_critic))
                metrics.update(compute_timing_metrics(batch=batch, timing_raw=timing_raw))
                # TODO: implement actual tflpo and theoretical tflpo
                n_gpus = self.resource_pool_manager.get_n_gpus()
                metrics.update(compute_throughout_metrics(batch=batch, timing_raw=timing_raw, n_gpus=n_gpus))

                # this is experimental and may be changed/removed in the future in favor of a general-purpose one
                if isinstance(self.train_dataloader.sampler, AbstractCurriculumSampler):
                    self.train_dataloader.sampler.update(batch=batch)

                # TODO: make a canonical logger that supports various backend
                logger.log(data=metrics, step=self.global_steps)

                progress_bar.update(1)
                self.global_steps += 1

                if is_last_step:
                    pprint(f"Final validation metrics: {last_val_metrics}")
                    progress_bar.close()
                    return

                # this is experimental and may be changed/removed in the future
                # in favor of a general-purpose data buffer pool
                if hasattr(self.train_dataset, "on_batch_end"):
                    # The dataset may be changed after each training batch
                    self.train_dataset.on_batch_end(batch=batch)
"""
main_self_reflective.py代码如下
"""
# Copyright 2024 Bytedance Ltd. and/or its affiliates
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Note that we don't combine the main with ray_trainer as ray_trainer is used by other main.
"""

import os
import socket

import hydra
import ray
from omegaconf import OmegaConf

from verl.experimental.dataset.sampler import AbstractSampler
from verl.trainer.constants_ppo import get_ppo_ray_runtime_env
from verl.trainer.ppo.self_reflective_trainer import SelfReflectiveGRPOTrainer
from verl.trainer.ppo.teacher_student_reflective_trainer import TeacherStudentReflectiveTrainer
from verl.trainer.ppo.reward import load_reward_manager
from verl.utils.device import is_cuda_available
from verl.utils.import_utils import load_extern_type


@hydra.main(config_path="config", config_name="ppo_trainer", version_base=None)
def main(config):
    """Main entry point for PPO training with Hydra configuration management.

    Args:
        config_dict: Hydra configuration dictionary containing training parameters.
    """
    run_ppo(config)


# Define a function to run the PPO-like training process
def run_ppo(config) -> None:
    """Initialize Ray cluster and run distributed PPO training process.

    Args:
        config: Training configuration object containing all necessary parameters
                for distributed PPO training including Ray initialization settings,
                model paths, and training hyperparameters.
    """
    # Check if Ray is not initialized
    if not ray.is_initialized():
        # Initialize Ray with a local cluster configuration
        # Set environment variables in the runtime environment to control tokenizer parallelism,
        # NCCL debug level, VLLM logging level, and allow runtime LoRA updating
        # `num_cpus` specifies the number of CPU cores Ray can use, obtained from the configuration
        ray.init(
            runtime_env=get_ppo_ray_runtime_env(),
            num_cpus=config.ray_init.num_cpus,
        )

    # Create a remote instance of the TaskRunner class, and
    # Execute the `run` method of the TaskRunner instance remotely and wait for it to complete
    if (
        is_cuda_available
        and config.trainer.get("profile_steps") is not None
        and len(config.trainer.get("profile_steps", [])) > 0
    ):
        nsight_options = OmegaConf.to_container(config.trainer.controller_nsight_options)
        runner = TaskRunner.options(runtime_env={"nsight": nsight_options}).remote()
    else:
        runner = TaskRunner.remote()
    ray.get(runner.run.remote(config))

    # [Optional] get the path of the timeline trace file from the configuration, default to None
    # This file is used for performance analysis
    timeline_json_file = config.ray_init.get("timeline_json_file", None)
    if timeline_json_file:
        ray.timeline(filename=timeline_json_file)


@ray.remote(num_cpus=1)  # please make sure main_task is not scheduled on head
class TaskRunner:
    """Ray remote class for executing distributed PPO training tasks.

    This class encapsulates the main training logic and runs as a Ray remote actor
    to enable distributed execution across multiple nodes and GPUs.
    """

    def run(self, config):
        """Execute the main PPO training workflow.

        This method sets up the distributed training environment, initializes
        workers, datasets, and reward functions, then starts the training process.

        Args:
            config: Training configuration object containing all parameters needed
                   for setting up and running the PPO training process.
        """
        # Print the initial configuration. `resolve=True` will evaluate symbolic values.
        from pprint import pprint

        from omegaconf import OmegaConf

        from verl.utils.fs import copy_to_local

        print(f"TaskRunner hostname: {socket.gethostname()}, PID: {os.getpid()}")
        pprint(OmegaConf.to_container(config, resolve=True))
        OmegaConf.resolve(config)

        # Download the checkpoint from HDFS to the local machine.
        # `use_shm` determines whether to use shared memory, which could lead to faster model loading if turned on
        local_path = copy_to_local(
            config.actor_rollout_ref.model.path, use_shm=config.actor_rollout_ref.model.get("use_shm", False)
        )

        # Instantiate the tokenizer and processor.
        from verl.utils import hf_processor, hf_tokenizer

        trust_remote_code = config.data.get("trust_remote_code", False)
        tokenizer = hf_tokenizer(local_path, trust_remote_code=trust_remote_code)
        # Used for multimodal LLM, could be None
        processor = hf_processor(local_path, trust_remote_code=trust_remote_code, use_fast=True)

        # Define worker classes based on the actor strategy.
        if config.actor_rollout_ref.actor.strategy in {"fsdp", "fsdp2"}:
            assert config.critic.strategy in {"fsdp", "fsdp2"}
            from verl.single_controller.ray import RayWorkerGroup
            from verl.workers.fsdp_workers import ActorRolloutRefWorker, AsyncActorRolloutRefWorker

            use_legacy_worker_impl = config.trainer.get("use_legacy_worker_impl", "auto")
            if use_legacy_worker_impl in ["auto", "enable"]:
                # import warnings
                # warnings.warn(f"Legacy worker impl is going to be deprecated, will be removed in the future. \
                #   Please set trainer.use_legacy_worker_impl = false to switch to the new worker implementation.")
                from verl.workers.fsdp_workers import CriticWorker
            elif use_legacy_worker_impl == "disable":
                from verl.workers.roles import CriticWorker

                print("Using new worker implementation")
            else:
                raise ValueError(f"Invalid use_legacy_worker_impl: {use_legacy_worker_impl}")

            actor_rollout_cls = (
                AsyncActorRolloutRefWorker
                if config.actor_rollout_ref.rollout.mode == "async"
                else ActorRolloutRefWorker
            )
            ray_worker_group_cls = RayWorkerGroup

        elif config.actor_rollout_ref.actor.strategy == "megatron":
            assert config.actor_rollout_ref.actor.strategy == config.critic.strategy
            from verl.single_controller.ray.megatron import NVMegatronRayWorkerGroup
            from verl.workers.megatron_workers import ActorRolloutRefWorker, AsyncActorRolloutRefWorker, CriticWorker

            actor_rollout_cls = (
                AsyncActorRolloutRefWorker
                if config.actor_rollout_ref.rollout.mode == "async"
                else ActorRolloutRefWorker
            )
            ray_worker_group_cls = NVMegatronRayWorkerGroup

        else:
            raise NotImplementedError

        from verl.trainer.ppo.ray_trainer import ResourcePoolManager, Role

        # Map roles to their corresponding remote worker classes.
        role_worker_mapping = {
            Role.ActorRollout: ray.remote(actor_rollout_cls),
            Role.Critic: ray.remote(CriticWorker),
        }

        # Define the resource pool specification.
        # Map roles to the resource pool.
        global_pool_id = "global_pool"
        resource_pool_spec = {
            global_pool_id: [config.trainer.n_gpus_per_node] * config.trainer.nnodes,
        }
        mapping = {
            Role.ActorRollout: global_pool_id,
            Role.Critic: global_pool_id,
        }

        # We should adopt a multi-source reward function here:
        # - for rule-based rm, we directly call a reward score
        # - for model-based rm, we call a model
        # - for code related prompt, we send to a sandbox if there are test cases
        # finally, we combine all the rewards together
        # The reward type depends on the tag of the data
        if config.reward_model.enable:
            if config.reward_model.strategy in {"fsdp", "fsdp2"}:
                from verl.workers.fsdp_workers import RewardModelWorker
            elif config.reward_model.strategy == "megatron":
                from verl.workers.megatron_workers import RewardModelWorker
            else:
                raise NotImplementedError
            role_worker_mapping[Role.RewardModel] = ray.remote(RewardModelWorker)
            mapping[Role.RewardModel] = global_pool_id

        # Add a reference policy worker if KL loss or KL reward is used.
        # For TeacherStudentReflectiveTrainer, we ALWAYS need a RefPolicy to act as the Teacher.
        # So we remove the conditional check or ensure it's always added.
        
        # 强制添加 RefPolicy
        role_worker_mapping[Role.RefPolicy] = ray.remote(ActorRolloutRefWorker)
        mapping[Role.RefPolicy] = global_pool_id

        # Load the reward manager for training and validation.
        reward_fn = load_reward_manager(
            config, tokenizer, num_examine=0, **config.reward_model.get("reward_kwargs", {})
        )
        val_reward_fn = load_reward_manager(
            config, tokenizer, num_examine=1, **config.reward_model.get("reward_kwargs", {})
        )
        resource_pool_manager = ResourcePoolManager(resource_pool_spec=resource_pool_spec, mapping=mapping)

        from verl.utils.dataset.rl_dataset import collate_fn

        # Create training and validation datasets.
        train_dataset = create_rl_dataset(config.data.train_files, config.data, tokenizer, processor, is_train=True)
        val_dataset = create_rl_dataset(config.data.val_files, config.data, tokenizer, processor, is_train=False)
        train_sampler = create_rl_sampler(config.data, train_dataset)

        # Initialize the PPO trainer.
        trainer = TeacherStudentReflectiveTrainer(
            config=config,
            tokenizer=tokenizer,
            processor=processor,
            role_worker_mapping=role_worker_mapping,
            resource_pool_manager=resource_pool_manager,
            ray_worker_group_cls=ray_worker_group_cls,
            reward_fn=reward_fn,
            val_reward_fn=val_reward_fn,
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            collate_fn=collate_fn,
            train_sampler=train_sampler,
        )
        # Initialize the workers of the trainer.
        trainer.init_workers()
        # Start the training process.
        trainer.fit()


def create_rl_dataset(data_paths, data_config, tokenizer, processor, is_train=True):
    """Create a dataset.

    Arguments:
        data_paths: List of paths to data files.
        data_config: The data config.
        tokenizer (Tokenizer): The tokenizer.
        processor (Processor): The processor.

    Returns:
        dataset (Dataset): The dataset.
    """
    from torch.utils.data import Dataset

    from verl.utils.dataset.rl_dataset import RLHFDataset

    # Check if a custom dataset class is specified in the data configuration
    # and if the path to the custom class is provided
    if "custom_cls" in data_config and data_config.custom_cls.get("path", None) is not None:
        # Dynamically load the custom dataset class
        dataset_cls = load_extern_type(data_config.custom_cls.path, data_config.custom_cls.name)
        # Verify that the custom dataset class inherits from torch.utils.data.Dataset
        if not issubclass(dataset_cls, Dataset):
            raise TypeError(
                f"The custom dataset class '{data_config.custom_cls.name}' from "
                f"'{data_config.custom_cls.path}' must inherit from torch.utils.data.Dataset"
            )
    elif "datagen" in data_config and data_config.datagen.get("path", None) is not None and is_train:
        # If a data generation strategy is specified, use the DynamicGenDataset class
        from verl.utils.dataset.dynamicgen_dataset import DynamicGenDataset

        dataset_cls = DynamicGenDataset
        print("Using DynamicGenDataset for data generation.")

    else:
        # Use the default RLHFDataset class if no custom class is specified
        dataset_cls = RLHFDataset
    print(f"Using dataset class: {dataset_cls.__name__}")

    # Instantiate the dataset using the determined dataset class
    dataset = dataset_cls(
        data_files=data_paths,
        tokenizer=tokenizer,
        processor=processor,
        config=data_config,
    )

    return dataset


def create_rl_sampler(data_config, dataset):
    """Create a sampler for the dataset.

    Arguments:
        data_config: The data config.
        dataset (Dataset): The dataset.

    Returns:
        sampler (Sampler): The sampler.
    """
    import torch
    from torch.utils.data import RandomSampler, SequentialSampler

    if data_config.sampler is not None and data_config.sampler.get("class_path", None) is not None:
        curriculum_class = load_extern_type(
            data_config.sampler.class_path,
            data_config.sampler.class_name,
        )
        sampler = curriculum_class(
            data_source=dataset,
            data_config=data_config,
        )
        assert isinstance(sampler, AbstractSampler)
        assert data_config.get("dataloader_num_workers", 8) == 0, (
            "If using curriculum, num_workers must be 0 to prevent data caching. "
            "If the dataloader caches data before the batch is done the "
            "curriculum sampler won't have the opportunity to reorder it. "
        )

    # Use a sampler to facilitate checkpoint resumption.
    # If shuffling is enabled in the data configuration, create a random sampler.
    elif data_config.shuffle:
        train_dataloader_generator = torch.Generator()
        train_dataloader_generator.manual_seed(data_config.get("seed", 1))
        sampler = RandomSampler(data_source=dataset, generator=train_dataloader_generator)
    else:
        # If shuffling is disabled, use a sequential sampler to iterate through the dataset in order.
        sampler = SequentialSampler(data_source=dataset)

    return sampler


if __name__ == "__main__":
    main()
"""
dp_actor.py修改代码如下
"""
# Copyright 2024 Bytedance Ltd. and/or its affiliates
# Copyright 2023-2024 SGLang Team
# Copyright 2025 ModelBest Inc. and/or its affiliates
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
Single Process Actor
"""

import logging
import os

import torch
from torch import nn
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP

import verl.utils.torch_functional as verl_F
from verl import DataProto
from verl.trainer.ppo.core_algos import agg_loss, compute_policy_loss, get_policy_loss_fn, kl_penalty
from verl.utils.device import get_device_name, is_cuda_available, is_npu_available
from verl.utils.fsdp_utils import FSDPModule, fsdp2_clip_grad_norm_
from verl.utils.profiler import GPUMemoryLogger
from verl.utils.py_functional import append_to_dict
from verl.utils.seqlen_balancing import prepare_dynamic_batch, restore_dynamic_batch
from verl.utils.torch_functional import logprobs_from_logits
from verl.utils.ulysses import gather_outputs_and_unpad, ulysses_pad, ulysses_pad_and_slice_inputs
from verl.workers.actor import BasePPOActor

if is_cuda_available:
    from flash_attn.bert_padding import index_first_axis, pad_input, rearrange, unpad_input
elif is_npu_available:
    from transformers.integrations.npu_flash_attention import index_first_axis, pad_input, rearrange, unpad_input


__all__ = ["DataParallelPPOActor"]

logger = logging.getLogger(__file__)
logger.setLevel(os.getenv("VERL_LOGGING_LEVEL", "WARN"))


class DataParallelPPOActor(BasePPOActor):
    def __init__(self, config, actor_module: nn.Module, actor_optimizer: torch.optim.Optimizer = None):
        """When optimizer is None, it is Reference Policy"""
        super().__init__(config)
        self.actor_module = actor_module
        self.actor_optimizer = actor_optimizer

        self.use_remove_padding = self.config.get("use_remove_padding", False)
        if torch.distributed.get_rank() == 0:
            print(f"Actor use_remove_padding={self.use_remove_padding}")
        self.use_fused_kernels = self.config.get("use_fused_kernels", False)
        if torch.distributed.get_rank() == 0:
            print(f"Actor use_fused_kernels={self.use_fused_kernels}")

        self.ulysses_sequence_parallel_size = self.config.ulysses_sequence_parallel_size
        self.use_ulysses_sp = self.ulysses_sequence_parallel_size > 1

        if self.config.entropy_from_logits_with_chunking:
            entropy_from_logits = verl_F.entropy_from_logits_with_chunking
        else:
            entropy_from_logits = verl_F.entropy_from_logits

        self.compute_entropy_from_logits = (
            torch.compile(entropy_from_logits, dynamic=True)
            if self.config.get("use_torch_compile", True)  #  use torch compile by default
            else entropy_from_logits
        )
        self.device_name = get_device_name()

    def _forward_micro_batch(
        self, micro_batch, temperature, calculate_entropy=False
    ) -> tuple[torch.Tensor, torch.Tensor]:
        """
        Returns:
            entropy: # (bs, response_len)
            log_probs: # (bs, response_len)
        """
        response_length = micro_batch["responses"].size(-1)
        multi_modal_inputs = {}
        if "multi_modal_inputs" in micro_batch.keys():
            if "image_bound" in micro_batch["multi_modal_inputs"][0]:  # minicpm-o logic
                for key in micro_batch["multi_modal_inputs"][0].keys():
                    multi_modal_inputs[key] = [inputs[key] for inputs in micro_batch["multi_modal_inputs"]]
            else:
                for key in micro_batch["multi_modal_inputs"][0].keys():
                    multi_modal_inputs[key] = torch.cat(
                        [inputs[key] for inputs in micro_batch["multi_modal_inputs"]], dim=0
                    )

        with torch.autocast(device_type=self.device_name, dtype=torch.bfloat16):
            input_ids = micro_batch["input_ids"]
            batch_size, seqlen = input_ids.shape
            attention_mask = micro_batch["attention_mask"]
            position_ids = micro_batch["position_ids"]
            entropy = None
            if position_ids.dim() == 3:  # qwen2vl mrope
                position_ids = position_ids.transpose(0, 1)  # (bsz, 3, seqlen) -> (3, bsz, seqlen)

            if self.use_remove_padding:
                input_ids_rmpad, indices, cu_seqlens, *_ = unpad_input(
                    input_ids.unsqueeze(-1), attention_mask
                )  # input_ids_rmpad (total_nnz, ...)
                input_ids_rmpad = input_ids_rmpad.transpose(0, 1)  # (1, total_nnz)

                # unpad the position_ids to align the rotary
                if position_ids.dim() == 3:
                    position_ids_rmpad = (
                        index_first_axis(rearrange(position_ids, "c b s ... -> (b s) c ..."), indices)
                        .transpose(0, 1)
                        .unsqueeze(1)
                    )  # (3, bsz, seqlen) -> (3, 1, bsz * seqlen)
                else:
                    position_ids_rmpad = index_first_axis(
                        rearrange(position_ids.unsqueeze(-1), "b s ... -> (b s) ..."), indices
                    ).transpose(0, 1)

                if "image_bound" in multi_modal_inputs:
                    from verl.utils.dataset.vision_utils import process_multi_modal_inputs_for_minicpmo

                    multi_modal_inputs = process_multi_modal_inputs_for_minicpmo(
                        input_ids, attention_mask, position_ids, cu_seqlens, multi_modal_inputs
                    )

                # for compute the log_prob
                input_ids_rmpad_rolled = torch.roll(input_ids_rmpad, shifts=-1, dims=1)  # (1, total_nnz)

                # pad and slice the inputs if sp > 1
                if self.use_ulysses_sp:
                    is_vlm_model = "multi_modal_inputs" in micro_batch.keys()
                    if is_vlm_model:
                        # vlm model's inputs will be sliced after embedding
                        input_ids_rmpad, position_ids_rmpad, pad_size = ulysses_pad(
                            input_ids_rmpad,
                            position_ids_rmpad=position_ids_rmpad,
                            sp_size=self.ulysses_sequence_parallel_size,
                        )
                    else:
                        input_ids_rmpad, position_ids_rmpad, pad_size = ulysses_pad_and_slice_inputs(
                            input_ids_rmpad,
                            position_ids_rmpad=position_ids_rmpad,
                            sp_size=self.ulysses_sequence_parallel_size,
                        )
                    input_ids_rmpad_rolled, _, _ = ulysses_pad_and_slice_inputs(
                        input_ids_rmpad_rolled,
                        position_ids_rmpad=None,
                        sp_size=self.ulysses_sequence_parallel_size,
                    )

                input_ids_rmpad_rolled = input_ids_rmpad_rolled.squeeze(0)  # ((total_nnz / sp) + pad)

                # only pass input_ids and position_ids to enable flash_attn_varlen
                extra_args = {}
                if self.use_fused_kernels:
                    extra_args["temperature"] = temperature
                    extra_args["return_dict"] = True

                output = self.actor_module(
                    input_ids=input_ids_rmpad,
                    attention_mask=None,
                    position_ids=position_ids_rmpad,
                    **multi_modal_inputs,
                    use_cache=False,
                    **extra_args,
                )  # prevent model thinks we are generating

                if self.use_fused_kernels:
                    log_probs = output.log_probs.squeeze(0)  # (total_nnz,)
                    entropy_rmpad = output.entropy.squeeze(0)  # (total_nnz,)

                else:
                    logits_rmpad = output.logits.squeeze(0)  # (total_nnz, vocab_size)
                    logits_rmpad.div_(temperature)

                    # if use_sp: ((total_nnz / sp) + pad) ; if not use_sp: (batch, seqlen)
                    inplace_backward = True
                    if calculate_entropy:
                        inplace_backward = False
                    log_probs = logprobs_from_logits(
                        logits=logits_rmpad,
                        labels=input_ids_rmpad_rolled,
                        inplace_backward=inplace_backward,
                    )

                    # compute entropy
                    if calculate_entropy:
                        if not self.config.entropy_checkpointing:
                            entropy_rmpad = self.compute_entropy_from_logits(logits_rmpad)  # ((total_nnz / sp) + pad)
                        else:
                            entropy_rmpad = torch.utils.checkpoint.checkpoint(
                                self.compute_entropy_from_logits, logits_rmpad
                            )

                # gather log_prob if sp > 1
                if self.use_ulysses_sp:
                    # gather and unpad for the ulysses sp
                    log_probs = gather_outputs_and_unpad(
                        log_probs,
                        gather_dim=0,
                        unpad_dim=0,
                        padding_size=pad_size,
                    )
                    if calculate_entropy:
                        entropy_rmpad = gather_outputs_and_unpad(
                            entropy_rmpad,
                            gather_dim=0,
                            unpad_dim=0,
                            padding_size=pad_size,
                        )
                # pad back to (bsz, seqlen)
                if calculate_entropy:
                    full_entropy = pad_input(
                        hidden_states=entropy_rmpad.unsqueeze(-1),
                        indices=indices,
                        batch=batch_size,
                        seqlen=seqlen,
                    )
                full_log_probs = pad_input(
                    hidden_states=log_probs.unsqueeze(-1),
                    indices=indices,
                    batch=batch_size,
                    seqlen=seqlen,
                )

                # only return response part:
                if calculate_entropy:
                    entropy = full_entropy.squeeze(-1)[:, -response_length - 1 : -1]  # (bsz, response_length)
                log_probs = full_log_probs.squeeze(-1)[:, -response_length - 1 : -1]  # (bsz, response_length)

            else:  # not using rmpad and no ulysses sp
                extra_args = {}
                if self.use_fused_kernels:
                    extra_args["temperature"] = temperature
                    extra_args["return_dict"] = True

                output = self.actor_module(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    position_ids=position_ids,
                    **multi_modal_inputs,
                    use_cache=False,
                    **extra_args,
                )  # prevent model thinks we are generating

                if self.use_fused_kernels:
                    log_probs = output.log_probs[:, -response_length - 1 : -1]
                    entropy = output.entropy[:, -response_length - 1 : -1]  # (bsz, response_length)

                else:
                    logits = output.logits

                    logits.div_(temperature)
                    logits = logits[:, -response_length - 1 : -1, :]  # (bsz, response_length, vocab_size)
                    log_probs = logprobs_from_logits(logits, micro_batch["responses"])
                    if calculate_entropy:
                        if not self.config.entropy_checkpointing:
                            entropy = verl_F.entropy_from_logits(logits)  # (bsz, response_length)
                        else:
                            entropy = torch.utils.checkpoint.checkpoint(verl_F.entropy_from_logits, logits)

            return entropy, log_probs

    def _optimizer_step(self):
        assert self.config.grad_clip is not None

        if isinstance(self.actor_module, FSDP):
            grad_norm = self.actor_module.clip_grad_norm_(max_norm=self.config.grad_clip)
        elif isinstance(self.actor_module, FSDPModule):
            grad_norm = fsdp2_clip_grad_norm_(self.actor_module.parameters(), max_norm=self.config.grad_clip)
        else:
            grad_norm = torch.nn.utils.clip_grad_norm_(self.actor_module.parameters(), max_norm=self.config.grad_clip)

        # if grad_norm is not finite, skip the update
        if not torch.isfinite(grad_norm):
            print(f"WARN: rank {torch.distributed.get_rank()} grad_norm is not finite: {grad_norm}")
            self.actor_optimizer.zero_grad()
        else:
            self.actor_optimizer.step()
        return grad_norm

    @GPUMemoryLogger(role="dp actor", logger=logger)
    def compute_log_prob(self, data: DataProto, calculate_entropy=False) -> torch.Tensor:
        """Compute the log probability of the responses given input_ids, attention_mask and position_ids"""
        # set to eval
        self.actor_module.eval()

        micro_batch_size = data.meta_info["micro_batch_size"]
        temperature = data.meta_info["temperature"]
        use_dynamic_bsz = data.meta_info["use_dynamic_bsz"]
        has_multi_modal_inputs = "multi_modal_inputs" in data.non_tensor_batch.keys()
        select_keys = ["responses", "input_ids", "attention_mask", "position_ids"]
        non_tensor_select_keys = ["multi_modal_inputs"] if has_multi_modal_inputs else []

        data = data.select(batch_keys=select_keys, non_tensor_batch_keys=non_tensor_select_keys)

        if use_dynamic_bsz:
            max_token_len = data.meta_info["max_token_len"] * self.ulysses_sequence_parallel_size
            micro_batches, batch_idx_list = prepare_dynamic_batch(data, max_token_len=max_token_len)
        else:
            micro_batches = data.split(micro_batch_size)

        log_probs_lst = []
        entropy_lst = []
        for micro_batch in micro_batches:
            model_inputs = {**micro_batch.batch, **micro_batch.non_tensor_batch}
            with torch.no_grad():
                entropy, log_probs = self._forward_micro_batch(
                    model_inputs, temperature=temperature, calculate_entropy=calculate_entropy
                )
            log_probs_lst.append(log_probs)
            if calculate_entropy:
                entropy_lst.append(entropy)

        log_probs = torch.concat(log_probs_lst, dim=0)
        entropys = None
        if calculate_entropy:
            entropys = torch.concat(entropy_lst, dim=0)

        if use_dynamic_bsz:
            log_probs = restore_dynamic_batch(log_probs, batch_idx_list)
            if calculate_entropy:
                entropys = restore_dynamic_batch(entropys, batch_idx_list)

        return log_probs, entropys

    @GPUMemoryLogger(role="dp actor", logger=logger)
    def update_policy(self, data: DataProto):
        # make sure we are in training mode
        self.actor_module.train()

        temperature = data.meta_info["temperature"]

        select_keys = [
            "responses",
            "response_mask",
            "input_ids",
            "attention_mask",
            "position_ids",
            "old_log_probs",
            "advantages",
        ]
        if self.config.use_kl_loss:
            select_keys.append("ref_log_prob")

        has_multi_modal_inputs = "multi_modal_inputs" in data.non_tensor_batch.keys()
        non_tensor_select_keys = ["multi_modal_inputs"] if has_multi_modal_inputs else []

        data = data.select(batch_keys=select_keys, non_tensor_batch_keys=non_tensor_select_keys)

        # Split to make minibatch iterator
        mini_batches = data.split(self.config.ppo_mini_batch_size)

        metrics = {}
        for _ in range(self.config.ppo_epochs):
            for batch_idx, mini_batch in enumerate(mini_batches):
                if self.config.use_dynamic_bsz:
                    max_token_len = self.config.ppo_max_token_len_per_gpu * self.ulysses_sequence_parallel_size
                    micro_batches, _ = prepare_dynamic_batch(mini_batch, max_token_len=max_token_len)
                else:
                    self.gradient_accumulation = (
                        self.config.ppo_mini_batch_size // self.config.ppo_micro_batch_size_per_gpu
                    )
                    micro_batches = mini_batch.split(self.config.ppo_micro_batch_size_per_gpu)

                self.actor_optimizer.zero_grad()

                for micro_batch in micro_batches:
                    micro_batch_metrics = {}
                    model_inputs = {**micro_batch.batch, **micro_batch.non_tensor_batch}
                    response_mask = model_inputs["response_mask"]
                    old_log_prob = model_inputs["old_log_probs"]
                    advantages = model_inputs["advantages"]

                    clip_ratio = self.config.clip_ratio
                    clip_ratio_low = (
                        self.config.clip_ratio_low if self.config.clip_ratio_low is not None else clip_ratio
                    )
                    clip_ratio_high = (
                        self.config.clip_ratio_high if self.config.clip_ratio_high is not None else clip_ratio
                    )
                    clip_ratio_c = self.config.get("clip_ratio_c", 3.0)
                    entropy_coeff = self.config.entropy_coeff
                    loss_agg_mode = self.config.loss_agg_mode

                    calculate_entropy = False
                    if entropy_coeff != 0:
                        calculate_entropy = True
                    entropy, log_prob = self._forward_micro_batch(
                        model_inputs, temperature=temperature, calculate_entropy=calculate_entropy
                    )

                    # =========================================================================
                    # 修复: 健壮地读取 loss_mode
                    # =========================================================================
                    if isinstance(self.config.policy_loss, str):
                        loss_mode = self.config.policy_loss
                    else:
                        loss_mode = self.config.policy_loss.get("loss_mode", "vanilla")

                    # =========================================================================
                    # 新增: distillation 模式 (Pure Policy Gradient / No Clipping)
                    # =========================================================================
                    if loss_mode == "distillation":
                        # 直接最大化 (advantages * log_prob)
                        # 我们假设 advantages 已经包含了 (log P_T - log P_S) 的信号
                        # 这是一个无偏的 KL 散度梯度估计
                        pg_loss = -verl_F.masked_mean(advantages * log_prob, response_mask)
                        
                        # 占位符，避免日志报错
                        pg_clipfrac = torch.tensor(0.0, device=pg_loss.device)
                        ppo_kl = torch.tensor(0.0, device=pg_loss.device)
                        pg_clipfrac_lower = torch.tensor(0.0, device=pg_loss.device)

                    elif loss_mode == "vanilla":
                        pg_loss, pg_clipfrac, ppo_kl, pg_clipfrac_lower = compute_policy_loss(
                            old_log_prob=old_log_prob,
                            log_prob=log_prob,
                            advantages=advantages,
                            response_mask=response_mask,
                            cliprange=clip_ratio,
                            cliprange_low=clip_ratio_low,
                            cliprange_high=clip_ratio_high,
                            clip_ratio_c=clip_ratio_c,
                            loss_agg_mode=loss_agg_mode,
                        )

                    else:
                        policy_loss_fn = get_policy_loss_fn(loss_mode)
                        pg_loss, pg_clipfrac, ppo_kl, pg_clipfrac_lower = policy_loss_fn(
                            old_log_prob=old_log_prob,
                            log_prob=log_prob,
                            advantages=advantages,
                            response_mask=response_mask,
                            loss_agg_mode=loss_agg_mode,
                            config=self.config,
                        )

                    if entropy_coeff != 0:
                        entropy_loss = agg_loss(loss_mat=entropy, loss_mask=response_mask, loss_agg_mode=loss_agg_mode)
                        policy_loss = pg_loss - entropy_loss * entropy_coeff
                    else:
                        policy_loss = pg_loss

                    if self.config.use_kl_loss:
                        ref_log_prob = model_inputs["ref_log_prob"]
                        kld = kl_penalty(
                            logprob=log_prob, ref_logprob=ref_log_prob, kl_penalty=self.config.kl_loss_type
                        )
                        kl_loss = agg_loss(loss_mat=kld, loss_mask=response_mask, loss_agg_mode=loss_agg_mode)

                        policy_loss = policy_loss + kl_loss * self.config.kl_loss_coef
                        micro_batch_metrics["actor/kl_loss"] = kl_loss.detach().item()
                        micro_batch_metrics["actor/kl_coef"] = self.config.kl_loss_coef

                    if self.config.use_dynamic_bsz:
                        loss = policy_loss * (response_mask.shape[0] / self.config.ppo_mini_batch_size)
                    else:
                        loss = policy_loss / self.gradient_accumulation
                    loss.backward()

                    micro_batch_metrics.update(
                        {
                            "actor/pg_loss": pg_loss.detach().item(),
                            "actor/pg_clipfrac": pg_clipfrac.detach().item(),
                            "actor/ppo_kl": ppo_kl.detach().item(),
                            "actor/pg_clipfrac_lower": pg_clipfrac_lower.detach().item(),
                        }
                    )
                    append_to_dict(metrics, micro_batch_metrics)

                grad_norm = self._optimizer_step()
                mini_batch_metrics = {"actor/grad_norm": grad_norm.detach().item()}
                append_to_dict(metrics, mini_batch_metrics)
        self.actor_optimizer.zero_grad()
        return metrics
"""
启动脚本代码如下
"""
export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7
export DATA_DIR='/data/kcl/lpy/data'
WAND_PROJECT="self_reflect"
export SWANLAB_API_KEY="DDMUFxYQU6Om1ZuHWYoxm"           # 设置在线跟踪模式API
export BASE_MODEL='/data/kcl/lpy/models/qwen3_4b_instruct'
export EXPERIMENT_NAME=sr_dl_teacher_student_gsm8k_qwen3_4b_instruct
N_NODES=1
n_gpus_per_node=8
export BASE_DIR=$(pwd)
export HF_ENDPOINT=https://hf-mirror.com
export VLLM_USE_V1=1
PYTHONUNBUFFERED=1 python3 -m verl.trainer.main_self_reflective \
 data.train_files=$DATA_DIR/gsm8k/train.parquet \
 data.val_files=$DATA_DIR/gsm8k/test.parquet \
 data.train_batch_size=256 \
 data.max_prompt_length=512 \
 data.max_response_length=512 \
 actor_rollout_ref.model.path=$BASE_MODEL \
 actor_rollout_ref.actor.optim.lr=5e-7 \
 actor_rollout_ref.actor.ppo_mini_batch_size=64 \
 actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu=4 \
 actor_rollout_ref.actor.policy_loss=distillation \
 actor_rollout_ref.rollout.name=vllm \
 actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu=8 \
 actor_rollout_ref.rollout.tensor_model_parallel_size=1 \
 actor_rollout_ref.rollout.gpu_memory_utilization=0.4 \
 actor_rollout_ref.ref.log_prob_micro_batch_size_per_gpu=4 \
 critic.optim.lr=1e-5 \
 critic.model.path=$BASE_MODEL \
 critic.ppo_micro_batch_size_per_gpu=4 \
 algorithm.kl_ctrl.kl_coef=0.001 \
 trainer.logger=console \
 trainer.val_before_train=False \
 trainer.n_gpus_per_node=$n_gpus_per_node \
 trainer.nnodes=$N_NODES \
 trainer.save_freq=60 \
 trainer.test_freq=60 \
 trainer.logger="['swanlab', 'console']" \
 trainer.project_name=$WAND_PROJECT \
 trainer.experiment_name=$EXPERIMENT_NAME \
 trainer.default_local_dir=${BASE_DIR}/verl_checkpoints/$EXPERIMENT_NAME \
 trainer.total_epochs=15 2>&1 | tee verl_log/$EXPERIMENT_NAME.log
"""
现在运行我发现问题，日志输出如下
"""
[36m(TaskRunner pid=1613436)[0m ==================== Teacher-Student Reflection Debug (Step 1) ====================
[36m(TaskRunner pid=1613436)[0m --- [0] Summary Generation Input ---
[36m(TaskRunner pid=1613436)[0m <|im_start|>system
[36m(TaskRunner pid=1613436)[0m You are a helpful math assistant.<|im_end|>
[36m(TaskRunner pid=1613436)[0m <|im_start|>user
[36m(TaskRunner pid=1613436)[0m Question: Tom receives a $12 allowance per month. In the first week, he spends a third of it; in the second week, he spends a quarter of what he has left. How much money does he have left to finish the month? Let's think step by step and output the final answer after "####".
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Standard Answer: 6
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Student Answer: Let's solve this step by step.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Tom receives a $12 allowance per month.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m **First week:**
[36m(TaskRunner pid=1613436)[0m He spends a third of it.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m A third of $12 is:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m \frac{1}{3} \times 12 = 4
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m So, after the first week, he has:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 12 - 4 = 8 \text{ dollars left}
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m **Second week:**
[36m(TaskRunner pid=1613436)[0m He spends a quarter of what he has left.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m A quarter of $8 is:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m \frac{1}{4} \times 8 = 2
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m So, after spending in the second week, he has:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 8 - 2 = 6 \text{ dollars left}
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m #### 6
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Task: Verify the Student Answer step-by-step. Is it correct?
[36m(TaskRunner pid=1613436)[0m Answer concisely.<|im_end|>
[36m(TaskRunner pid=1613436)[0m <|im_start|>assistant
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m --- [1] Summary Output ---
[36m(TaskRunner pid=1613436)[0m Yes, the student answer is correct.  
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Step-by-step:  
[36m(TaskRunner pid=1613436)[0m - Initial allowance: $12  
[36m(TaskRunner pid=1613436)[0m - First week: spends $1/3 of $12 = $4 → remaining: $12 - $4 = $8  
[36m(TaskRunner pid=1613436)[0m - Second week: spends $1/4 of $8 = $2 → remaining: $8 - $2 = $6  
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Final amount left: $6  
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m #### 6
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m --- [2] Teacher LogProb Input (Full Context) ---
[36m(TaskRunner pid=1613436)[0m <|im_start|>system
[36m(TaskRunner pid=1613436)[0m You are a helpful math assistant.<|im_end|>
[36m(TaskRunner pid=1613436)[0m <|im_start|>user
[36m(TaskRunner pid=1613436)[0m Problem: Tom receives a $12 allowance per month. In the first week, he spends a third of it; in the second week, he spends a quarter of what he has left. How much money does he have left to finish the month? Let's think step by step and output the final answer after "####".
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Analysis/Hint: Yes, the student answer is correct.  
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Step-by-step:  
[36m(TaskRunner pid=1613436)[0m - Initial allowance: $12  
[36m(TaskRunner pid=1613436)[0m - First week: spends $1/3 of $12 = $4 → remaining: $12 - $4 = $8  
[36m(TaskRunner pid=1613436)[0m - Second week: spends $1/4 of $8 = $2 → remaining: $8 - $2 = $6  
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Final amount left: $6  
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m #### 6
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Based on the analysis above, please provide the correct, step-by-step solution.<|im_end|>
[36m(TaskRunner pid=1613436)[0m <|im_start|>assistant
[36m(TaskRunner pid=1613436)[0m Let's solve this step by step.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m Tom receives a $12 allowance per month.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m **First week:**
[36m(TaskRunner pid=1613436)[0m He spends a third of it.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m A third of $12 is:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m \frac{1}{3} \times 12 = 4
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m So, after the first week, he has:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 12 - 4 = 8 \text{ dollars left}
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m **Second week:**
[36m(TaskRunner pid=1613436)[0m He spends a quarter of what he has left.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m A quarter of $8 is:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m \frac{1}{4} \times 8 = 2
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m So, after spending in the second week, he has:
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 8 - 2 = 6 \text{ dollars left}
[36m(TaskRunner pid=1613436)[0m $$
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m #### 6<|im_end|>
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m ============================================================
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m ==================== Token-Level KL Analysis (Step 1) ====================
[36m(TaskRunner pid=1613436)[0m --- Sequence Alignment Check ---
[36m(TaskRunner pid=1613436)[0m [OK] Token IDs match perfectly.
[36m(TaskRunner pid=1613436)[0m 
[36m(TaskRunner pid=1613436)[0m --- Token-wise KL Breakdown (First 50 tokens) ---
[36m(TaskRunner pid=1613436)[0m Token           | ID     | S_LogP   | T_LogP   | Diff    
[36m(TaskRunner pid=1613436)[0m ---------------------------------------------------------------------------
[36m(TaskRunner pid=1613436)[0m Let             | 10061  | -0.00   | -12.61   | -12.61
[36m(TaskRunner pid=1613436)[0m 's              | 594    | -0.00   | -15.50   | -15.50
[36m(TaskRunner pid=1613436)[0m  solve          | 11625  | -0.04   | -14.15   | -14.11
[36m(TaskRunner pid=1613436)[0m  this           | 419    | -0.01   | -18.05   | -18.04
[36m(TaskRunner pid=1613436)[0m  step           | 3019   | -0.00   | -17.83   | -17.83
[36m(TaskRunner pid=1613436)[0m  by             | 553    | -0.00   | -17.40   | -17.40
[36m(TaskRunner pid=1613436)[0m  step           | 3019   | -0.00   | -17.83   | -17.83
[36m(TaskRunner pid=1613436)[0m .\n\n           | 382    | -0.31   | -13.00   | -12.69
[36m(TaskRunner pid=1613436)[0m Tom             | 24732  | -0.25   | -9.66   | -9.40
[36m(TaskRunner pid=1613436)[0m  receives       | 21189  | -0.00   | -14.84   | -14.83
[36m(TaskRunner pid=1613436)[0m  a              | 264    | -0.00   | -14.08   | -14.08
[36m(TaskRunner pid=1613436)[0m  $              | 400    | -0.01   | -14.63   | -14.61
[36m(TaskRunner pid=1613436)[0m 1               | 16     | -0.00   | -11.31   | -11.31
[36m(TaskRunner pid=1613436)[0m 2               | 17     | -0.00   | -12.61   | -12.61
[36m(TaskRunner pid=1613436)[0m  allowance      | 40358  | -0.00   | -18.77   | -18.77
[36m(TaskRunner pid=1613436)[0m  per            | 817    | -0.00   | -19.28   | -19.28
[36m(TaskRunner pid=1613436)[0m  month          | 2254   | -0.00   | -17.45   | -17.45
[36m(TaskRunner pid=1613436)[0m .\n\n           | 382    | -0.00   | -13.00   | -13.00
[36m(TaskRunner pid=1613436)[0m **              | 334    | -0.00   | -13.63   | -13.63
[36m(TaskRunner pid=1613436)[0m First           | 5338   | -0.85   | -14.77   | -13.93
[36m(TaskRunner pid=1613436)[0m  week           | 2003   | -0.00   | -22.22   | -22.22
[36m(TaskRunner pid=1613436)[0m :               | 25     | -0.25   | -16.95   | -16.70
[36m(TaskRunner pid=1613436)[0m **\n            | 1019   | -0.00   | -14.02   | -14.02
[36m(TaskRunner pid=1613436)[0m He              | 1519   | -0.00   | -13.55   | -13.55
[36m(TaskRunner pid=1613436)[0m  spends         | 37102  | -0.00   | -14.45   | -14.45
[36m(TaskRunner pid=1613436)[0m  a              | 264    | -0.00   | -14.08   | -14.08
[36m(TaskRunner pid=1613436)[0m  third          | 4843   | -0.00   | -18.66   | -18.66
[36m(TaskRunner pid=1613436)[0m  of             | 315    | -0.00   | -15.58   | -15.58
[36m(TaskRunner pid=1613436)[0m  it             | 432    | -0.01   | -16.15   | -16.14
[36m(TaskRunner pid=1613436)[0m .\n\n           | 382    | -0.00   | -13.00   | -13.00
[36m(TaskRunner pid=1613436)[0m A               | 32     | -0.29   | -11.69   | -11.40
[36m(TaskRunner pid=1613436)[0m  third          | 4843   | -0.00   | -18.66   | -18.66
[36m(TaskRunner pid=1613436)[0m  of             | 315    | -0.00   | -15.58   | -15.58
[36m(TaskRunner pid=1613436)[0m  $              | 400    | -0.00   | -14.63   | -14.63
[36m(TaskRunner pid=1613436)[0m 1               | 16     | -0.00   | -11.31   | -11.31
[36m(TaskRunner pid=1613436)[0m 2               | 17     | -0.00   | -12.61   | -12.61
[36m(TaskRunner pid=1613436)[0m  is             | 374    | -0.00   | -14.92   | -14.92
[36m(TaskRunner pid=1613436)[0m :\n             | 510    | -0.00   | -15.54   | -15.54
[36m(TaskRunner pid=1613436)[0m $$              | 14085  | -0.02   | -9.88   | -9.86
[36m(TaskRunner pid=1613436)[0m \n              | 198    | -0.00   | -13.74   | -13.74
[36m(TaskRunner pid=1613436)[0m \               | 59     | -0.00   | -11.16   | -11.16
[36m(TaskRunner pid=1613436)[0m frac            | 37018  | -0.00   | -10.91   | -10.91
[36m(TaskRunner pid=1613436)[0m {               | 90     | -0.00   | -12.13   | -12.13
[36m(TaskRunner pid=1613436)[0m 1               | 16     | -0.00   | -11.31   | -11.31
[36m(TaskRunner pid=1613436)[0m }{              | 15170  | -0.00   | -9.66   | -9.66
[36m(TaskRunner pid=1613436)[0m 3               | 18     | -0.00   | -13.41   | -13.41
[36m(TaskRunner pid=1613436)[0m }               | 92     | -0.00   | -8.97   | -8.97
[36m(TaskRunner pid=1613436)[0m  \              | 1124   | -0.00   | -12.17   | -12.17
[36m(TaskRunner pid=1613436)[0m times           | 15136  | -0.00   | -11.63   | -11.63
[36m(TaskRunner pid=1613436)[0m                 | 220    | -0.00   | -16.90   | -16.90
[36m(TaskRunner pid=1613436)[0m ============================================================
"""
Token-wise KL Breakdown的日志里显示S_LogP都很低，接近0，T_LogP都很大，你分析下原因，是不是哪里出问题了。
''')

这是一个非常敏锐的观察。日志中的数据确实揭示了一个潜在的严重问题，主要表现为 **Student LogProb (`S_LogP`) 异常地接近 0**（例如 `-0.00`），而 **Teacher LogProb (`T_LogP`) 却非常低**（例如 `-12.61`）。

这通常意味着以下两个问题之一或同时存在：
1.  **Temperature（温度）设置不一致**：Student 在计算 LogProb 时可能使用了极低的温度（接近 0），导致分布极度尖锐（Probability $\approx 1.0$），而 Teacher 使用了默认温度（通常为 1.0）。
2.  **Prompt 上下文严重偏移（Context Shift）**：Teacher 看到的 Prompt（包含 Hint 和 Instruction）使得 Student 生成的 Token（例如 "Let"）在 Teacher 眼里变得极度不可能。

以下是详细的分析和修复方案。

---

### 1. 问题分析

#### A. 为什么 `S_LogP` 是 `-0.00`？
*   **现象**：`S_LogP` 为 `-0.00` 意味着模型对该 Token 的预测概率接近 100%。对于一个未经多轮强化学习训练的基座模型（Qwen-Instruct），在 Step 1 就能达到这种确定性是不正常的，除非使用了 **Greedy Decoding** 或者 **极低的 Temperature**。
*   **原因**：
    *   在 `TeacherStudentReflectiveTrainer.fit` 中，Student 的 `batch` 是从 `gen_batch` 继承而来的。
    *   `gen_batch` 的 `meta_info` 包含了生成时的参数。如果在配置文件中 `rollout` 的 `temperature` 设置得很低（或者 `do_sample=False`），那么这个低温度会被带入 `compute_log_prob`。
    *   在 `DP_Actor.compute_log_prob` 中，代码执行了 `logits.div_(temperature)`。如果 `temperature` 例如是 `0.01`，

In [5]:
response = chat('''
如果设置为1.0会破坏我想要训练on policy distillation的目的吗？
''')

这是一个非常深刻且关键的问题。

简短的回答是：**不会破坏，恰恰相反，设置为 1.0 是实现正确的 On-Policy Distillation（以及 PPO/KL 散度计算）的必要条件。**

如果不设置为 1.0（特别是你现在的日志显示 Student 似乎用了极低的温度），你的 Distillation 实际上是**失效**的。

以下是详细的数学和逻辑解释：

### 1. 区分“生成温度”与“计算温度”

你需要区分两个阶段的 Temperature：

1.  **Rollout / Generation 阶段（生成时）**：
    *   这里 Temperature 控制**探索性**。
    *   如果你设置 $T=0.1$，Student 会生成非常确定性的回答。
    *   如果你设置 $T=1.0$，Student 会生成多样性的回答。
    *   **On-Policy 的含义**：指数据必须是由当前 Student 策略生成的。只要数据是 Student 产生的，无论生成时用了什么温度，它都是 On-Policy 数据。

2.  **Forward / Compute LogProb 阶段（计算概率时）**：
    *   这里 Temperature 决定了**概率分布的形状**。
    *   Distillation（蒸馏）的核心数学目标是最小化两个分布的距离（通常是 KL 散度）。
    *   模型的**真实分布**是由原始 Logits 定义的（即 $T=1.0$）。

### 2. 为什么计算概率时必须是 T=1.0？

假设 Student 输出的 Logits 为 $z_S$，Teacher 输出的 Logits 为 $z_T$。

#### 情况 A：如果你使用 T=1.0 (标准做法)
概率 $P(x) = \text{Softmax}(z)$。
LogProb $\log P(x) \approx z_x - \text{Const}$。
你的 Reward (Reverse KL) 近似于：
$$ R \approx \log P_T(x) - \log P_S(x) \approx z_T - z_S $$
这意味着：**你要拉近 Student 的 Logits 和 Teacher 的

In [55]:
response = chat('''
修改之后日志如下
"""
[36m(TaskRunner pid=344141)[0m --- [1] Summary Output ---
[36m(TaskRunner pid=344141)[0m Let's verify the student's reasoning step by step:
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m 1. **Trade-in value**:  
[36m(TaskRunner pid=344141)[0m    80% of $150 = $120 → correct.
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m 2. **Net cash paid**:  
[36m(TaskRunner pid=344141)[0m    $80 - $10 = $70 → correct.
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m 3. **Game value**:  
[36m(TaskRunner pid=344141)[0m    $30 game received for free → this is value Tom receives.
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m 4. **Total value Tom receives**:  
[36m(TaskRunner pid=344141)[0m    Trade-in ($120) + game ($30) = $150 → correct.
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m 5. **Net cost to Tom**:  
[36m(TaskRunner pid=344141)[0m    Cash paid: $70  
[36m(TaskRunner pid=344141)[0m    Value received: $30 (game)  
[36m(TaskRunner pid=344141)[0m    So net outflow = $70 - $30 = $40
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m 6. **What is the price of the NES?**  
[36m(TaskRunner pid=344141)[0m    The NES was purchased with:  
[36m(TaskRunner pid=344141)[0m    - A $120 trade-in credit (store gives it to him)  
[36m(TaskRunner pid=344141)[0m    - $70 cash paid  
[36m(TaskRunner pid=344141)[0m    - He gets $30 in value (the game)  
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m    The trade-in credit offsets part of the price.  
[36m(TaskRunner pid=344141)[0m    The price of the NES must equal the amount Tom effectively pays, which is:  
[36m(TaskRunner pid=344141)[0m    Net cash paid ($70) minus value of free item ($30) = $40  
[36m(TaskRunner pid=344141)[0m    But the store gives $120 in trade-in — this is money *received* by Tom, not paid.  
[36m(TaskRunner pid=344141)[0m 
[36m(TaskRunner pid=344141)[0m    So the total value of the transaction is $150 (trade-in + game).  
[36m(TaskRunner pid=344141)[0m    The only outflow is $70 in cash.  
[36m(TaskRunner pid=344141)[0m    Therefore, the NES must have been priced at $150 — because that’s the total value of what Tom received (in value), and the means of payment were $70
"""
Summary Output看起来还是截断了，我已经设置了token限制是1024，但看起来没有生效？ 日志里的输出远不到1024就截断了
''')

这是一个非常奇怪的现象。文本看起来只有几百个 Token，远未达到 1024 的限制，却突然截断了。

这通常有三种可能：
1.  **配置未生效**：代码读取 `max_response_length` 失败，回退到了某个默认值（比如 256）。
2.  **EOS 误判**：模型生成了一个特殊的 Token，被 Tokenizer 判定为结束，或者被 `skip_special_tokens=True` 隐去了，导致看起来像截断。
3.  **vLLM 内部限制**：vLLM 启动时可能锁定了最大生成长度。

为了彻底解决这个问题，请在 `_prepare_summary_generation_batch` 中做以下 **3 点修改**：

1.  **强制硬编码一个大数值**（例如 4096），排除配置读取错误的可能性。
2.  **打印调试信息**：输出 Summary 的 Prompt 长度和设置的 `max_new_tokens`，确信参数传进去了。
3.  **增加 `ignore_eos=False`**：显式告诉 vLLM 不要忽略结束符（虽然默认就是 False，但强调一下）。

请修改 `teacher_student_reflective_trainer.py`：

```python
    def _prepare_summary_generation_batch(self, batch: DataProto, raw_prompts: List[str], ground_truths: List[str]) -> DataProto:
        # ... (前文不变) ...

        # === FIX: 增强 Summary 生成的鲁棒性 ===
        
        # 1. 尝试读取配置，如果读不到，直接给 4096
        # 注意：OmegaConf 有时需要用 .key 访问，有时用 get
        try:
            max_tokens = self.config.data.max_response_length
        except:
            max_tokens = 4096
            
        # 强制保底，防

In [58]:
response = chat('''
我的意思是metric里val-core/openai/gsm8k/reward/mean@1这个具体计算逻辑是什么样的，代码是在哪里写的？
''')

这个指标的计算逻辑分为两步：**单题打分** 和 **汇总平均**。

具体的代码位置和逻辑如下：

### 1. 汇总平均的逻辑 (Mean Calculation)
**位置**：`verl/trainer/ppo/metric_utils.py`
**函数**：`process_validation_metrics` 和 `compute_score_metrics`

这是生成 `mean@1` 这个数值的地方。逻辑非常简单，就是对所有样本的分数取平均值。

```python
# verl/trainer/ppo/metric_utils.py

def compute_score_metrics(scores, n=1):
    """
    scores: 一个列表，包含所有验证集样本的得分 (0.0 或 1.0)
    n: 每个 Prompt 采样的回复数量 (验证集通常 n=1)
    """
    total = len(scores)
    # 核心逻辑：直接用 numpy 求平均
    # 这就是 mean@1 的来源
    mean = np.mean(scores) 
    
    return {
        f"mean@{n}": mean,  # -> "mean@1"
        # ... 其他如 min, max ...
    }

def process_validation_metrics(data_sources, ...):
    # ...
    # 根据 data_source (例如 "openai/gsm8k") 对分数进行分组
    # 然后调用上面的 compute_score_metrics
    # ...
```

### 2. 单题打分的逻辑 (0/1 Logic)
**位置**：`verl/utils/reward_score/gsm8k.py` (通常情况下)
**函数**：`compute_score`

这是决定每个样本是得 1 分还是 0 分的地方。逻辑是**正则匹配**。

```python
# verl/utils/reward_score/gsm8k.py

def compute_score(solution_str, ground_tr

In [33]:
response = chat('''
我在启动脚本里传了 trainer.test_freq=60, 但是日志里没有测试指标，
能在metric里加入测试集的评测信息吗？
''')

这是一个很常见的问题。原因很简单：你在重写 `TeacherStudentReflectiveTrainer` 的 `fit` 函数时，**把父类 `RayPPOTrainer` 中关于 Validation (测试) 的代码块给漏掉了**。

虽然你在启动脚本里传了参数，且 `main` 函数里也初始化了 `val_reward_fn`，但因为 `fit` 循环里没有调用它，所以永远不会执行测试。

请在 `verl/trainer/ppo/teacher_student_reflective_trainer.py` 的 `fit` 函数中补上这段逻辑。

### 修改位置

在 `fit` 函数的末尾，**在 `logger.log` 之前**，插入验证逻辑。

```python
    def fit(self):
        # ... (前面的代码保持不变) ...

                with marked_timer("update_actor", timing_raw):
                    actor_output = self.actor_rollout_wg.update_actor(batch)
                    actor_output_metrics = reduce_metrics(actor_output.meta_info["metrics"])
                    metrics.update(actor_output_metrics)

                # =================================================================
                # === FIX: 添加 Validation/Testing 逻辑 ===
                # =================================================================
                # 检查是否满足测试条件：
                # 1. 存在验证集 Reward Function
              

In [34]:
response = chat('''
actor/entropy一直在4.5附近波动，这正常吗？ 感觉做强化学习的时候没有那么大
''')

**结论：在 LLM 的 On-Policy Distillation（或 PPO）训练初期，Actor Entropy 维持在 4.5 左右是完全正常的，甚至是一个健康的信号。**

你感觉“偏高”，可能是因为习惯了较小的动作空间（如 Atari 游戏动作空间只有几百，Entropy 通常在 1.0-2.0）或者传统的 NLP 分类任务。

以下是详细的分析，告诉你为什么 4.5 是合理的，以及如何通过它判断训练状态。

### 1. 数学直觉：4.5 意味着什么？

熵（Entropy）衡量的是**不确定性**。公式是 $H(x) = - \sum p(x) \ln p(x)$。

*   **最大熵（完全随机）**：Qwen 的词表大小约为 152,000。
    $$ H_{max} = \ln(152,000) \approx 11.9 $$
    如果 Entropy 接近 11-12，说明模型坏了，在输出均匀噪声。
*   **最小熵（One-Hot/完全确定）**：
    $$ H_{min} = 0 $$
    如果 Entropy 接近 0，说明模型发生了**模式坍塌（Mode Collapse）**，只会输出固定的几句话，失去了多样性。

*   **当前值 4.5**：
    $$ e^{4.5} \approx 90 $$
    这大致意味着模型在每一步生成时，平均在考虑约 **90 个合理的候选 Token**。
    考虑到 Qwen 的词表很大，且自然语言（尤其是思维链 CoT）本身就有多种表达方式（比如 "First," vs "To start," vs "Step 1:"），保留一定的“困惑度”是正常的。

### 2. 为什么比“传统 RL”高？

在你的场景（On-Policy Distillation）中，Entropy 较高的原因主要有两点：

#### A. 蒸馏损失 (Distillation Loss) 的特性
你使用的 Loss 本质上是 **KL Divergence**（或者 Reverse KL）。
*   **纯 PPO (Sparse Reward)**：模型只要找到**一条**能拿满分的路径，就会疯狂强化这条路径，导致概率分布迅速尖锐化，Entropy 骤降（可能降到 1.0 以下）。这

In [35]:
response = chat('''
学习的过程中reward/reflection_kl最初step 1在-0.05，然后快速上升，step 12上升到-0.03，然后一直在0.03附近波动，这个正常吗？
''')

**结论：非常正常，而且这个曲线揭示了你的蒸馏过程正在按预期工作。**

这不仅是正常的，而且从数学和原理上解释，这正是 **Teacher-Student Distillation（尤其是 Student 和 Teacher 源自同一基座模型时）** 应有的表现。

以下是详细的分析：

### 1. 为什么是负数？ (-0.05)
你的 Reward 计算公式是：
$$ R = \log P_{\text{Teacher}}(y|x, \text{Hint}) - \log P_{\text{Student}}(y|x) $$

*   **数据来源**：Token $y$ 是由 **Student** 生成的（On-Policy）。
*   **自洽性偏差**：对于一个语言模型来说，它通常对自己生成的 Token 赋予最高的概率（或者接近最高）。
*   **Teacher 的视角**：Teacher 虽然有 Hint，但它面对的是 Student 生成的 Token。除非 Student 的生成完美符合 Teacher 结合 Hint 后的预期，否则 Teacher 对该 Token 的概率通常会略低于 Student 自己的自信度。
    *   即 $P_S \ge P_T \Rightarrow \log P_T - \log P_S \le 0$。
*   **因此**：Reward 略微为负（-0.05）是符合直觉的，表示 Student 对自己的生成比 Teacher 更“自信”一点点，或者两者分布非常接近。

### 2. 为什么会上升？ (-0.05 -> -0.03)
*   **优化目标**：你的训练目标是最大化 Reward，也就是最小化 $P_S$ 和 $P_T$ 之间的距离（Reverse KL）。
*   **上升的含义**：从 -0.05 上升到 -0.03，意味着差距变小了。Student 正在调整自己的参数，使其概率分布更接近 Teacher（拥有上帝视角）的分布。
*   **训练有效**：这证明梯度正在正确传导，Student 正在学习。

### 3. 为什么在 -0.03 处波动/停滞？ (Information Gap)
这是最有趣的一点。为什么它不变成 0 甚至正数？

*   **信息不对称（Informat

In [38]:
response = chat('''
日志中还输出了actor/pg_loss，这个有影响吗？我应该没用这个？
''')

**结论：完全没有影响，也不用担心。这只是变量命名的复用。**

在你的代码逻辑中，`actor/pg_loss` **实际上就是你的 Distillation Loss**。

### 1. 为什么会出现这个日志？

请回看你修改过的 `dp_actor.py` 代码：

```python
                    # ... (你的修改) ...
                    if loss_mode == "distillation":
                        # 直接最大化 (advantages * log_prob)
                        # 这里 pg_loss 被赋值为你定义的蒸馏损失
                        pg_loss = -verl_F.masked_mean(advantages * log_prob, response_mask)
                        
                        # 占位符
                        pg_clipfrac = torch.tensor(0.0, device=pg_loss.device)
                        # ...

                    elif loss_mode == "vanilla":
                        # ... (PPO 的逻辑) ...
                        pg_loss, ... = compute_policy_loss(...)

                    # ... (后续代码) ...
                    
                    # 这里的 metrics 字典 Key 是写死的字符串 "actor/pg_loss"
                    micro_batch_metrics.update(
                        {
                            "actor/pg_loss": 

In [39]:
response = chat('''
pg_loss = -verl_F.masked_mean(advantages * log_prob, response_mask)
这个改动符合我的onpolicy distillation的逻辑吗？ 我是想让student去学teacher的logit？
''')

**结论：完全符合。你的写法是标准的“基于 Policy Gradient 的 Reverse KL 最小化”。**

虽然你没有直接对 Logits 进行均方误差（MSE）回归（那是 Offline Distillation 的做法），但在 On-Policy（强化学习）的框架下，你的写法是在数学上等价于让 Student 的概率分布去逼近 Teacher。

以下是详细的数学推导和直观解释，证明你的写法是对的。

### 1. 数学证明

你的目标是最小化 Student ($S$) 和 Teacher ($T$) 之间的 **Reverse KL Divergence**：
$$ J(\theta) = D_{KL}(P_S || P_T) = \sum_y P_S(y|x) \log \frac{P_S(y|x)}{P_T(y|x)} $$
$$ J(\theta) = \mathbb{E}_{y \sim P_S} [ \log P_S(y|x) - \log P_T(y|x) ] $$

我们需要计算关于 Student 参数 $\theta$ 的梯度 $\nabla_\theta J(\theta)$。利用 Log-Derivative Trick (REINFORCE 技巧)：

$$ \nabla_\theta J(\theta) = \mathbb{E}_{y \sim P_S} \left[ \nabla_\theta \log P_S(y|x) \cdot \underbrace{(\log P_S(y|x) - \log P_T(y|x))}_{\text{KL Difference term}} \right] $$

现在看你的代码：
*   **Loss**: `pg_loss = - mean(advantages * log_prob)`
*   **Gradient**: $\nabla L = - \mathbb{E} [ \text{advantages} \cdot \nabla \log P_S ]$
*   **Advantages**: 在你的 `fit` 函数中，`advantages` 被赋值为 `token_level_rewards`。
*   **Rewards**: 计算方式为 `kl_diff = lo

In [40]:
response = chat('''
我还想在metric中加入几个关键调试信息
"""
1. teacher中token最大/最小的logit
2. student中token最大/最小的logit
3. teacher和student logit 最大/最小的diff
"""
''')

这是一个非常实用的需求。监控 Logits 的极值和差异分布，能帮你快速判断训练是否异常（比如梯度爆炸、Teacher 过于强势、或者 Student 彻底躺平）。

请在 `verl/trainer/ppo/teacher_student_reflective_trainer.py` 的 `fit` 函数中，在计算完 `kl_diff` 之后，插入以下统计逻辑。

### 修改代码

找到 `fit` 函数中计算 `kl_diff` 的地方，插入标记为 `=== FIX: Add Debug Metrics ===` 的代码块。

```python
                    # ... (Step 3.6 Reward Calculation) ...
                    token_level_rewards = torch.zeros_like(student_full_log_probs)
                    s_probs = student_full_log_probs
                    t_probs = teacher_full_log_probs_aligned # 确保这里用的是对齐后的！
                    
                    min_len = min(s_probs.shape[1], t_probs.shape[1])
                    s_part = s_probs[:, :min_len]
                    t_part = t_probs[:, :min_len]
                    
                    kl_diff = t_part - s_part
                    
                    # =================================================================
                    # === FIX: Add Debug Metrics (Logits Statistics) ===
        

In [52]:
response = chat('''
现在运行过程中metric发现debug/student_logp_min有-14，这么小的logit是如何采样出来的呢？
然后我发现我把sequence length增大了之后 entropy从4.5提升到了6.5，这正常吗？
''')

### 问题 1：为什么 `debug/student_logp_min` 会有 -14？

你可能会觉得：“如果一个词的 LogProb 是 -14 ($P \approx 8 \times 10^{-7}$)，采样时怎么可能选中它？”

**原因有三点：**

1.  **Top-P (Nucleus) Sampling 的长尾效应**：
    *   虽然 -14 很小，但如果 `top_p` 设置为 0.9 或 1.0（默认通常是 1.0），且分布非常平坦（Entropy 高），那么长尾中的低概率词也是有机会被选中的。
    *   更常见的是：**采样时的 Temperature $\neq$ 计算 LogProb 时的 Temperature**。
        *   **生成时**：假设你用了 `temperature=0.8`。原本 -14 的 Logits 会被放大，概率分布会变，虽然还是很低，但如果上下文合适，有可能被选。
        *   **计算时**：我们强制用了 `temperature=1.0`。在原始 Logits 下，这个词可能就是这么低分。

2.  **Teacher 和 Student 的分布差异 (最可能的原因)**：
    *   注意你的 Metric 是 `debug/student_logp_min`。
    *   这是 Student 模型对 **自己生成的 Token** 的打分。
    *   **Wait...** 如果是 Student 自己生成的，它通常应该比较自信（比如 -0.5）。为什么它会对自己生成的词打 -14 分？
    *   这通常发生在 **"被迫"生成** 的情况下。
        *   例如：Prompt 强制要求某种格式，或者之前的 Token 已经把路走窄了，导致下一个 Token 即使是“最优解”，其绝对概率也很低（因为模型很困惑）。
        *   或者：**EOS Token**。在长文本生成被迫截断或者逻辑结束时，模型可能对何时输出 `<|im_end|>` 感到非常纠结，导致该 Token 概率极低。

3.  **计算对齐的边缘效应**：
    *   虽然我们对齐了 Padding，但如果 `response_mask` 在边界处

In [53]:
response = chat('''
我发现运行过程中summary出现截断的现象，如何增大summary的长度限制
''')

Summary 截断通常是因为 `generate_sequences` 时传入的 `max_new_tokens` 限制太紧。

在你的代码 `teacher_student_reflective_trainer.py` 的 `_prepare_summary_generation_batch` 函数中，你硬编码了一个限制：

```python
        summary_batch.meta_info = {
            "do_sample": True,
            "temperature": 0.1, 
            "max_new_tokens": 512, # <--- 罪魁祸首
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id
        }
```

### 修复方案

1.  **直接增大数值**：将 512 改为 1024 或 2048。
2.  **从 Config 读取**：最好从配置文件中读取，这样不用改代码就能调参。

**建议修改代码如下：**

```python
    def _prepare_summary_generation_batch(self, batch: DataProto, raw_prompts: List[str], ground_truths: List[str]) -> DataProto:
        # ... (前文不变) ...

        # 尝试从 config 中读取，如果没有则默认 1024
        # 假设你在 config.actor_rollout_ref.rollout 下加了一个 summary_max_new_tokens
        # 或者直接复用 data.max_response_length
        
        max_tokens = self.config.data.get("max_response_length", 1024)
        
        summary_batch.meta_inf

In [41]:
response = chat('''
现在的代码实现是每次rollout几条数据进行蒸馏呢？
''')

根据你提供的启动脚本和代码逻辑，每次 Rollout（生成并进行蒸馏）的数据量是 **256 条**。

具体的计算逻辑如下：

### 1. 核心参数
在你的启动脚本中：
```bash
data.train_batch_size=256
```
这是控制每次 PPO/蒸馏迭代（Iteration/Step）处理的总 Prompt 数量的核心参数。

### 2. 代码逻辑解析
在 `RayPPOTrainer`（你的 `TeacherStudentReflectiveTrainer` 的父类）的 `_create_dataloader` 方法中：

```python
self.train_dataloader = StatefulDataLoader(
    dataset=self.train_dataset,
    # 如果没设置 gen_batch_size，默认使用 train_batch_size
    batch_size=self.config.data.get("gen_batch_size", self.config.data.train_batch_size),
    # ...
)
```

在你的 `fit` 循环中：
```python
# 每次从 Dataloader 取出一个 batch
for batch_dict in self.train_dataloader:
    batch = DataProto.from_single_dict(batch_dict)
    
    # 获取 N (每个 Prompt 生成几个回答，默认为 1)
    N = self.config.actor_rollout_ref.rollout.n 
    
    # ...
    
    # Student 生成 (Rollout)
    gen_batch_output = self.actor_rollout_wg.generate_sequences(gen_batch)
```

### 3. 最终计算
*   **Prompts 数量**：256 (来自 `data.train_batch_size`)
*   **每条 Prompt 生成数 (N)**：通常默认为 1 (除非你在 config 文件里改了

In [45]:
response = chat('''
我想在日志中每个step输出的调试信息中，加入答案是否正确的标记
''')

这是一个非常有用的功能，能让你直接在滚动日志中看到 Student 是否做对了，而不仅仅是看 LogProb。

我们需要做两件事：
1.  **实现一个简单的答案提取函数**（针对 GSM8K 的 `####` 格式）。
2.  **在 Debug 打印模块中调用它**，对比 Student 回答和 Ground Truth。
3.  **(可选但推荐)** 计算当前 Batch 的训练集准确率（Training Accuracy）并记录到 Metric 中。

请修改 `teacher_student_reflective_trainer.py` 的 `fit` 函数。

### 1. 添加提取函数 (放在 `TeacherStudentReflectiveTrainer` 类内部或外部)

```python
def extract_answer(text: str) -> str:
    """从文本中提取 #### 后的答案，用于简单的正确性匹配"""
    if "####" in text:
        return text.split("####")[-1].strip()
    return "[No Answer]"
```

### 2. 修改 `fit` 函数中的 Debug 部分

找到 `=== Log Part 1: Generation Content ===` 区域，插入以下代码：

```python
                    # ... (Step 3.5 Student Computes LogProb 之后) ...

                    # === Log Part 1: Generation Content ===
                    if True:
                        print(f"\n{'='*20} Teacher-Student Reflection Debug (Step {self.global_steps}) {'='*20}")
                        try:
                            idx = 0 
                         

In [49]:
response = chat('''
现在是4卡做student，4卡做teacher，均衡分配的，但我看到有4张卡经常空置，感觉是不是其中某一项的GPU任务会更重一些？这样均衡分配不合理？
''')

你的观察非常敏锐，而且你的直觉是完全正确的。

**目前的 4:4 分配是非常不合理的，导致了巨大的资源浪费。**

### 1. 为什么 4:4 不合理？

让我们对比一下 **Student** 和 **Teacher** 在一个训练 Step 中的工作量：

*   **Student (Actor) 的工作：**
    1.  **Generate (Rollout)**: 生成 256 条回答（这是 Token-by-Token 的，非常慢）。
    2.  **Generate (Summary)**: 生成 256 条摘要（同样是 Token-by-Token）。
    3.  **Forward (Student LogProb)**: 计算一次概率。
    4.  **Backward (PPO Update)**: **最重的一步**。需要计算梯度并更新参数，计算量通常是 Forward 的 3 倍。
*   **Teacher (RefPolicy) 的工作：**
    1.  **Forward (Teacher LogProb)**: 仅仅计算一次概率（Parallel Forward）。不生成，不反向传播。

**结论**：Student 的计算负载是 Teacher 的 **10 倍以上**。
目前的状况是：Teacher 的 4 张卡瞬间算完，然后“围观”Student 的 4 张卡在那儿哼哧哼哧地生成和训练。

### 2. 推荐的分配方案

既然你的配置中 `tensor_model_parallel_size=1`（即使用 Data Parallelism，单卡跑模型），我们可以随意调整 GPU 数量，不受模型 Head 数能否整除的限制。

**强烈建议改为：Student 7卡 : Teacher 1卡**
或者保守一点：**Student 6卡 : Teacher 2卡**

对于 4B 这种小模型，1 张卡做 Teacher (Inference Only) 处理 256 的 Batch Size 绰绰有余（显存不够可以调小 Teacher 的 `micro_batch_size`，反正它算得快，多切几份也比 Student 快）。

### 3. 代码修改方案

请修改 `teacher_student_re

In [48]:
del chat_history[-1]

In [37]:
import numpy as np
np.exp(-0.035)

np.float64(0.9656054162575665)

In [32]:
print(chat_history[-3].content)

这是最关键的一步。之前的修改其实已经让 `DP_Actor` 正确计算了概率（因为 Left Padding 配合了切片逻辑），但**返回的结果仍然是 Left Padded 的**。

而你的 Student 数据（以及后续的 PPO 逻辑、日志打印）全部假设数据是 **Right Padded**（有效数据在左，Padding 在右）。

这就导致了：
1.  **日志误导**：你打印第 0 个位置，Student 是 "Let"，Teacher 是 "Pad"。你看到的是 "Pad" 的概率（-14.82），却以为是 "Let" 的概率。
2.  **Reward 错误**：直接相减 `T - S`，等于用 `LogP(Pad)` 减去 `LogP(Let)`，完全算错。

### 解决方案：在 `fit` 中对齐 Tensor

我们需要在拿到 `teacher_full_log_probs` 后，立刻将其**从 Left Padding 转为 Right Padding**，使其与 Student 完美对齐。

请修改 `teacher_student_reflective_trainer.py` 的 `fit` 函数。

#### 1. 添加一个辅助函数 (放在类外或类内均可)

```python
def left_to_right_padding(tensor: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
    """
    将 Left Padded 的 Tensor 转换为 Right Padded。
    tensor: [B, L]
    mask: [B, L] (1 for valid, 0 for pad)
    """
    B, L = tensor.shape
    device = tensor.device
    
    # 计算每个样本的有效长度
    lens = mask.sum(dim=-1).long() # [B]
    
    # 创建新的 Right Padded Tensor
    new_tensor = torch.zeros_like(tensor)
    
    for i in range(B):
       

In [100]:
response = chat('''
修改之后报错
"""
        please increase max_num_batched_tokens or disable chunked prefill
Error executing job with overrides: ['data.train_files=/data/kcl/lpy/data/OpenThoughts/train_verl.parquet', 'data.val_files=[/data/kcl/lpy/data/AIME_2024/test_verl.parquet,/data/kcl/lpy/data/AIME_2025/test_verl.parquet,/data/kcl/lpy/data/HMMT_FEB_2025/test_verl.parquet]', 'data.train_batch_size=56', 'data.max_prompt_length=4096', 'data.max_response_length=38912', '+data.summary_max_new_tokens=38912', 'actor_rollout_ref.model.path=/data/kcl/lpy/models/qwen3_4b_instruct', '+actor_rollout_ref.model.torch_dtype=bfloat16', 'actor_rollout_ref.actor.optim.lr=5e-7', 'actor_rollout_ref.actor.ppo_mini_batch_size=28', 'actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu=2', 'actor_rollout_ref.actor.policy_loss=distillation', 'actor_rollout_ref.rollout.name=vllm', 'actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu=4', 'actor_rollout_ref.rollout.tensor_model_parallel_size=1', 'actor_rollout_ref.rollout.gpu_memory_utilization=0.4', 'actor_rollout_ref.rollout.max_num_batched_tokens=40000', 'actor_rollout_ref.ref.log_prob_micro_batch_size_per_gpu=2', 'critic.optim.lr=1e-5', 'critic.model.path=/data/kcl/lpy/models/qwen3_4b_instruct', '+critic.model.torch_dtype=bfloat16', 'critic.ppo_micro_batch_size_per_gpu=4', 'algorithm.kl_ctrl.kl_coef=0.001', 'trainer.logger=console', 'trainer.val_before_train=False', 'trainer.n_gpus_per_node=8', 'trainer.nnodes=1', 'trainer.save_freq=30', 'trainer.test_freq=30', "trainer.logger=['swanlab', 'console']", 'trainer.project_name=self_reflect', 'trainer.experiment_name=sr_dl_teacher_student_openthoughts_qwen3_4b_instruct_alleval', 'trainer.default_local_dir=/data/kcl/lpy/verl/verl_checkpoints/sr_dl_teacher_student_openthoughts_qwen3_4b_instruct_alleval', 'trainer.total_epochs=15']
Traceback (most recent call last):
  File "/data/kcl/lpy/verl/verl/trainer/main_self_reflective.py", line 41, in main
    run_ppo(config)
  File "/data/kcl/lpy/verl/verl/trainer/main_self_reflective.py", line 75, in run_ppo
    ray.get(runner.run.remote(config))
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/worker.py", line 2967, in get
    values, debugger_breakpoint = worker.get_objects(
                                  ^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/worker.py", line 1015, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::TaskRunner.run() (pid=3248513, ip=192.168.131.187, actor_id=1fdf7901d5079229fd44cfae01000000, repr=<main_self_reflective.TaskRunner object at 0x7fefa300fd70>)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/trainer/main_self_reflective.py", line 243, in run
    trainer.init_workers()
  File "/data/kcl/lpy/verl/verl/trainer/ppo/teacher_student_reflective_trainer.py", line 198, in init_workers
    self.actor_rollout_wg.init_model()
  File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 50, in __call__
    output = ray.get(output)
             ^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^
                                  ^^^^^^^^^^^^^^^^^^^
ray.exceptions.RayTaskError(ValueError): ray::WorkerDict.actor_rollout_init_model() (pid=3250675, ip=192.168.131.187, actor_id=8ab9ac0bb6d5e3d71852dae201000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7ef0a4bea720>)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
    return getattr(self.worker_dict[key], name)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
    self.rollout, self.rollout_sharding_manager = self._build_rollout(
                                                  ^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
    rollout = vllm_rollout_cls(
              ^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
    raise ValueError(
ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.
(TaskRunner pid=3248513) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3249599, ip=192.168.131.187, actor_id=773eb9ddd45817c5b9df55bc01000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7ee9074ae780>)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3248513)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3248513)     return func(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3248513)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3248513)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3248513)     rollout = vllm_rollout_cls(
(TaskRunner pid=3248513)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3248513)     raise ValueError(
(TaskRunner pid=3248513) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
(TaskRunner pid=3248513) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3250680, ip=192.168.131.187, actor_id=87c0b9404342b2232e78ee2101000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7ee83a9edb80>)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3248513)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3248513)     return func(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3248513)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3248513)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3248513)     rollout = vllm_rollout_cls(
(TaskRunner pid=3248513)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3248513)     raise ValueError(
(TaskRunner pid=3248513) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
(TaskRunner pid=3248513) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3250679, ip=192.168.131.187, actor_id=a20cf69b4a4a413f4e54919b01000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7fc61bfbad20>)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3248513)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3248513)     return func(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3248513)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3248513)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3248513)     rollout = vllm_rollout_cls(
(TaskRunner pid=3248513)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3248513)     raise ValueError(
(TaskRunner pid=3248513) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
(TaskRunner pid=3248513) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3250677, ip=192.168.131.187, actor_id=36a7c2af2fce7e97e3606c7301000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7f54f784ee70>)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3248513)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3248513)     return func(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3248513)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3248513)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3248513)     rollout = vllm_rollout_cls(
(TaskRunner pid=3248513)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3248513)     raise ValueError(
(TaskRunner pid=3248513) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
(TaskRunner pid=3248513) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3250676, ip=192.168.131.187, actor_id=340592e3d978c4a7d945e6d101000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7ef007e6e9f0>)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3248513)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3248513)     return func(*args, **kwargs)
(TaskRunner pid=3248513)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3248513)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3248513)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3248513)     rollout = vllm_rollout_cls(
(TaskRunner pid=3248513)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3248513)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3248513)     raise ValueError(
(TaskRunner pid=3248513) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
"""
怎么解决
''')

这是一个配置参数的数学逻辑冲突。

### 错误原因分析

```text
ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len
```

1.  **最大上下文长度 (`max_model_len`)**：
    Verl 会根据你的数据配置计算模型需要的最大序列长度：
    $$ \text{Max Model Len} = \text{Max Prompt} + \text{Max Response} $$
    $$ 4096 + 38912 = \mathbf{43008} $$

2.  **当前批处理上限 (`max_num_batched_tokens`)**：
    你设置了：`actor_rollout_ref.rollout.max_num_batched_tokens=40000`

3.  **冲突**：
    $$ 40000 < 43008 $$
    vLLM（在 Verl 的封装下）要求 `max_num_batched_tokens` 必须至少能容纳一个完整的最大长度序列，否则无法保证能处理最坏情况下的长文本。

### 解决方案

请在启动脚本中，将 `max_num_batched_tokens` 增加到大于 **43008** 的数值。

**建议设置为 45000 或 48000。**

#### 修改启动脚本

找到 `actor_rollout_ref.rollout.max_num_batched_tokens` 这一行，修改如下：

```bash
# 原来是 40000，改为 45000
actor_rollout_ref.rollout.max_num_batched_tokens=45000
```

---

### 额外建议：关于 Summary 的长度

我注意到你还设置了：
`+data.summary_max_new_tokens=38912`

**这可能会导致 Teacher 节点 OOM。**
*   Teacher 负责生成 Summary（对 Student 的回答进行点评）。
*   Teacher 只有 1 张显卡（在 7:1 配置下）。
*   Su

In [102]:
response = chat('''
为啥把max_num_batched_tokens设置成45000之后运行的非常慢
''')

将 `max_num_batched_tokens` 设置得很大（如 45000）会导致运行变慢，主要原因在于 **vLLM 的内存管理机制** 和 **GPU 计算效率** 的变化。

### 1. KV Cache 碎片化与调度开销
*   **大内存池**：当你设置 45000 时，vLLM 会尝试在显存中预留能容纳 45000 个 Token 的 KV Cache 空间（Block Table）。
*   **碎片化**：在生成过程中，不同的 Sequence 长度不一（有的 100 Token，有的 30k Token）。vLLM 需要频繁地分配和回收这些 Block。
*   **调度压力**：由于总 Token 数上限很高，vLLM 可能会尝试在一个 Batch 中塞入更多的小请求，或者不得不为了一个超长请求而驱逐其他请求的 Cache。这种复杂的调度逻辑会消耗 CPU 时间。

### 2. Attention 计算的二次方复杂度 (虽有 FlashAttn)
*   虽然 FlashAttention 优化了显存，但计算量依然与序列长度相关。
*   如果你的 Batch 中混入了一个 30k 长度的请求，整个 Batch 的计算时间会被这个长请求拖慢（木桶效应）。
*   在 `max_num_batched_tokens` 很大时，vLLM 倾向于“能塞就塞”，导致单次 Forward 的计算延迟显著增加。

### 3. Chunked Prefill 的副作用
*   你开启了 Chunked Prefill（因为上下文太长）。
*   当 Token 数上限很高时，vLLM 可能会选择更大的 Chunk Size 进行 Prefill。
*   这虽然避免了 OOM，但会导致 GPU 计算流（Stream）被长时间占用，阻塞了 Decode 阶段的小 Token 生成，导致整体吞吐量（Tokens/sec）下降，虽然利用率看起来很高。

### 4. 显存带宽瓶颈 (Memory Bound)
*   4B 模型本身计算量不大，但在处理 30k+ 上下文时，瓶颈从计算转移到了显存带宽。
*   加载庞大的 KV Cache 需要大量带宽。
*   如果 `max_num_batched_tokens` 设置过大，导致 Cache Miss

In [101]:
response = chat('''
我想把teacher最后forward的input再增加拼接上groundtruth到summary之后，你需要修改代码
''')

这是一个非常好的想法！将 Ground Truth (Standard Answer) 显式地包含在 Teacher 的 Forward 上下文中，能进一步增强 Teacher 的“上帝视角”能力，使其 LogProb 对正确推理路径更敏感。

我们需要修改 `teacher_student_reflective_trainer.py` 中的 `_prepare_teacher_forward_batch` 函数。

### 修改逻辑

1.  **原逻辑**：
    *   System Prompt: Hint (Summary) + Instruction
    *   User: Question
    *   Assistant: Student Response (Target)

2.  **新逻辑**：
    *   System Prompt: **Hint (Summary) + Ground Truth** + Instruction
    *   User: Question
    *   Assistant: Student Response (Target)

### 代码修改

请替换 `_prepare_teacher_forward_batch` 函数中的 System Prompt 构造部分。

```python
    def _prepare_teacher_forward_batch(self, batch: DataProto, summaries: torch.Tensor) -> DataProto:
        # ... (前文不变: 获取 prompts, responses, cleaning ...) ...

        # 注意：你需要在这里获取 ground_truths
        # fit 函数调用时并没有传 ground_truths 进来，我们需要修改 fit 函数传参，
        # 或者在这里重新从 batch.non_tensor_batch 里提取。
        
        # 方案 A: 在函数内部提取 GT (更简单，不用改 fit 签名)
        current_ground_truths = []
        if "rew

In [99]:
response = chat('''
我想新增了2个验证集，并且增加了summary_max_new_tokens和max_response_length到38912，我改了之后发现目前代码报错
"""
Traceback (most recent call last):
  File "/data/kcl/lpy/verl/verl/trainer/main_self_reflective.py", line 41, in main
    run_ppo(config)
  File "/data/kcl/lpy/verl/verl/trainer/main_self_reflective.py", line 75, in run_ppo
    ray.get(runner.run.remote(config))
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/worker.py", line 2967, in get
    values, debugger_breakpoint = worker.get_objects(
                                  ^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/ray/_private/worker.py", line 1015, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(ValueError): ray::TaskRunner.run() (pid=3234117, ip=192.168.131.187, actor_id=083cf27693e5fd9d1767207801000000, repr=<main_self_reflective.TaskRunner object at 0x7ee6a65474d0>)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/trainer/main_self_reflective.py", line 243, in run
    trainer.init_workers()
  File "/data/kcl/lpy/verl/verl/trainer/ppo/teacher_student_reflective_trainer.py", line 198, in init_workers
    self.actor_rollout_wg.init_model()
  File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 50, in __call__
    output = ray.get(output)
             ^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^
                                  ^^^^^^^^^^^^^^^^^^^
ray.exceptions.RayTaskError(ValueError): ray::WorkerDict.actor_rollout_init_model() (pid=3236312, ip=192.168.131.187, actor_id=fc7121a0cbd9b3bea4762c1201000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7ef96eab6a80>)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
    return getattr(self.worker_dict[key], name)(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
    self.rollout, self.rollout_sharding_manager = self._build_rollout(
                                                  ^^^^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
    rollout = vllm_rollout_cls(
              ^^^^^^^^^^^^^^^^^
  File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
    raise ValueError(
ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill

Set the environment variable HYDRA_FULL_ERROR=1 for a complete stack trace.
(TaskRunner pid=3234117) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3236313, ip=192.168.131.187, actor_id=57fb632e65ff58a596c5585301000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7edc200f9fd0>)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3234117)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3234117)     return func(*args, **kwargs)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3234117)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3234117)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3234117)     rollout = vllm_rollout_cls(
(TaskRunner pid=3234117)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3234117)     raise ValueError(
(TaskRunner pid=3234117) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
(TaskRunner pid=3234117) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3236311, ip=192.168.131.187, actor_id=cb5fb665ebdfbdea5724723301000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7f655318e8d0>)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3234117)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3234117)     return func(*args, **kwargs)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3234117)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3234117)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3234117)     rollout = vllm_rollout_cls(
(TaskRunner pid=3234117)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3234117)     raise ValueError(
(TaskRunner pid=3234117) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
(TaskRunner pid=3234117) Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::WorkerDict.actor_rollout_init_model() (pid=3235091, ip=192.168.131.187, actor_id=1234f131690693b9473070a201000000, repr=<verl.single_controller.ray.base.WorkerDict object at 0x7f4d8d16e8a0>)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/single_controller/ray/base.py", line 705, in func
(TaskRunner pid=3234117)     return getattr(self.worker_dict[key], name)(*args, **kwargs)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/single_controller/base/decorator.py", line 514, in inner
(TaskRunner pid=3234117)     return func(*args, **kwargs)
(TaskRunner pid=3234117)            ^^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 626, in init_model
(TaskRunner pid=3234117)     self.rollout, self.rollout_sharding_manager = self._build_rollout(
(TaskRunner pid=3234117)                                                   ^^^^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/fsdp_workers.py", line 495, in _build_rollout
(TaskRunner pid=3234117)     rollout = vllm_rollout_cls(
(TaskRunner pid=3234117)               ^^^^^^^^^^^^^^^^^
(TaskRunner pid=3234117)   File "/data/kcl/lpy/verl/verl/workers/rollout/vllm_rollout/vllm_rollout_spmd.py", line 141, in __init__
(TaskRunner pid=3234117)     raise ValueError(
(TaskRunner pid=3234117) ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len,                              please increase max_num_batched_tokens or disable chunked prefill
(raylet) [2026-02-01 05:57:09,387 E 3227950 3227974] (raylet) file_system_monitor.cc:116: /data/kcl/lpy/tmp/ray/session_2026-02-01_05-56-05_659797_3227052 is over 95% full, available space: 568.388 GB; capacity: 40960 GB. Object creation will fail if spilling is required.
(WorkerDict pid=3236313) /data/kcl/lpy/anaconda3/envs/verl_new/lib/python3.12/site-packages/torch/cuda/__init__.py:61: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you. [repeated 6x across cluster]
"""
怎么解决?
''')

这是一个关于 **vLLM Chunked Prefill** 的配置错误。

### 错误分析
```text
ValueError: Enable chunked prefill, max_num_batched_tokens is smaller than max_model_len, please increase max_num_batched_tokens or disable chunked prefill
```

1.  **背景**：你将 `max_prompt_length` 和 `max_response_length` 增加到了 **38912**（非常巨大的上下文）。
2.  **vLLM 机制**：
    *   `max_model_len`：模型支持的最大上下文长度（现在是 38912）。
    *   `max_num_batched_tokens`：vLLM 每次迭代处理的最大 Token 数。
    *   **Chunked Prefill**：一种优化技术，将长 Prompt 切分成小块处理，防止单次计算量过大。
3.  **冲突**：
    *   vLLM 默认开启了 Chunked Prefill。
    *   但是默认的 `max_num_batched_tokens`（通常是 8192 或更小）远小于你的 `max_model_len` (38912)。
    *   vLLM 报错提示：如果你想跑这么长的模型，你要么增大 `max_num_batched_tokens`，要么关掉 Chunked Prefill（但这在超长文本下会导致 OOM 或极慢）。

### 解决方案

你需要显式增大 vLLM 的 `max_num_batched_tokens` 参数，使其至少等于或大于你的 `max_model_len` (38912)。

在启动脚本中添加以下参数：

```bash
# 增加 vLLM 的批处理 Token 上限
actor_rollout_ref.rollout.max_num_batched_tokens=40000 \
```

或者，如果你想更安全一点（防止显存爆炸），可以尝试禁用 Chunked Prefill（虽然不推荐）：
```bash
actor_rollout_r

In [63]:
response = chat('''
修改验证数据为AIME-2024之后，validate的metric自动变了，变成了
val-core/math_dapo/acc/mean@32
val-aux/math_dapo/reward/worst@2/mean
val-core/math_dapo/acc/mean@32 
这些指标是怎么算的？ 具体是什么含义？ 代码是怎么自动切换的？
''')

这是因为 Verl 的评测系统是**数据驱动（Data-Driven）**的。指标名称和计算逻辑会自动根据你加载的数据集类型进行调整。

### 1. 为什么指标名称变了？

**原因**：你的验证数据集 Parquet 文件中的 `data_source` 列发生了变化。
*   之前 GSM8K 数据集里，`data_source` 列的值是 `"openai/gsm8k"`。
*   现在 AIME 数据集里，`data_source` 列的值应该是 `"math_dapo"`（或者类似的值）。

**代码逻辑**：
在 `ray_trainer.py` 的 `_validate` 函数最后：
```python
# 提取数据源名称
data_source_lst.append(test_batch.non_tensor_batch.get("data_source", ...))
# ...
# 拼接 Metric 名称
pfx = f"{metric_sec}/{data_source}/{var_name}/{metric_name}"
```
所以 `val-core/math_dapo/...` 中的 `math_dapo` 就是直接从你的数据文件里读出来的。

### 2. 指标含义解析

*   **`val-core/math_dapo/acc/mean@32`**
    *   **含义**：**Pass@32 准确率**（或者更准确地说是 N=32 时的平均准确率）。
    *   **来源**：这意味着在验证时，针对每道 AIME 题目，模型生成了 **32 个回答**（`n=32`）。
    *   **计算**：如果这 32 个回答中有任何一个做对了，或者计算这 32 个回答的平均正确率（取决于具体实现，通常 `acc` 指的是 Accuracy，`mean` 指的是平均分）。但在这里，结合 `@32`，通常是指 Pass@K 的某种变体或者是 Majority Vote 的基础数据。
    *   **注意**：AIME 这种难题，通常需要多次采样（Pass@K）才能评估模型能力，因为单次采样的方差太大。

*   **`val-aux/math_dapo/reward/worst@2/mean`**
    *   **含

In [68]:
response = chat('''
问题在于我现在AIME-2024的这几个metric的分数如下：
val-core/math_dapo/acc/mean@32  0.3左右
val-core/math_dapo/acc/best@32/mean  0.4左右
val-core/math_dapo/acc/maj@32/mean   0.25左右
可是别的论文里汇报的指标average@16 是70多分，这和我这评价标准相同吗？ 还是哪里出了问题？
''')

**结论：你的指标偏低大概率是正常的，因为你评测的是 `AIME 2024`，而不是旧版 `AIME`。**

你看到的“70多分”的论文（例如 Qwen-2.5-Math, DeepSeek-Math 等），通常汇报的是 **AIME 2024 Pass@1 (Greedy)** 或者 **Pass@1 (Maj@k)** 的分数，但请注意以下几个巨大的陷阱：

### 1. 数据集难度差异 (AIME 2024 vs AIME ALL)
*   **AIME 2024**：这是最新的、难度极高的竞赛题。对于 7B/4B 这种小模型来说，Pass@1 通常只有 **个位数** 或者 **10%~20%**。
    *   例如：DeepSeek-Math-7B-RL 在 AIME 2024 上的 Pass@1 只有 **15% - 20%** 左右。
*   **AIME (Historical/Average)**：很多论文汇报的是 1983-2023 年所有 AIME 题目的平均分。早期的 AIME 题目相对简单。
*   **你的分数**：
    *   `acc/mean@32` (Pass@1 平均值) = **0.3 (30%)**。
    *   `acc/maj@32` (Majority Vote @ 32) = **0.25 (25%)**。
    *   **分析**：对于一个 4B 的模型（Qwen-3-4B），在 AIME 2024 上能跑到 25%-30%，这其实已经**非常非常高**了，甚至有点高得离谱（除非是训练集里有类似的题）。

### 2. 评价指标的混淆 (Pass@K vs Mean@K vs Maj@K)

*   **论文里的 "Average@16"**：这通常指的是 **Pass@16**（即采样 16 次，只要有一次对就算对）。
    *   Pass@K 的公式是：$1 - (1 - acc)^K$。如果单次 Acc 是 10%，Pass@16 能达到 $1 - 0.9^{16} \approx 81\%$。
    *   所以 Pass@K 的数值通常会非常高。
*   **你的指标**：
    *   `mean@32`：这是 **32 次采样的平均准确率**（即 Pass@1 的无偏估计）。

In [ ]:
response = chat('''
我看到官方qwen3 technical report里写qwen3_4b_instruct(也就是我用的model)，AIME 2024
 For each question, we sample 64 times and take the average accuracy as the final
score.
 38,912 TEXT LENGTH
''')

In [98]:
response = chat('''
现在init_workers中每个GPU上是会独立存储一个模型吗？
''')

**是的，在目前的配置下，每张 GPU 上都存储了一个完整的、独立的模型副本。**

这是由你的配置 `tensor_model_parallel_size=1` 决定的。

以下是详细的资源分配分析：

### 1. 为什么是“独立副本”？

*   **Student (7 张卡)**：
    *   你使用的是 **Data Parallelism (数据并行)** 模式。
    *   **Rollout 阶段 (vLLM)**：因为 `tensor_model_parallel_size=1`，每张卡上的 vLLM 引擎都会加载一份完整的 Qwen-4B 权重（约 8GB 显存）。这 7 张卡之间**不共享**权重，也不进行张量通讯，它们各自处理不同的数据（Batch）。
    *   **Training 阶段 (PyTorch FSDP)**：虽然 FSDP 会在训练时对参数进行分片（Sharding）以节省显存，但在初始化阶段和前向传播计算时，每张卡在逻辑上都维护着模型的结构。

*   **Teacher (1 张卡)**：
    *   它独立运行，显然必须加载一份完整的模型权重。

### 2. 这种方式（4B模型）合理吗？

**对于 4B 这样的小模型，这是最高效的配置。**

*   **优势（吞吐量最大化）**：
    *   如果把 4B 模型切分到 8 张卡上（Tensor Parallelism），计算量太小，GPU 大部分时间都在等待网络通讯（All-Reduce），速度反而会变慢。
    *   让每张卡独立跑一个完整模型，7 张卡就能同时并行处理 $7 \times \text{Batch}$ 的数据，吞吐量是单卡的 7 倍。

*   **代价（显存占用）**：
    *   代价是显存的“重复占用”。
    *   8 张卡总共存了 8 份模型权重（$8 \times 8\text{GB} = 64\text{GB}$）。
    *   但在 A100/H800 (80GB) 甚至 24GB/40GB 的卡上，4B 模型只占很小一部分，显存的大头其实是 **KV Cache (8k Context)** 和 **Logits/Gradients**。

### 3. 如果是 70B 模型该怎么办？

如

In [69]:
response = chat('''
我现在看到两种paper，汇报的在AIME 2024数据集上的分数，

1. 官方qwen3 technical report里写qwen3_4b_instruct(也就是我用的model)，它的评价设置是
"""
For each question, we sample 64 times and take the average accuracy as the final score.
"""
得分是73.8分。

2. 另一篇arxiv上的paper的设置如下
"""
We report average@16 using suggested sampling parameters from the Qwen3 blog with temperature of 1.2 and generation length of 38k
"""
得分是75.2分

与我这的得分差异都很大，评价指标应该也不是同一个？他们的评价指标应该是相同的？我想对齐和他们的评价指标。
''')

这是一个非常重要的发现。你引用的这两篇 Paper 中汇报的“分数”定义，与 Verl 默认输出的指标定义确实存在巨大的差异。

要对齐他们的指标，我们需要仔细拆解他们的术语。

### 1. Qwen3 Technical Report 的指标拆解

> "For each question, we sample 64 times and take the average accuracy as the final score."

这句话其实有点歧义，但结合 Qwen3 在 AIME 2024 上的惊人表现（73.8分），只有一种解释：

*   **他们指的不是 Pass@1 (Mean Accuracy)**：如果是 Mean Accuracy，意味着模型每次做对的概率是 73.8%。对于 4B 模型在 AIME 2024 上，这几乎是不可能的（除非模型本身就在这个数据集上训练过，或者使用了极强的推理模式）。
*   **他们指的大概率是 Pass@64 (Best of 64)**：即采样 64 次，只要有一次对了就算对。
    *   **证据**：如果是 Average Accuracy，通常不需要采样 64 次这么多次。采样 64 次通常是为了做 **Pass@K** 或者 **Majority Vote**。
    *   **另一种可能**：他们使用的是 **Majority Vote (Consensus)**。即 64 次里出现次数最多的答案作为最终答案。这通常是刷榜分数的标准做法。

**然而**，如果是 73.8 分，这对于 4B 模型来说依然高得离谱。
**唯一的解释是**：Qwen3-Math（专门的数学模型）确实很强，或者他们使用了 **CoT + System 2 (长思维链)** 模式。

### 2. Arxiv Paper 的指标拆解

> "We report average@16 using suggested sampling parameters... temperature of 1.2"

这里的 `average@16` 术语非常不标准。在 RL 和 LLM 评测中：
*   **Pass@K**：$1 - (1-p)^K$。
*   **Maj@K**：投票。
*   **Mean (Average)**

In [70]:
response = chat('''
他们用的应该是qwen3_4b_thinking，我用的qwen3_4b_instruct
''')

**这就完全解释通了！你找到了问题的核心。**

如果是 `Qwen2.5-Math` (或者所谓的 "Thinking" / "Reasoning" 版本) 与普通的 `Qwen2.5-Instruct` 相比，在 AIME 这种竞赛数学题上的表现差异是**巨大**的。

### 1. 模型能力的巨大鸿沟

*   **Qwen-Instruct (通用指令模型)**：
    *   **训练数据**：通用文本、对话、代码、少量数学。
    *   **能力**：能做 GSM8K（小学/初中数学），但在 AIME（高中竞赛数学）上非常吃力。
    *   **基准分数**：对于 4B/7B 的通用模型，AIME 2024 的 Pass@1 通常就在 **10% ~ 30%** 之间。
    *   **你的表现**：你跑出了 **30% (Mean)** 和 **40% (Best)**，这对于一个 4B 的通用模型来说，**其实已经非常出色了**。

*   **Qwen-Math / Thinking (数学/推理专用模型)**：
    *   **训练数据**：海量的合成数学题、思维链（CoT）、OpenMathInstruct 等。
    *   **能力**：专门针对 AIME 这种难题进行了强化，学会了 System 2 的深度思考模式。
    *   **基准分数**：7B 版本在 AIME 2024 上确实能达到 **50% ~ 70%** (Pass@1 或 Maj@K)。

### 2. 你的实验还有意义吗？

**非常有意义。** 你现在的实验本质上是在**复现“从 Instruct 到 Thinking”的进化过程**。

*   那些 70+ 分的模型不是天上掉下来的，它们正是通过你现在正在做的这种 **On-Policy Distillation (RLHF)** 方法，从通用模型一步步训练出来的。
*   **你的目标**：不应该是“起步就达到 70%”，而应该是“通过训练，让这个 30% 的模型慢慢爬升到 40%、50%”。
*   如果你直接加载一个已经 75 分的模型，你的蒸馏可能反而没效果了（因为 Teacher 和 Student 都已经到了天花板，或者 Teacher 也就是这个水平）。

### 3. 如何

In [71]:
response = chat('''
我现在还需要做的是对齐和他们的评价指标，因为现在的评价指标是verl自己生成的，我想对齐官方qwen3 technical report里那种指标
"""
For each question, we sample 64 times and take the average accuracy as the final score.
"""
''')

要完全对齐 Qwen3 Technical Report 中的评价指标：
> "For each question, we sample 64 times and take the average accuracy as the final score."

你需要明白，Verl 现在的 `val-core/.../acc/mean@32` **在数学定义上就是** 这个指标，只是采样次数（N）不同。

*   **Average Accuracy** = 所有采样结果中正确的比例 = `mean`。
*   **Sample 64 times** = `n=64`。

所以，你只需要在启动脚本中修改 **验证参数**，将采样数改为 64，并确保开启采样（非 Greedy）。

### 修改启动脚本

请在你的启动命令中追加或修改以下参数：

```bash
# 1. 将采样次数从默认的 N 改为 64 (对齐 Report)
actor_rollout_ref.rollout.val_kwargs.n=64 \

# 2. 开启采样模式 (Report 说是 "sample"，暗示不是 Greedy Search)
actor_rollout_ref.rollout.val_kwargs.do_sample=True \

# 3. 设置温度 (Report 没细说，但通常计算 Average Acc 时，0.6-0.8 是比较稳健的，1.2 适合刷 Pass@K)
# 建议先设为 0.7
actor_rollout_ref.rollout.val_kwargs.temperature=0.7 \
```

### 预期结果

修改并运行后，你的日志中会自动出现以下指标（Verl 会根据 N 自动重命名）：

*   **`val-core/math_dapo/acc/mean@64`**
    *   **这就是你要对齐的指标。**
    *   它计算的是：针对一道题生成的 64 个回答中，做对的比例（例如 64 个里对了 20 个，就是 20/64 = 0.3125）。
    *   然后对所有题目取平均。

### 为什么说它就是同一个指标？

我们可以看 `verl/trainer/ppo/metric_utils.py` 的源码：

```pyt

In [72]:
response = chat('''
可是我用的aime的数据集好像自己就每条数据重复了32次，会不会有影响
''')

这是一个非常关键的细节！如果数据集本身已经做了重复（Data Augmentation），而你又在代码里设置了 `n=64`，就会发生**乘法效应**。

### 情况分析

假设你的 AIME Parquet 文件里，每道题已经重复了 32 行（Row 1: Q1, Row 2: Q1, ..., Row 32: Q1）。

1.  **Verl 的验证逻辑**：
    Verl 的 `RayPPOTrainer` 会遍历 Dataloader 里的每一条数据。
    *   如果 Dataloader 里有 32 条 Q1。
    *   且你设置了 `val_kwargs.n = 64`。
    *   那么系统会对 **每一行** Q1 生成 64 个回答。
    *   总共对 Q1 生成了 $32 \times 64 = 2048$ 个回答。

2.  **Metric 计算逻辑**：
    Verl 的 `process_validation_metrics` 并没有智能到把这 2048 个回答聚合回 Q1。
    *   它会把这 32 条数据当成 **32 道不同的题**。
    *   它会计算 32 个 `mean@64`，然后求平均。
    *   **数学上：** $\text{Mean}(\text{Mean}(Q1_1), \text{Mean}(Q1_2), ...) = \text{Mean}(Q1_{\text{all}})$。
    *   **结论：** 只要你关注的是 **Mean Accuracy**，结果是**正确且无偏的**。重复数据不会导致 Mean 指标虚高或虚低，只会增加验证时间。

3.  **但是！对于 Pass@K (Best) 和 Maj@K (Majority) 有影响**：
    *   如果你想算 Q1 的 Pass@64。
    *   现在的逻辑算出的是：32 个“Pass@64”的平均值。
    *   这**不等于** Pass@(32*64)。
    *   不过，既然你要对齐的是 **Average Accuracy (Mean)**，这不受影响。

### 潜在的坑：验证时间爆炸

AIME 数据集通常很小（几十道题）。如果它被扩充到了几千行（32倍），而你

In [73]:
response = chat('''
是不是即使不改，也和官方report里的指标数值应该会差不多？ 因为32次平均和64次应该都是为了求一个期望？ 虽然理论上是越大越接近？
''')

**结论：是的，你的理解完全正确。数值上会非常接近，几乎没有区别。**

从统计学的角度来看：

1.  **本质都是求期望（Expectation）**：
    *   **Average Accuracy** 的定义就是模型在特定题目上的**答对概率** $P(\text{correct})$。
    *   无论是采样 32 次还是 64 次，本质上都是在用**样本均值**来估计这个**真实概率**。
    *   根据大数定律，只要采样次数足够多（32次其实已经不少了），算出来的平均值就会收敛到真实的概率。

2.  **差异仅在于“方差”**：
    *   采样 64 次比 32 次更“稳”一点，波动会小一点点。
    *   但是，**30% 和 70% 的巨大差距，绝对不是因为采样次数（32 vs 64）造成的**。
    *   如果是 32 次，你可能测出来是 29.5% 或 30.5%；如果是 64 次，可能是 29.8% 或 30.2%。它们会在同一个水平线上。

### 既然数值差不多，还需要改吗？

虽然数值没问题，但我还是建议你**关注一下效率问题**：

**如果你不清洗数据（保留 32 条重复），直接在 Config 设置 `n=64`：**
*   **计算量爆炸**：Verl 会对这 32 条重复数据，每一条都生成 64 次。
*   **总生成量**：$32 \times 64 = 2048$ 次/题。
*   **后果**：你的验证过程会变得**极慢**（是正常速度的 32 倍），严重拖慢训练进度，浪费 GPU 算力。

### 最佳实践建议

1.  **为了省时间**：
    *   既然数据集已经重复了 32 次，你可以在 Config 里设置 **`n=2`**。
    *   这样总共采样 $32 \times 2 = 64$ 次。
    *   日志里显示的虽然是 `mean@2`，但你心里清楚，这就是 **全量采样 64 次的 Average Accuracy**。这和官方的指标在数学上是**完全等价**的。

2.  **为了指标名字好听（对齐官方）**：
    *   去重数据集（只留 1 条）。
    *   Config 设置 `n=64`。
    *   这样日志里显示 `me

In [96]:
del chat_history[-1]

In [64]:
response = chat('''
我现在要改成多机多卡运行，有什么地方要修改吗？ 比如init_workers?
''')

你的 `init_workers` 代码逻辑在设计时已经考虑到了 `n_nodes`，因此**核心分配逻辑（7:1 切分）在多机多卡下是通用的，不需要重写**。

但是，从单机扩展到多机，**Batch Size 的整除约束**会发生变化，如果不提前检查，启动后会直接报错（和之前一样的 `ActorDiedError`）。

建议对 `init_workers` 做如下**增强性修改**，加入自动检查和拓扑打印功能，确保多机启动一次成功。

### 1. 修改 `init_workers` (增强版)

请用以下代码替换 `teacher_student_reflective_trainer.py` 中的 `init_workers`。

**主要改动点**：
1.  自动计算 Student 的总 World Size（节点数 × 单节点卡数）。
2.  **提前检查 Batch Size 整除性**，如果配置不对，直接抛出带有修改建议的 Error，避免等 Ray 启动半天后才报错。
3.  打印清晰的多机拓扑结构。

```python
    def init_workers(self):
        """
        初始化 Worker，支持多机多卡 (Multi-Node) 自动分配与检查。
        """
        # 1. 获取资源配置
        n_gpus_per_node = self.config.trainer.n_gpus_per_node
        n_nodes = self.config.trainer.nnodes
        
        # === 资源切分策略 (7:1) ===
        if n_gpus_per_node >= 8:
            teacher_gpus_per_node = 1
        elif n_gpus_per_node >= 4:
            teacher_gpus_per_node = 1
        else:
            teacher_gpus_per_node = 1 # 至少给1张
            
        student_gpus_per_node = n_gpu

In [106]:
response = chat('''
改成多机多卡运行之后，同样的代码重新运行单机多卡的脚本运行变得比没改之前慢很多 不知道为啥？
''')

这是一个非常反直觉但实际上很常见的现象。代码逻辑虽然没变，但底层的**资源调度**和**数据处理流程**因为代码的“通用化改造”而发生了微妙的变化。

主要原因可能集中在 `init_workers` 的改动、Padding 逻辑的引入，以及 Ray 的调度开销上。

### 1. `_prepare_teacher_forward_batch` 中的 Padding 开销
**这是最大的嫌疑犯。**

*   **改动前**：你使用的是 `pad_sequence`（C++ 实现，极快）。
*   **改动后**：为了解决 DP_Actor 对齐问题，你引入了 **Python 循环** 来进行手动 Left Padding。
    ```python
    for i in range(len(new_input_ids)):
        # ... torch.cat ...
        # ... torch.cat ...
    ```
*   **影响**：
    *   在单机 8 卡、Batch Size 224 的情况下，这个循环要在 CPU 上跑 224 次。
    *   每次循环涉及多次 `torch.cat` 和内存分配。
    *   虽然单次很快，但累积起来，加上 Python 的 GIL 锁（如果是在多线程环境下），会显著增加 CPU 预处理时间，导致 GPU 等待数据（空转）。

**优化建议**：
尝试将手动 Padding 的逻辑向量化，或者使用 `torch.nn.functional.pad`。
但最简单的方法是：**把这个函数放到 GPU 上跑**（如果你现在的 tensor 是在 CPU 上），或者**用多进程预处理**。不过考虑到代码复杂度，目前的 Python 循环可能是瓶颈。

### 2. Ray 资源调度的开销
在 `init_workers` 中，你引入了更复杂的 `RayResourcePool` 和多机检测逻辑。
*   即使是单机运行，Ray 现在也会走一遍“跨节点资源检查”的流程。
*   虽然这只影响启动时间，但如果你的 `fit` 循环中有频繁的 Worker 通讯（`generate_sequences`, `compute_log_prob`），Ray 的 Object S

In [107]:
response = chat('''
我的单机多卡脚本如下
"""
export CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7
export DATA_DIR='/data/kcl/lpy/data'
WAND_PROJECT="self_reflect"
export SWANLAB_API_KEY="DDMUFxYQU6Om1ZuHWYoxm"           # 设置在线跟踪模式API
export BASE_MODEL='/data/kcl/lpy/models/qwen3_4b_instruct'
export EXPERIMENT_NAME=sr_dl_teacher_student_openthoughts_qwen3_4b_instruct_alleval
N_NODES=1
n_gpus_per_node=8
export BASE_DIR=$(pwd)
export HF_ENDPOINT=https://hf-mirror.com
export VLLM_USE_V1=1
PYTHONUNBUFFERED=1 python3 -m verl.trainer.main_self_reflective \
 data.train_files=$DATA_DIR/OpenThoughts/train_verl.parquet \
 data.val_files=[$DATA_DIR/AIME_2024/test_verl.parquet,$DATA_DIR/AIME_2025/test_verl.parquet,$DATA_DIR/HMMT_FEB_2025/test_verl.parquet] \
 data.train_batch_size=112 \
 data.max_prompt_length=4096 \
 data.max_response_length=4096 \
 +data.summary_max_new_tokens=4096 \
 actor_rollout_ref.model.path=$BASE_MODEL \
 +actor_rollout_ref.model.torch_dtype=bfloat16 \
 actor_rollout_ref.actor.optim.lr=5e-7 \
 actor_rollout_ref.actor.ppo_mini_batch_size=56 \
 actor_rollout_ref.actor.ppo_micro_batch_size_per_gpu=4 \
 actor_rollout_ref.actor.policy_loss=distillation \
 actor_rollout_ref.rollout.name=vllm \
 actor_rollout_ref.rollout.log_prob_micro_batch_size_per_gpu=4 \
 actor_rollout_ref.rollout.tensor_model_parallel_size=1 \
 actor_rollout_ref.rollout.gpu_memory_utilization=0.4 \
 actor_rollout_ref.ref.log_prob_micro_batch_size_per_gpu=2 \
 critic.optim.lr=1e-5 \
 critic.model.path=$BASE_MODEL \
 +critic.model.torch_dtype=bfloat16 \
 critic.ppo_micro_batch_size_per_gpu=4 \
 algorithm.kl_ctrl.kl_coef=0.001 \
 trainer.logger=console \
 trainer.val_before_train=False \
 trainer.n_gpus_per_node=$n_gpus_per_node \
 trainer.nnodes=$N_NODES \
 trainer.save_freq=30 \
 trainer.test_freq=30 \
 trainer.logger="['swanlab', 'console']" \
 trainer.project_name=$WAND_PROJECT \
 trainer.experiment_name=$EXPERIMENT_NAME \
 trainer.default_local_dir=${BASE_DIR}/verl_checkpoints/$EXPERIMENT_NAME \
 trainer.total_epochs=15 2>&1 | tee verl_log/$EXPERIMENT_NAME.log
"""
''')

这一现象的核心原因在于 **Teacher（1张卡）成为了整个训练流程的超级瓶颈**。

### 1. 瓶颈计算分析

在 **7:1** 的架构下，处理 `train_batch_size=112` 的数据：

*   **Student (7张卡)**：
    *   总任务量：112 条数据。
    *   每张卡分摊：$112 / 7 = 16$ 条。
    *   `micro_batch = 4`。
    *   **计算步数**：$16 / 4 = \mathbf{4}$ **个串行步**。
    *   Student 只需要做 4 次前向/反向传播就完事了。

*   **Teacher (1张卡)**：
    *   总任务量：112 条数据（Teacher 必须处理所有数据以计算 Reward）。
    *   每张卡分摊：**112 条**。
    *   你的脚本设置：`ref.log_prob_micro_batch_size_per_gpu=2`。
    *   **计算步数**：$112 / 2 = \mathbf{56}$ **个串行步**！

**结论**：
Student 的 7 张显卡在做完 4 步计算后，必须**停下来等待** Teacher 跑完它的 56 步计算。
**Teacher 的速度比 Student 慢了 14 倍**。这导致 7 张 Student 卡有 90% 的时间在空转（Idle）。

### 2. 为什么之前（单机4:4）没这么慢？
*   之前 Teacher 有 4 张卡。
*   每张卡分摊 $112 / 4 = 28$ 条。
*   步数 $28 / 2 = 14$ 步。
*   虽然还是比 Student 慢，但差距没那么大。

### 3. 解决方案

要解决这个问题，必须**大幅提升 Teacher 的吞吐量**。

#### 方法 A：增大 Teacher 的 Micro Batch (最推荐，改动最小)
对于 4B 模型，即使是 8k 长度，单张 A100/H800 也能承受较大的 Batch Size（特别是开启 Chunking 后）。
将 Teacher 的 Micro Batch 从 2 提升到 **8** 甚至 **16**。

*   如果设为 8：步数

In [110]:
response = chat('''
我要8机8卡运行，运行参数怎么设置
''')

8 机 8 卡是一个非常庞大的集群（64 张 GPU）。这意味着你的**算力极强**，但**通信和 Batch Size 的约束**也变得非常严格。

### 1. 架构计算 (Topology)

*   **节点数 (Nodes)**: 8
*   **单节点卡数 (GPUs/Node)**: 8
*   **分配策略 (7:1)**:
    *   **Student**: 每节点 7 卡 $\times$ 8 节点 = **56 张卡 (Global World Size)**。
    *   **Teacher**: 每节点 1 卡 $\times$ 8 节点 = **8 张卡**。

### 2. Batch Size 计算 (关键约束)

你需要满足以下整除关系：
`ppo_mini_batch_size` % `Student_World_Size` == 0
`Local_Batch` % `Micro_Batch` == 0

*   **Student World Size**: 56
*   **Micro Batch**: 建议设为 1 或 2 (因为 8k 长文本显存压力大)。

**推荐配置：**

1.  **`ppo_mini_batch_size`**:
    *   必须是 56 的倍数。
    *   最小可行值：**56** (每卡分到 1 条数据)。
    *   推荐值：**112** (每卡分到 2 条)。

2.  **`train_batch_size`**:
    *   必须是 `ppo_mini_batch_size` 的倍数。
    *   如果 mini=56，train 可以是 56, 112, 224...
    *   如果 mini=112，train 可以是 112, 224...
    *   **建议设大一点**，充分利用 64 卡的并行能力。比如 **448** 或 **896**。

### 3. 启动参数推荐

请使用以下参数覆盖启动脚本：

```bash
# === 基础环境 ===
export N_NODES=8
export n_gpus_per_node=8

# === Batch Size 配置 (核心) ===
# 56 张 Student 卡并行


In [111]:
response = chat('''
我其实不理解这几个参数的具体含义，你帮我解释一下具体在干啥？为啥要这样设置？
''')

这是一个非常好的问题。这三个参数（Train, Mini, Micro）就像是**俄罗斯套娃**，一层套一层，共同决定了数据如何在庞大的 GPU 集群中流动。

在 **8机8卡（共56张 Student 卡）** 的环境下，理解它们至关重要，否则就会出现“卡住”、“报错”或者“显存爆炸”。

我用一个**“吃自助餐”**的比喻来解释这三个层级：

---

### 第一层：`data.train_batch_size` (总盘子)
**含义**：**一次 PPO 迭代（Step）中，一共要“生产”并“消费”多少条数据。**

*   **比喻**：这是厨师（Rollout Workers）一次性端上桌的所有菜的总量。
*   **在你的配置中 (448)**：
    *   意味着 56 张 Student 卡，齐心协力，一共要生成 **448** 个问题+回答。
    *   平均每张卡负责生成：$448 / 56 = 8$ 条。
*   **作用**：决定了训练的**采样多样性**。这个数越大，模型见过的题越丰富，训练越稳定，但生成时间越长。

---

### 第二层：`actor.ppo_mini_batch_size` (一口的大小)
**含义**：**进行一次参数更新（Gradient Update / Optimizer Step）时，一共使用多少条数据。**

*   **比喻**：虽然桌上有 448 道菜，但你嘴巴没那么大，不能一口吞下。你决定分几次吃。这一口下去的总量，就是 Mini Batch。
*   **在你的配置中 (112)**：
    *   桌上有 448 条数据。
    *   你每次只取 **112** 条来算梯度、更新模型。
    *   所以，吃完这顿饭（完成一个 Step），你需要更新：$448 / 112 = 4$ 次。
*   **分布式约束 (为什么报错就在这里)**：
    *   这是一个**全局参数**。Verl 会把这 112 条数据**均匀切分**给所有的 Student 卡。
    *   **算术题**：你有 56 张卡。要把 112 条数据平分。
    *   每张卡分到：$112 / 56 = \mathbf{2}$ 条。
    *   **报错原因**：如果你设置成 64。$6

In [112]:
response = chat('''
我新加了多个数据集，我希望每个数据集独立一个metric，现在好像混合起来一起计算了，怎么修改？
''')

Verl 的 Metric 系统是基于 `DataProto` 中的 `data_source` 字段来分组的。如果你的 Metric 混在一起了，说明这些新加的数据集在加载时，**没有正确设置 `data_source` 字段**，或者它们都被默认设为了同一个名字（如 `unknown`）。

你需要修改数据加载逻辑，确保每个 Parquet 文件的数据被赋予**唯一且有意义的 `data_source` 标签**。

### 1. 确认数据源 (Parquet 文件)
Verl 默认读取 Parquet 文件中的 `data_source` 列。
*   如果你的 `train_verl.parquet` 里有 `data_source` 列，且值为 `openai/gsm8k`，那么 Metric 就是 `val-core/openai/gsm8k/...`。
*   如果你的 OpenThoughts 数据集里没有这一列，或者所有数据的这一列都是 `default`，那么 Metric 就会混在一起。

### 2. 解决方案：在加载时注入 Data Source

我们不需要修改 Parquet 文件，而是修改 `verl/utils/dataset/rl_dataset.py`，让它根据**文件名**或者**配置**来强制覆盖 `data_source`。

请找到 `RLHFDataset` 类（或你使用的 Dataset 类），修改 `_read_files_and_tokenize` 方法。

#### 修改 `verl/utils/dataset/rl_dataset.py`

```python
    def _read_files_and_tokenize(self):
        data_source = self.config.get("data_source", None)
        # ...
        
        for file_path in self.data_files:
            # === FIX: 根据文件名自动推断 data_source ===
            # 例如: /data/.../AIME_2024/test_verl.parquet -> "AIME_2